In [1]:
import pandas as pd
import numpy as np
import openpyxl
from lower_case_dict import d_dict as D_dict
from lower_case_dict import d_describe as D_describe
from lower_case_dict import D_ambito_a_rama
from lower_case_dict import D_ocupaciones
from lower_case_dict import D_cnae_a_num
from upper_case_dict_Master19 import diccionario 
from traduce import leyenda
from traduce import traduce
import traduce as tr
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300

# Objetivo: Inspección y ajustes

**Objetivo**: Inspección del dataset, imputación de valores perdidos debido a los itinerarios de la encuesta y creación de variables nuevas para análisis descriptivo y predictivo.   

De esta etapa del proceso se obtienen **dos datasets**:    

1- Un dataset con las **variables originales** (designadas con nombres en mayúsculas) a las que, a lo sumo, se aplica alguna de las transformaciones siguientes:  
- Imputación de valores perdidos que pueden deducirse de los itinerarios de la encuesta.  
- Sustitución de las no respuestas y las respuestas NS/NC por el valor None.  

2- Un **dataset nuevo** formado por algunas de las variables originales escogidas como candidatas a formar parte de los modelos descriptivos o predictivos y por algunas variables nuevas creadas a partir de las originales. Las variables de este dataset se designan por nombres en minúsculas para facilitar su identificación. Algunas de las transformaciones que dan lugar a este conjunto de variables son:  
- Sustitución del valor asociado a la respuesta No en las preguntas de respuesta Sí o No: se cambia el valor 2 asociado al No en el dataset original por el valor 0.  
- Creación de variables binarias para cada una de las categorías de algunas de las variables cualitativas del dataset original.  
- Creación de variables nuevas para información que puede deducirse de una o varias de las columnas del dataset original.  
- Creación de variables con información del dataset (año de la encuesta, población objeto de muestreo, etc.) con el fin de que sea posible identificar la procedencia de cada observación del el dataset que resulte de reunir todos los empleados en este estudio.  

En esta etapa **no se imputan valores a las no respuestas o a las respuestas NS/NC** que no puedan deducirse de los itinerarios de la encuesta. Solo se sustituyen por el valor None. Se procede así para evitar que los valores imputados interfieran en los resultados del análisis descriptivo. 

# Diccionarios de variables categóricas

Los **diccionarios de las variables originales** en la encuesta y los definidos para las **variables nuevas** (creadas inicialmente para el dataset de graduados 2019) se importan, con algunas excepciones, del **módulo lower_case_dict**:  

- **D_dict**; diccionarios de traducción de las variables categóricas. Construidos de acuerdo con los códigos facilitados en el archivo de diseño de la encuesta 'dr_EILU_GRAD_2019.xlsx' y con los definidos para las variables nuevas creadas en la inspeccion de la encuesta de graduados 2019. Para cada variable: D_dict[variable][código_categórico]=interpretación_código.  

- **D_describe**; diccionario de interpretación de las variables originales y nuevas: D_describe[variable]=interpretación_variable.  

Las **excepciones** son algunos diccionnarios que no son iguales para las encuestas a graduados y titulados de máster 2019. Estos se importan a través de la función diccionario definida en el **módulo upper_case_dict_Master19**, una adaptación del módulo módulo upper_case_dict ajustada a la encuesta de titulados de máster 2019. Se invocan y aplican caso a caso cuando procede. 

En el proceso de inspección se redefinen algunas variables originales y se definen otras nuevas. Estas se añaden a los diccionarios D_dict y D_describe para ser recuperados con el módulo lower_case_dict.  

También se importa del módulo lower_case_dict el diccionario D_ambito_a_rama que asocia los ámbitos de estudios a su rama de conocimiento. 

### Funciones de traducción

Se importan las funciones traduce y leyenda del módulo traduce. 

# Recuperación del dataset
Dataset 'Encuesta_insercion_laboral_titulados_universitarios_2019' en formato csv.   

Enlace a fuente (INE): https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176991&menu=resultados&idp=1254735573113#!tabs-1254736195339

In [2]:
ruta_dataset='data/EILU_MAST_2019.csv'

In [3]:
df_raw=pd.read_csv(ruta_dataset,sep='\t',low_memory=False)

In [4]:
df=df_raw.copy()

# Declaración de variables globales en en módulo traduce (tr):
tr.D_dict=D_dict
tr.df=df

In [5]:
df.shape

(11483, 291)

In [6]:
for n,col in enumerate(df.columns):
    print(n,col,df[col].dtype)

0 IDENT int64
1 SEXO int64
2 EDAD int64
3 NACIO int64
4 NACIO1 object
5 NACIO2 object
6 PAIS_NACI int64
7 TITU int64
8 RAMA int64
9 AMBITO int64
10 T_UNIV int64
11 DISCA int64
12 PROXY int64
13 LUG_RES_PAIS int64
14 PAIS_NAC_PADRE int64
15 ESTUDIOS_PADRE int64
16 PAIS_NAC_MADRE int64
17 ESTUDIOS_MADRE int64
18 EST_B1 int64
19 EST_B2_1 object
20 EST_B2_2 object
21 EST_B2_3 object
22 EST_B2_4 object
23 EST_B2_5 object
24 EST_M1 int64
25 EST_M2 object
26 EST_M3 object
27 EST_M4 object
28 EST_MES int64
29 EST_B8 int64
30 SAT1 int64
31 SAT2 object
32 EST_B11_1 int64
33 EST_B11_2 int64
34 EST_B11_3 int64
35 EST_B11_4 int64
36 EST_B11_5 int64
37 EST_B11_6 int64
38 EST_B12 object
39 EST_B13_AMB1 object
40 EST_B14_1 object
41 EST_B15_1 object
42 EST_B13_AMB2 object
43 EST_B14_2 object
44 EST_B15_2 object
45 EST_B13_AMB3 object
46 EST_B14_3 object
47 EST_B15_3 object
48 EST_B16 object
49 EST_B17_M1 object
50 EST_B18_1 object
51 EST_B19_1 object
52 EST_B17_M2 object
53 EST_B18_2 object
54 EST_B19

# Análisis exploratorio: funciones 

Funciones definidas para facilitar la inspección de variables categóricas: distribución de frecuencias relativas, distribuciones condicionadas, etc.    

In [7]:
def proporciones(col,dropna=False):
    """ Frecuencias relativas de los valores que toma una variable categórica en orden ascendente.
    Si dropna=False, incluye los valores NaN."""
    out=df[col].value_counts(normalize=True,ascending=False,dropna=dropna)
    out=pd.DataFrame(out)
    return out

In [8]:
# Lista de variables originales. Permite recuperar el dataset original después de imputar los valores perdidos ...
# ... como consecuencia de los itinerarios de la encuesta. 
v_originales=df.columns.tolist()
# Lista a la que se agregarán las variables candidatas a ser incluidas en el modelo predictivo
v_modelos=list()
# Listas de variables literales (interpretación de códigos de variables categóricas)
v_originales_L=list()
v_modelos_L=list()

# Declaración de variables globales en en módulo traduce (tr):
tr.v_originales_L=v_originales_L
tr.v_modelos=v_modelos
tr.v_modelos_L=v_modelos_L

In [9]:
def no_respuesta(col,blank=' ',nsnc='9',tipo=None):
    """ Sustituye respuestas en blanco y NS/NC por None y establece tipo 'float' si es posible'.
    """
    if tipo=='int':
        blank=None
        nsnc=int(nsnc)
    try: df[col].replace({blank:None,nsnc:None},inplace=True)
    except: 
        serie=df[col].astype('O') # Evita conflictos cuando se aplica más de una vez
        serie.replace({blank:None,str(nsnc):None},inplace=True)
    try:df[col]=df[col].astype('float')
    except:pass # Algunas variables categóricas tienen valores no numéricos (e.g. CNAE)
    try:traduce(col,check=False)
    except:pass # Para variables no categóricas (sin diccionario)
    print (df[col].sort_values().unique())
    return

In [10]:
def redefine(col,nuevo=True,cat=True,si_no=False):
    """ Crea una columna denominada 'nuevo' (por defecto, el nombre original en minúsculas) a partir 
    de la columna 'col', la añade a la lista de variables para los modelos descriptivos y predictivos. 
    Si 'cat' True, asocia la variable a su diccionario, traduce y conserva la variable traducida.
    Si si_no=True, modifica el valor asociado a respuesta No (0 en lugar de 2).  
    """
    if nuevo==True:
        nuevo=col.lower()
    df[nuevo]=df[col]
    #D_describe[nuevo]=D_describe[col]
    if str(nuevo) not in v_modelos: 
        v_modelos.append(nuevo)
    if cat==True: 
        D_dict[nuevo]=D_dict[col]
        traduce(nuevo,check=False)
        if str(nuevo)+'_' not in v_modelos_L: 
            v_modelos_L.append(str(nuevo)+'_')
    if si_no==True: 
        df[nuevo].replace({2:0},inplace=True)
        D_dict[nuevo]=D_dict['si_no']
        traduce(nuevo)
    return

In [11]:
def a_lista(col):
    if col==col.lower() and col==col.strip('_') and str(col) not in v_modelos:
        v_modelos.append(col)   
    return

In [12]:
def print_listas():
    print ('v_originales_L:','\t',v_originales_L)
    print ('v_modelos:','\t',v_modelos,'\nv_modelos_L:','\t',v_modelos_L)
    return

In [13]:
def reset(col_or_list):
    """ Argumentos: columna o lista de columnas.
    Elimina la variable col y su literal col_ de las listas de variables redefinidas o literales.
    Si col es una variable en el dataset original, reestablece sus valores iniciales. 
    Si el argumento es una lista de columnas, aplica reset a cada columna de la lista.
    """
    global v_originales_L, v_modelos, v_modelos_L
    if type(col_or_list)!=list:
        L=[col_or_list]
    else: 
        L=col_or_list
    for col in L:
        if col==col.upper() and str(col)+'_' in v_originales_L:
            v_originales_L.remove(str(col)+'_')
        if col==col.lower() and str(col) in v_modelos:
            v_modelos.remove(str(col))
        if col==col.lower() and str(col)+'_' in LM_:
            v_modelos_L.remove(str(col)+'_')
        try:
            df[col]=df_raw[col]
            del df[str(col)+'_']
        except:pass
    return

# BLOQUE A. DATOS PERSONALES Y SOCIODEMOGRÁFICOS

## 'IDENT' 
**Índice** único del entrevistado.

In [14]:
df['IDENT'].nunique()

11483

## Nueva variable: 'año' 
**Año** en el que se realizó la encuesta.  
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets.

In [15]:
df['año']=2019 
df['año'].unique() # Comprobación

array([2019])

In [16]:
a_lista('año')

## Nueva variable: 'est_encuesta' 
**Estudios** oficiales objeto de la encuesta (los que definen la población objeto de muestreo).   
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets.

In [17]:
df['est_encuesta']=2 
D_dict['est_encuesta'][2]='Máster universitario'  

In [18]:
a_lista('est_encuesta')

## 'SEXO' (redefinida como 'sexo')
**Sexo** del entrevistado.  

**'fem'**: 1= Mujer, 0=Hombre. 

In [19]:
df['SEXO'].unique() # Sin valores no informados

array([1, 2])

In [20]:
traduce('SEXO')

     aux_traduce
          unique
SEXO            
1       [Hombre]
2        [Mujer]


In [21]:
df['sexo']=0
df['sexo'].mask(df['SEXO']==2,1,inplace=True)
df[['SEXO_','sexo']].groupby('sexo').agg(['unique']) # Comprobación

,SEXO_
,unique
sexo,
0,[Hombre]
1,[Mujer]


In [22]:
traduce('sexo')

     aux_traduce
          unique
sexo            
0       [Hombre]
1        [Mujer]


## 'EDAD' (redefinida como 'edad')
**Edad** del entrevistado.

In [23]:
D_describe['EDAD']

'Grupo de edad en el momento de la encuesta'

In [24]:
df['EDAD'].unique() # Sin valores no informados

array([1, 2, 3])

In [25]:
traduce('EDAD')

               aux_traduce
                    unique
EDAD                      
1     [Menores de 30 años]
2        [De 30 a 34 años]
3       [De 35 y más años]


In [26]:
redefine('EDAD')

## 'NACIO'
**Nacionalidad** del entrevistado.

In [27]:
df['NACIO'].unique() # Sin valores no informados

array([1, 3, 2])

In [28]:
traduce('NACIO')

               aux_traduce
                    unique
NACIO                     
1               [Española]
2        [Española y otra]
3      [Otra nacionalidad]


## Nueva variable: 'nacionalidad'
**Nacionalidad española** u otra: 0=Nacionalidad no española, 1=Nacionalidad española. 

In [29]:
df['nacionalidad']=1
df['nacionalidad'].mask(df['NACIO']==3,0,inplace=True)
df[['nacionalidad','NACIO']].groupby(['NACIO']).agg(['unique']) # Comprobación

,nacionalidad
,unique
NACIO,
1,[1]
2,[1]
3,[0]


In [30]:
traduce('nacionalidad')

                             aux_traduce
                                  unique
nacionalidad                            
0             [Nacionalidad no española]
1                [Nacionalidad española]


## 'NACIO1'

(Solo a quienes declaran nacionalidad distinta de 'Española' -respuesta distinta a 1 en 'NACIO'-)  

**País de primera nacionalidad**.

In [31]:
df['NACIO1'].unique() # Valores no informados: ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [32]:
traduce('NACIO1')

                     aux_traduce
                          unique
NACIO1                          
                  [No aplicable]
1             [Un país de la UE]
2       [Un país fuera de la UE]
9                        [NS/NC]


In [33]:
df[['NACIO_','NACIO1_']].groupby('NACIO_',dropna=False).agg(['unique'])
# No informado para quienes declararon solo nacionalidad española en 'NACIO'

,NACIO1_
,unique
NACIO_,
Española,[No aplicable]
Española y otra,"[Un país de la UE, Un país fuera de la UE, NS/NC]"
Otra nacionalidad,"[Un país fuera de la UE, Un país de la UE, NS/NC]"


In [34]:
# Se imputa provisionalmente None a no informados
no_respuesta('NACIO1')

[ 1.  2. nan]


In [35]:
# Se imputa nacionalidad de un país de la UE (1) a quien declaró solo nacionalidad española en variable 'NACIO'
# (Pueden quedar nacionales españoles con doble nacionalidad sin identificar como EU en NACIO1)
df['NACIO1'].mask(df['NACIO']==1,1,inplace=True)
traduce('NACIO1',check=False)
df[['NACIO_','NACIO1_']].groupby('NACIO_').agg(['unique']) # Comprobación

,NACIO1_
,unique
NACIO_,
Española,[Un país de la UE]
Española y otra,"[Un país de la UE, Un país fuera de la UE, nan]"
Otra nacionalidad,"[Un país fuera de la UE, Un país de la UE, nan]"


## 'NACIO2'

(Solo a algunos de los que declaran 'Otra nacionalidad' -respuesta 3 en variable 'NACIO')  

**País de segunda nacionalidad**.

In [36]:
df['NACIO2'].unique() # No informados como ' '

array([' ', '2', '1'], dtype=object)

In [37]:
traduce('NACIO2')

                     aux_traduce
                          unique
NACIO2                          
                  [No aplicable]
1             [Un país de la UE]
2       [Un país fuera de la UE]


In [38]:
df[['NACIO_','NACIO1_','NACIO2_']].groupby(['NACIO_','NACIO1_']).agg(['unique'])
# Sin respuesta para quienes declaran 'nacionalidad española' y 'nacionalidad española y otra' (1 y 2 en 'NACIO').
# No se consigue afinar por completo el criterio para establecer cuándo es 'No aplicable'.

NACIO2_
                                                                                            unique
NACIO_            NACIO1_                                                                         
Española          Un país de la UE                                                  [No aplicable]
Española y otra   Un país de la UE                                                  [No aplicable]
                  Un país fuera de la UE                                            [No aplicable]
Otra nacionalidad Un país de la UE                          [No aplicable, Un país fuera de la UE]
                  Un país fuera de la UE  [No aplicable, Un país de la UE, Un país fuera de la UE]

In [39]:
# Provisionalmente se imputa None a las no respuestas:
no_respuesta('NACIO2')

[ 1.  2. nan]


In [40]:
# Se imputa 'NACIO2'=1 (UE)  a quienes declararon 'nacionalidad española y otra' (2 en 'NACIO') y la primera ...
# ... no es nacionalidad EU ('NACIO1' distinta de 1), ya que la española es su segunda nacionalidad:
df['NACIO2'].mask((df['NACIO']==2)&(df['NACIO1']!=1),1,inplace=True)
traduce('NACIO2',check=False)
df[['NACIO_','NACIO2_']].groupby('NACIO_').agg(['unique']) # Comprobación: 

,NACIO2_
,unique
NACIO_,
Española,[nan]
Española y otra,"[nan, Un país de la UE]"
Otra nacionalidad,"[nan, Un país fuera de la UE, Un país de la UE]"


## 'PAIS_NACI'
**País de nacimiento**.

In [41]:
df['PAIS_NACI'].unique() # Sin valores no informados

array([1, 2, 3])

In [42]:
traduce('PAIS_NACI')

                          aux_traduce
                               unique
PAIS_NACI                            
1                            [España]
2                [Otro país de la UE]
3          [Otro país fuera de la UE]


## Nueva variable 'eu27'
**Nacido en EU27**: 1=Nacido en EU27, 0=No nacido en EU27

In [43]:
df['eu27']=0
df['eu27'].mask((df['PAIS_NACI']==1)|(df['PAIS_NACI']==2),1,inplace=True)
df[['eu27','PAIS_NACI_']].groupby('eu27').agg(['unique']) # Comprobación

,PAIS_NACI_
,unique
eu27,
0,[Otro país fuera de la UE]
1,"[España, Otro país de la UE]"


In [44]:
traduce('eu27')

              aux_traduce
                   unique
eu27                     
0     [No nacido en EU27]
1        [Nacido en EU27]


## 'TITU' (redefinida como 'titulo_m1')
**Titulación** de máster universitario  

Nota: el diccionario asociado es el propio de los títulos de máster, definido en el fichero dr_EILU_MAST_2019.xlsx. 

In [45]:
D_describe['TITU']

'Titulación'

In [46]:
df['TITU'].sort_values().unique() # Sin valores no informados

array([ 111,  113,  114,  119,  211,  213,  215,  219,  221,  222,  223,
        229,  231,  232,  239,  311,  312,  313,  314,  319,  321,  322,
        411,  412,  413,  414,  416,  419,  421,  511,  512,  519,  521,
        522,  531,  532,  533,  541,  542,  599,  612,  613,  619,  711,
        712,  713,  714,  715,  716,  719,  721,  722,  731,  732,  811,
        821,  831,  841,  911,  912,  913,  914,  915,  916,  919,  923,
       1013, 1014, 1015, 1022, 1032, 1041])

In [47]:
df['TITU'].nunique()

72

In [48]:
# Ajuste en el diccionario asociado: 
D_dict['TITU']=diccionario('TTITULA')

In [49]:
traduce('TITU')

                                                                  aux_traduce
                                                                       unique
TITU                                                                         
111                                                [Ciencias de la Educación]
113                             [Formación de docentes de enseñanza primaria]
114   [Formación de docentes de educación secundaria y formación profesional]
119                                              [Educación (Otros estudios)]
211                         [Técnicas audiovisuales y medios de comunicación]
...                                                                       ...
1014                                       [Actividades físicas y deportivas]
1015                                                 [Viajes, turismo y ocio]
1022                                              [Salud y seguridad laboral]
1032          [Protección de la propiedad y las personas y enseñ

In [50]:
redefine('TITU','titulo_m1')

In [51]:
# Se añade indicador de título de máster para distinguirlos de títulos de grado con idéntica denominación: 
df['titulo_m1_']='Máster '+df['titulo_m1_']

In [52]:
proporciones('titulo_m1_').head(30)

,titulo_m1_
Máster Formación de docentes de educación secundaria y formación profesional,0.169903
Máster Dirección y administración,0.048681
Máster Salud y seguridad laboral,0.047200
Máster Derecho,0.034312
Máster Psicología,0.027345
"Máster Sociología, antropología y geografía social y cultural",0.021945
Máster Ciencias de la Educación,0.021858
Máster Trabajo social y orientación,0.021510
Máster Electricidad y energía,0.019333
Máster Maquinaria y metalurgia,0.018375


In [53]:
proporciones('titulo_m1_')[30:]

,titulo_m1_
Máster Tecnología de diagnóstico y tratamiento médico,0.011234
Máster Periodismo y comunicación,0.011147
Máster Ingeniería y profesiones afines (Otros estudios),0.010799
Máster Control y tecnología medioambiental,0.010799
Máster Ingeniería civil y de la edificación,0.010711
Máster Marketing y publicidad,0.010363
Máster Aprendizaje de segundas lenguas,0.010276
Máster Economía,0.010276
Máster Técnicas audiovisuales y medios de comunicación,0.010102
Máster Literatura y lingüística,0.010102


In [54]:
D_describe['titulo_m1']='Titulación del primer máster universitario'
# Actualización diccionario para filas no informadas en datasets de grado o doctorado:
D_dict['titulo_m1'][0]='No tiene'
D_dict['titulo_m1'][str(0)]='No tiene'
D_dict['titulo_m1'][1]='Título no conocido'
D_dict['titulo_m1'][str(1)]='Título no conocido'

## 'RAMA' (redefinida como 'rama_m1')
**Rama de conocimiento** de la titulación.

In [55]:
df['RAMA'].unique() # Sin valores no informados

array([5, 3, 4, 2, 1])

In [56]:
traduce('RAMA')

                          aux_traduce
                               unique
RAMA                                 
1               [Artes y humanidades]
2                          [Ciencias]
3     [Ciencias sociales y jurídicas]
4         [Ingeniería y arquitectura]
5              [Ciencias de la salud]


In [57]:
redefine('RAMA',nuevo='rama_m1')
D_describe['rama_m1']='Rama de conocimiento del primer máster universitario'

### Corrección de títulos adscritos a varias ramas

La mayor parte de los títulos aparecen asociados a varias ramas de conocimiento. Se les asigna la más correcta de acuerdo las clasificaciones en ámbitos de los títulos de máster. En todos los casos salvo dos, esta es la asignación que se hace con más frecuencia para cada título. 

Se crea una rama nueva de 'Servicios' por homogeneidad con la modificación hecha para los títulos de grado. Se añaden a la categoría de servicios los asociados a la actividad física y el deporte, al turismo, al transporte y a la protección de la propiedad y de las personas (de acuerdo con la clasificación en ámbitos de los títulos de máster). 

In [58]:
# Muchos títulos asociados a varias ramas: 
aux_ramas=df[['rama_m1_','titulo_m1']].groupby(['titulo_m1']).agg(['nunique'])
aux_ramas.columns=aux_ramas.columns.to_flat_index()
aux_ramas.rename(columns={aux_ramas.columns[-1]:'num_ramas'},inplace=True)
#aux_ramas[aux_ramas['num_ramas']>1]

In [59]:
lista_titulos_corregir=aux_ramas[aux_ramas['num_ramas']>1].index.tolist()
len(lista_titulos_corregir)

52

In [60]:
#lista_titulos_corregir

In [61]:
# Inspección: se revisan todos los títulos y ramas
df[['titulo_m1','titulo_m1_','rama_m1_']]\
        .groupby(['titulo_m1','titulo_m1_']).agg(['value_counts'])[:30]

rama_m1_
                                                                                                                     value_counts
titulo_m1 titulo_m1_                                                                   rama_m1_                                  
111       Máster Ciencias de la Educación                                              Ciencias sociales y jurídicas          200
                                                                                       Artes y humanidades                     51
113       Máster Formación de docentes de enseñanza primaria                           Ciencias sociales y jurídicas           88
114       Máster Formación de docentes de educación secundaria y formación profesional Ciencias sociales y jurídicas         1857
                                                                                       Artes y humanidades                     94
119       Máster Educación (Otros estudios)                                            Ciencias sociales y jurídicas           50
211       Máster Técnicas audiovisuales y medios de comunicación                       Ciencias sociales y jurídicas           86
                                                                                       Artes y humanidades                     22
                                                                                       Ingeniería y arquitectura                8
213       Máster Bellas artes                                                          Artes y humanidades                    107
                                                                                       Ciencias sociales y jurídicas            2
215       Máster Música y artes del espectáculo                                        Artes y humanidades                    101
                                                                                       Ciencias sociales y jurídicas            3
219       Máster Artes (Otros estudios)                                                Artes y humanidades                     50
                                                                                       Ciencias sociales y jurídicas            8
221       Máster Religión y teología                                                   Artes y humanidades                     30
222       Máster Historia y arqueología                                                Artes y humanidades                    174
                                                                                       Ciencias sociales y jurídicas            7
223       Máster Filosofía y ética                                                     Artes y humanidades                     82
                                                                                       Ciencias de la salud                    27
229       Máster Humanidades (Otros estudios)                                          Artes y humanidades                     76
                                                                                       Ciencias sociales y jurídicas            4
231       Máster Aprendizaje de segundas lenguas                                       Artes y humanidades                    118
232       Máster Literatura y lingüística                                              Artes y humanidades                    116
239       Máster Lenguas (Otros estudios)                                              Artes y humanidades                     66
                                                                                       Ingeniería y arquitectura                2
311       Máster Economía                                                              Ciencias sociales y jurídicas          116
                                                                                       Artes y humanidades                      2
312       Máster Ciencias políticas                                                    Cien

In [62]:
# Inspección: 
df[['titulo_m1','titulo_m1_','rama_m1_']]\
        .groupby(['titulo_m1','titulo_m1_']).agg(['value_counts'])[30:60]

rama_m1_
                                                                                                       value_counts
titulo_m1 titulo_m1_                                                     rama_m1_                                  
313       Máster Psicología                                              Ciencias sociales y jurídicas          161
                                                                         Ciencias de la salud                   153
314       Máster Sociología, antropología y geografía social y cultural  Ciencias sociales y jurídicas          143
                                                                         Artes y humanidades                    108
                                                                         Ciencias de la salud                     1
319       Máster Ciencias sociales y del comportamiento (Otros estudios) Ciencias sociales y jurídicas           90
                                                                         Artes y humanidades                      2
321       Máster Periodismo y comunicación                               Ciencias sociales y jurídicas          108
                                                                         Artes y humanidades                     20
322       Máster Biblioteconomía, documentación y archivos               Ciencias sociales y jurídicas           74
                                                                         Artes y humanidades                     13
                                                                         Ingeniería y arquitectura                1
411       Máster Contabilidad y gestión de impuestos                     Ciencias sociales y jurídicas          131
412       Máster Finanzas, banca y seguros                               Ciencias sociales y jurídicas          142
413       Máster Dirección y administración                              Ciencias sociales y jurídicas          525
                                                                         Ciencias de la salud                    33
                                                                         Ingeniería y arquitectura                1
414       Máster Marketing y publicidad                                  Ciencias sociales y jurídicas          119
416       Máster Ventas al por mayor y al por menor                      Ciencias sociales y jurídicas           60
419       Máster Negocios y administración (Otros estudios)              Ciencias sociales y jurídicas           43
                                                                         Ingeniería y arquitectura                5
421       Máster Derecho                                                 Ciencias sociales y jurídicas          394
511       Máster Biología                                                Ciencias                               103
                                                                         Ciencias de la salud                    28
512       Máster Bioquímica                                              Ciencias                                98
                                                                         Ciencias de la salud                    42
                                                                         Ingeniería y arquitectura                1
519       Máster Ciencias de la vida (Otros estudios)                    Ciencias de la salud                    64
                                                                         Ciencias                                36
521       Máster Ciencias del medio ambiente                             Ciencias                                78

In [63]:
# Inspección: 
df[['titulo_m1','titulo_m1_','rama_m1_']]\
        .groupby(['titulo_m1','titulo_m1_']).agg(['value_counts'])[59:90]

rama_m1_
                                                                                                                                   value_counts
titulo_m1 titulo_m1_                                                                                 rama_m1_                                  
521       Máster Ciencias del medio ambiente                                                         Ciencias                                78
                                                                                                     Ingeniería y arquitectura               18
                                                                                                     Ciencias sociales y jurídicas           11
522       Máster Entornos naturales y vida silvestre                                                 Ciencias                                84
                                                                                                     Ciencias sociales y jurídicas            1
531       Máster Química                                                                             Ciencias                               161
532       Máster Ciencias de la tierra                                                               Ciencias                                95
                                                                                                     Ingeniería y arquitectura               24
                                                                                                     Ciencias sociales y jurídicas           17
533       Máster Física                                                                              Ciencias                               121
                                                                                                     Ingeniería y arquitectura               12
                                                                                                     Ciencias de la salud                     3
541       Máster Matemáticas                                                                         Ciencias                                59
                                                                                                     Ingeniería y arquitectura               23
542       Máster Estadística                                                                         Ciencias                                50
                                                                                                     Ingeniería y arquitectura               11
599       Máster Ciencias químicas, físicas y geológicas; Matemáticas y estadística (Otros estudios) Ciencias                                34
612       Máster Diseño y administración de bases de datos y redes                                   Ingeniería y arquitectura               63
613       Máster Desarrollo y análisis de aplicaciones y de software                                 Ingeniería y arquitectura              183
619       Máster Tecnologías de la información y las comunicaciones (Otros estudios)                 Ingeniería y arquitectura              136
711       Máster Ingeniería y procesos químicos                                                      Ingeniería y arquitectura               50
                                                                                                     Ciencias                                 2
712       Máster Control y tecnología medioambiental                                                 Ingeniería y arquitectura              104
                                                                                                     Ciencias                                15
                                                                                                     Artes y humanidades                      4
                                                                                                     Ci

In [64]:
# Inspección: 
df[['titulo_m1','titulo_m1_','rama_m1_']]\
        .groupby(['titulo_m1','titulo_m1_']).agg(['value_counts'])[90:120]

rama_m1_
                                                                                                                    value_counts
titulo_m1 titulo_m1_                                                                  rama_m1_                                  
715       Máster Maquinaria y metalurgia                                              Ingeniería y arquitectura              208
                                                                                      Ciencias sociales y jurídicas            3
716       Máster Vehículos de motor, barcos y aeronaves                               Ingeniería y arquitectura               66
719       Máster Ingeniería y profesiones afines (Otros estudios)                     Ingeniería y arquitectura              119
                                                                                      Ciencias                                 4
                                                                                      Ciencias sociales y jurídicas            1
721       Máster Industria de la alimentación                                         Ciencias                                57
                                                                                      Ciencias de la salud                    57
                                                                                      Ingeniería y arquitectura               17
722       Máster Industrias de otros materiales (madera, papel, plástico, vidrio,..)  Ingeniería y arquitectura               62
                                                                                      Ciencias                                 3
731       Máster Arquitectura, urbanismo y construcción                               Ingeniería y arquitectura              110
                                                                                      Artes y humanidades                     17
                                                                                      Ciencias                                 4
                                                                                      Ciencias sociales y jurídicas            3
732       Máster Ingeniería civil y de la edificación                                 Ingeniería y arquitectura              121
                                                                                      Ciencias                                 2
811       Máster Producción agrícola, explotación ganadera, horticultura y jardinería Ingeniería y arquitectura               97
                                                                                      Ciencias                                36
                                                                                      Ciencias de la salud                    20
                                                                                      Ciencias sociales y jurídicas            3
821       Máster Silvicultura                                                         Ingeniería y arquitectura               31
                                                                                      Ciencias de la salud                     8
                                                                                      Ciencias                                 7
831       Máster Pesca                                                                Ciencias                                48
841       Máster Veterinaria                                                          Ciencias de la salud                    41
                                                                                      Ciencias                                13
911       Máster Odontología                                                          Ciencias de la salud                    76
912       Máster Medicina                                                             Ciencias de la salud               

In [65]:
# Inspección: 
df[['titulo_m1','titulo_m1_','rama_m1_']]\
        .groupby(['titulo_m1','titulo_m1_']).agg(['value_counts'])[120:]

rama_m1_
                                                                                                             value_counts
titulo_m1 titulo_m1_                                                           rama_m1_                                  
913       Máster Enfermería                                                    Ciencias de la salud                   163
914       Máster Tecnología de diagnóstico y tratamiento médico                Ciencias de la salud                    91
                                                                               Ingeniería y arquitectura               27
                                                                               Ciencias                                11
915       Máster Terapia y rehabilitación                                      Ciencias de la salud                   170
                                                                               Ciencias sociales y jurídicas            4
916       Máster Farmacia                                                      Ciencias de la salud                    86
                                                                               Ciencias                                 5
                                                                               Ciencias sociales y jurídicas            5
919       Máster Salud (Otros estudios)                                        Ciencias de la salud                   204
                                                                               Ciencias sociales y jurídicas            3
                                                                               Ciencias                                 1
923       Máster Trabajo social y orientación                                  Ciencias sociales y jurídicas          199
                                                                               Ciencias de la salud                    45
                                                                               Artes y humanidades                      3
1013      Máster Hostelería                                                    Ciencias sociales y jurídicas           24
1014      Máster Actividades físicas y deportivas                              Ciencias sociales y jurídicas           83
                                                                               Ciencias de la salud                    60
1015      Máster Viajes, turismo y ocio                                        Ciencias sociales y jurídicas           80
                                                                               Artes y humanidades                      1
1022      Máster Salud y seguridad laboral                                     Ciencias sociales y jurídicas          383
                                                                               Ingeniería y arquitectura               96
                                                                               Ciencias de la salud                    36
                                                                               Ciencias                                27
1032      Máster Protección de la propiedad y las personas y enseñanza militar Ciencias sociales y jurídicas           72
                                                                               Ciencias                                11
                                                                               Ciencias de la salud                     7
                                                                               Ingeniería y arquitectura                7
1041      Máster Servicios de transporte                                       Ingeniería y arquitectura               51
                                                                               Ciencias sociales y jurídicas           21

In [66]:
# En todos los casos excepto dos, la rama correcta es la que se asigna con más frecuencia. 

# Inicialmente se asigna a todos los títulos la rama más frecuente: 
aux_corrige=df[['titulo_m1','titulo_m1_','rama_m1']][df['titulo_m1'].isin(lista_titulos_corregir)]\
        .groupby(['titulo_m1','titulo_m1_']).agg(['value_counts']).groupby(['titulo_m1','titulo_m1_'])\
        .rank(ascending=False)
aux_corrige.columns=aux_corrige.columns.to_flat_index()
aux_corrige.rename(columns={aux_corrige.columns[-1]:'rank'},inplace=True)
aux_corrige=aux_corrige[aux_corrige['rank']==1]
aux_corrige.reset_index(inplace=True)
#aux_corrige

In [67]:
# Diccionario de códigos de título (los títulos a corregir) a códigos de rama (asignación inicial): 
D_corrige_titulos=dict(zip(aux_corrige['titulo_m1'],aux_corrige['rama_m1']))
#D_corrige_titulos

In [68]:
# Diccionario de códigos de título (modificación de excepciones en reasignación por duplicidades): 

D_corrige_titulos[519]=2  # Ciencias de la vida (Otros estudios) a rama Ciencias
D_corrige_titulos[721]=4  # Industria de la alimentación a rama Ingeniería y Arquitectura

In [69]:
# Otras ramas corregidas sin duplicidad porque corresponden a la rama 'Servicios' añadida: 

D_corrige_titulos[1013]=6 # Hostelería a rama Servicios
D_corrige_titulos[1014]=6 # Actividades físicas y deportivas a rama Servicios
D_corrige_titulos[1015]=6 # Viajes, turismo y ocio a rama Servicios
D_corrige_titulos[1032]=6 # Protección de la propiedad y las personas y enseñanza militar a rama Servicios
D_corrige_titulos[1041]=6 # Servicios de transporte a rama Servicios

In [70]:
# Corrección ramas por añadido de rama 'Servicios':
lista_titulos_corregir=lista_titulos_corregir+[1013,1014,1015,1032,1041]

In [71]:
# Variables 'rama_g1' numérica y literal:
df['rama_m1'].mask(df['titulo_m1'].isin(lista_titulos_corregir),
                   df['titulo_m1'].replace(D_corrige_titulos),inplace=True)
df['rama_m1_']=df['rama_m1'].astype('O').replace(D_dict['rama_m1'])

## Nuevas variables: 'n_art_hum', 'n_ciencias', 'n_cc_soc_ju', 'n_ing_arq', 'n_cc_salud', 'n_servicios'
**'n_art_hum'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de artes y humanidades    
**'n_ciencias'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de Ciencias  
**'n_cc_soc_ju'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias sociales y jurídicas  
**'n_ing_arq'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ingeniería y arquitectura  
**'n_cc_salud'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias de la salud  
**'n_servicios'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de servicios  

Nota: Se crean a partir de la rama asociada al título de grado que motiva esta encuesta y se actualizan con las respuestas a preguntas posteriores.  

In [72]:
L_ramas=['n_art_hum','n_ciencias','n_cc_soc_ju','n_ing_arq','n_cc_salud','n_servicios']
for i,rama in enumerate (L_ramas): 
    df[rama]=0
    df[rama].mask(df['rama_m1']==i+1,1,inplace=True)

D_describe['n_art_hum']='Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de \
artes y humanidades'  
D_describe['n_ciencias']='Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de \
Ciencias'
D_describe['n_cc_soc_ju']='Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama \
de ciencias sociales y jurídicas'  
D_describe['n_ing_arq']='Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de \
ingeniería y arquitectura'  
D_describe['n_cc_salud']='Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de \
ciencias de la salud'  
    
df[L_ramas+['rama_m1_']].groupby('rama_m1_').agg(['unique']) # Comprobación: 

,n_art_hum,n_ciencias,n_cc_soc_ju,n_ing_arq,n_cc_salud,n_servicios
,unique,unique,unique,unique,unique,unique
rama_m1_,,,,,,
Artes y humanidades,[1],[0],[0],[0],[0],[0]
Ciencias,[0],[1],[0],[0],[0],[0]
Ciencias de la salud,[0],[0],[0],[0],[1],[0]
Ciencias sociales y jurídicas,[0],[0],[1],[0],[0],[0]
Ingeniería y arquitectura,[0],[0],[0],[1],[0],[0]
Servicios,[0],[0],[0],[0],[0],[1]


In [73]:
for vble in L_ramas: 
    a_lista(vble)

In [74]:
def actualiza_ramas(col):
    """Actualiza los valores de las variables 'n_art_hum', 'n_ciencias', 'n_cc_soc_ju', 'n_ing_arq', 'n_cc_salud'
    de acuerdo con la información proporcionada por la columna 'col'.
    """
    for i,rama in enumerate(L_ramas): 
        df['incremento']=0 
        # Para cada rama, +1 si el título asociado a 'col' pertenece a la rama:
        df['incremento'].mask(df[col]==i+1,1,inplace=True) 
        df[rama]=df[rama]+df['incremento']
    return 

## 'AMBITO' (redefinida como 'ambito_m1')
**Ámbito de estudio** de titulación de máster universitario

### Nota: Varias leyendas para 'Ambito de estudios'
La leyenda de 'ambito de estudio' para este título de máster (el que hace a los titulados elegibles para esta muestra) es diferente a la empleada para el 'ámbito de estudios' para los títulos de grado o equivalentes en esta encuesta. 
La empleada en para este título de máster es menos detallada que la usada en los títulos de grado. Esta última es la misma que se emplea para los títulos de grado en la encuesta de graduados universitarios. 
Para el resto de títulos de máster de esta encuesta no se informa del ámbito de estudios'. 

In [75]:
df['AMBITO'].sort_values().unique() # Sin valores no informados

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [76]:
df['AMBITO'].nunique()

10

In [77]:
D_dict['AMBITO']=diccionario('T2AMBIT')
traduce('AMBITO')

                                                        aux_traduce
                                                             unique
AMBITO                                                             
1                                                       [EDUCACIÓN]
2                                             [ARTES Y HUMANIDADES]
3                   [CIENCIAS SOCIALES, PERIODISMO Y DOCUMENTACIÓN]
4                              [NEGOCIOS, ADMINISTRACIÓN Y DERECHO]
5                                                        [CIENCIAS]
6                                                     [INFORMÁTICA]
7                            [INGENIERÍA, INDUSTRIA Y CONSTRUCCIÓN]
8       [AGRICULTURA, GANADERÍA, SILVICULTURA, PESCA Y VETERINARIA]
9                                      [SALUD Y SERVICIOS SOCIALES]
10                                                      [SERVICIOS]


In [78]:
redefine('AMBITO','ambito_m1')
D_describe['ambito_m1']='Ámbito de conocimiento del primer máster universitario'
# Actualización diccionario para filas no informadas en datasets de grado o doctorado:
D_dict['ambito_m1'][0]='No tiene'
D_dict['ambito_m1'][str(0)]='No tiene'
D_dict['ambito_m1'][1]='Ámbito no conocido'
D_dict['ambito_m1'][str(1)]='Ámbito no conocido'

### Inspección de relaciones entre títulos, ámbitos de estudio y ramas de conocimiento

In [79]:
# Inspección de relaciones título, ámbito y rama (instrumental para la corrección de ramas): 
aux_inspecciona=df[['titulo_m1','titulo_m1_','ambito_m1_','ambito_m1','rama_m1_']]\
           .groupby(['titulo_m1','titulo_m1_','ambito_m1','ambito_m1_']).agg(['unique'])
aux_inspecciona.reset_index(inplace=True)
aux_inspecciona[:30]

,titulo_m1,titulo_m1_,ambito_m1,ambito_m1_,rama_m1_
,,,,,unique
0,111,Máster Ciencias de la Educación,1,EDUCACIÓN,[Ciencias sociales y jurídicas]
1,113,Máster Formación de docentes de enseñanza primaria,1,EDUCACIÓN,[Ciencias sociales y jurídicas]
2,114,Máster Formación de docentes de educación secundaria y formación profesional,1,EDUCACIÓN,[Ciencias sociales y jurídicas]
3,119,Máster Educación (Otros estudios),1,EDUCACIÓN,[Ciencias sociales y jurídicas]
4,211,Máster Técnicas audiovisuales y medios de comunicación,2,ARTES Y HUMANIDADES,[Ciencias sociales y jurídicas]
5,213,Máster Bellas artes,2,ARTES Y HUMANIDADES,[Artes y humanidades]
6,215,Máster Música y artes del espectáculo,2,ARTES Y HUMANIDADES,[Artes y humanidades]
7,219,Máster Artes (Otros estudios),2,ARTES Y HUMANIDADES,[Artes y humanidades]
8,221,Máster Religión y teología,2,ARTES Y HUMANIDADES,[Artes y humanidades]


In [80]:
# Inspección de relaciones título, ámbito y rama (instrumental para la corrección de ramas): 
aux_inspecciona[30:60]

,titulo_m1,titulo_m1_,ambito_m1,ambito_m1_,rama_m1_
,,,,,unique
30,512,Máster Bioquímica,5,CIENCIAS,[Ciencias]
31,519,Máster Ciencias de la vida (Otros estudios),5,CIENCIAS,[Ciencias]
32,521,Máster Ciencias del medio ambiente,5,CIENCIAS,[Ciencias]
33,522,Máster Entornos naturales y vida silvestre,5,CIENCIAS,[Ciencias]
34,531,Máster Química,5,CIENCIAS,[Ciencias]
35,532,Máster Ciencias de la tierra,5,CIENCIAS,[Ciencias]
36,533,Máster Física,5,CIENCIAS,[Ciencias]
37,541,Máster Matemáticas,5,CIENCIAS,[Ciencias]
38,542,Máster Estadística,5,CIENCIAS,[Ciencias]


In [81]:
# Inspección de relaciones título, ámbito y rama (instrumental para la corrección de ramas): 

aux_inspecciona[60:90]

,titulo_m1,titulo_m1_,ambito_m1,ambito_m1_,rama_m1_
,,,,,unique
60,913,Máster Enfermería,9,SALUD Y SERVICIOS SOCIALES,[Ciencias de la salud]
61,914,Máster Tecnología de diagnóstico y tratamiento médico,9,SALUD Y SERVICIOS SOCIALES,[Ciencias de la salud]
62,915,Máster Terapia y rehabilitación,9,SALUD Y SERVICIOS SOCIALES,[Ciencias de la salud]
63,916,Máster Farmacia,9,SALUD Y SERVICIOS SOCIALES,[Ciencias de la salud]
64,919,Máster Salud (Otros estudios),9,SALUD Y SERVICIOS SOCIALES,[Ciencias de la salud]
65,923,Máster Trabajo social y orientación,9,SALUD Y SERVICIOS SOCIALES,[Ciencias sociales y jurídicas]
66,1013,Máster Hostelería,10,SERVICIOS,[Servicios]
67,1014,Máster Actividades físicas y deportivas,10,SERVICIOS,[Servicios]
68,1015,"Máster Viajes, turismo y ocio",10,SERVICIOS,[Servicios]


In [82]:
# Comprobación: todos los títulos asociados a un único ámbito de estudio: 
df_comprobacion=df[['rama_m1_','titulo_m1']].groupby(['titulo_m1']).agg(['nunique'])
df_comprobacion[df_comprobacion.columns[-1]].unique()

array([1])

### Inspección ámbitos de estudios y ramas de conocimiento

Algunos ámbitos de estudio están asociados a más de una rama de conocimiento. Sin embargo, las correspondencias son correctas. Formalmente, la clasificación en ramas no es una partición de la clasificación en ámbitos de títulos de máster. 

Las multiplicidades no son un problema para procesar ninguna otra variable ya que del resto de títulos de máster de los encuestados solo conocemos la rama y de sus títulos de grado conocemos el ámbito clasificado de acuerdo con el criterio empleado para los títulos de grado, más granulado que el de ámbitos de títulos de máster.  

Tampoco es un problema para la agregación del los ámbitos de grado en familias (las familias de títulos universitarios se definen más adelante, una vez se han agregado todas las encuestas, y se corresponden con los ámbitos de máster) porque los ámbitos de grado sí son una partición de los ámbitos de máster (y también de la clasificación en ramas). 

In [83]:
# Algunos ámbitos de estudios están asociados a más de una rama de conocimiento: 
aux_num_ramas=df[['rama_m1_','ambito_m1_']].groupby('ambito_m1_').agg(['nunique'])
aux_num_ramas[aux_num_ramas.columns[-1]].unique()

array([3, 2, 1])

In [84]:
aux_num_ramas.columns=aux_num_ramas.columns.to_flat_index()
aux_num_ramas.rename(columns={aux_num_ramas.columns[-1]:'num_ramas'},inplace=True)
aux_num_ramas

,num_ramas
ambito_m1_,
"AGRICULTURA, GANADERÍA, SILVICULTURA, PESCA Y VETERINARIA",3
ARTES Y HUMANIDADES,2
CIENCIAS,1
"CIENCIAS SOCIALES, PERIODISMO Y DOCUMENTACIÓN",1
EDUCACIÓN,1
INFORMÁTICA,1
"INGENIERÍA, INDUSTRIA Y CONSTRUCCIÓN",1
"NEGOCIOS, ADMINISTRACIÓN Y DERECHO",1
SALUD Y SERVICIOS SOCIALES,2


In [85]:
aux_num_ramas=df[['titulo_m1_','ambito_m1_','rama_m1_']].merge(aux_num_ramas,on='ambito_m1_',how='left')
#aux_num_ramas

In [86]:
# Inspección: títulos de máster asociados a los ámbitos clasificados en más de una rama de conocimiento: 
aux_num_ramas=aux_num_ramas[aux_num_ramas['num_ramas']>1]
aux_num_ramas[['titulo_m1_','ambito_m1_','rama_m1_']].groupby(['ambito_m1_','titulo_m1_']).agg(['unique'])

rama_m1_
                                                                                                                                                                unique
ambito_m1_                                                titulo_m1_                                                                                                  
AGRICULTURA, GANADERÍA, SILVICULTURA, PESCA Y VETERINARIA Máster Pesca                                                                                      [Ciencias]
                                                          Máster Producción agrícola, explotación ganadera, horticultura y jardinería      [Ingeniería y arquitectura]
                                                          Máster Silvicultura                                                              [Ingeniería y arquitectura]
                                                          Máster Veterinaria                                                                    [Ciencias de la salud]
ARTES Y HUMANIDADES                                       Máster Aprendizaje de segundas lenguas                                                 [Artes y humanidades]
                                                          Máster Artes (Otros estudios)                                                          [Artes y humanidades]
                                                          Máster Bellas artes                                                                    [Artes y humanidades]
                                                          Máster Filosofía y ética                                                               [Artes y humanidades]
                                                          Máster Historia y arqueología                                                          [Artes y humanidades]
                                                          Máster Humanidades (Otros estudios)                                                    [Artes y humanidades]
                                                          Máster Lenguas (Otros estudios)                                                        [Artes y humanidades]
                                                          Máster Literatura y lingüística                                                        [Artes y humanidades]
                                                          Máster Música y artes del espectáculo                                                  [Artes y humanidades]
                                                          Máster Religión y teología                                                             [Artes y humanidades]
                                                          Máster Técnicas audiovisuales y medios de comunicación                       [Ciencias sociales y jurídicas]
SALUD Y SERVICIOS SOCIALES                                Máster Enfermería                                                                     [Ciencias de la salud]
                                                          Máster Farmacia                                                                       [Ciencias de la salud]
                                                          Máster Medicina                                                                       [Ciencias de la salud]
                                                          Máster Odontología                                                                    [Ciencias de la salud]
                                                          Máster Salud (Otros estudios)                                                         [Ciencias de la salud]
                                                          Máster Tecnología de diagnóstico y tratamiento médico                                 [Ciencias de la salud]
                                                          Máster Terapia y rehabilitación                                                       [Cienc

In [87]:
proporciones('ambito_m1_')

,ambito_m1_
EDUCACIÓN,0.203780
"NEGOCIOS, ADMINISTRACIÓN Y DERECHO",0.126535
"INGENIERÍA, INDUSTRIA Y CONSTRUCCIÓN",0.120613
SALUD Y SERVICIOS SOCIALES,0.109205
CIENCIAS,0.102238
"CIENCIAS SOCIALES, PERIODISMO Y DOCUMENTACIÓN",0.099538
ARTES Y HUMANIDADES,0.094836
SERVICIOS,0.083515
INFORMÁTICA,0.033267
"AGRICULTURA, GANADERÍA, SILVICULTURA, PESCA Y VETERINARIA",0.026474


## 'T_UNIV'
**Tipo de universidad**.

In [88]:
df['T_UNIV'].unique() # Sin valores no informados

array([3, 4, 1, 2])

In [89]:
traduce('T_UNIV')

                              aux_traduce
                                   unique
T_UNIV                                   
1        [Universidad Pública presencial]
2       [Universidad Pública a distancia]
3        [Universidad Privada presencial]
4       [Universidad Privada a distancia]


## Nueva variable: 'tipo_centro'
**Tipo de centro educativo**: 1=Centros públicos, 0=Centros privados  

Variable proxi (en otros datasets las preguntas sobre tipo de centro están referidas a niveles educativos diferentes).

In [90]:
df['tipo_centro']=0
df['tipo_centro'].mask((df['T_UNIV']==1)|(df['T_UNIV']==2),1,inplace=True)
df[['T_UNIV','tipo_centro']].groupby('tipo_centro').agg(['unique']) # Comprobación: 

,T_UNIV
,unique
tipo_centro,
0,"[3, 4]"
1,"[1, 2]"


In [91]:
traduce('tipo_centro')

            aux_traduce
                 unique
tipo_centro            
0             [Privado]
1             [Público]


## Nueva variable: 'tipo_docencia'
**Formación presencial o a distancia**: 1=Formación a distancia, 0=Formación presencial  

In [92]:
df['tipo_docencia']=0
df['tipo_docencia'].mask((df['T_UNIV']==2)|(df['T_UNIV']==4),1,inplace=True)
df[['T_UNIV','tipo_docencia']].groupby('tipo_docencia').agg(['unique']) # Comprobación: 

,T_UNIV
,unique
tipo_docencia,
0,"[3, 1]"
1,"[4, 2]"


In [93]:
traduce('tipo_docencia')

                           aux_traduce
                                unique
tipo_docencia                         
0               [Formación presencial]
1              [Formación a distancia]


## 'DISCA' (redefinida como 'discapacidad')
Tiene **discapacidad reconocida** superior al 33 %.  
Nota: Se modifica en 'disca' el valor No: 0=No. 

In [94]:
df['DISCA'].unique() # Sin valores no informados

array([2, 1])

In [95]:
traduce('DISCA')

      aux_traduce
           unique
DISCA            
1            [Sí]
2            [No]


In [96]:
redefine('DISCA',nuevo='discapacidad',si_no=True)

             aux_traduce
                  unique
discapacidad            
0                   [No]
1                   [Sí]


## 'PROXY'
**Tipo de entrevista**.

In [97]:
df['PROXY'].unique() # Sin valores no informados

array([1, 2])

In [98]:
traduce('PROXY',save=False) 

                                            aux_traduce
                                                 unique
PROXY                                                  
1      [Entrevista directa con la persona seleccionada]
2                                    [Entrevista proxy]


## 'LUG_RES_PAIS' (redefinida como 'reside')
**País de residencia actual**.

In [99]:
df['LUG_RES_PAIS'].sort_values().unique() # Sin valores no informados

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [100]:
traduce('LUG_RES_PAIS')

                                                 aux_traduce
                                                      unique
LUG_RES_PAIS                                                
1                                                   [España]
2                                              [Reino Unido]
3                                                 [Alemania]
4                                                  [Francia]
5                            [Otro país de la Unión Europea]
6                       [Otro país europeo (fuera de la UE)]
7                             [Un país de América del Norte]
8             [Un país de América Central o América del Sur]
9                                          [Un país de Asia]
10                                       [Un país de África]
11                                      [Un país de Oceanía]


In [101]:
redefine('LUG_RES_PAIS','reside')

## 'PAIS_NAC_PADRE'
**País de nacimiento del padre**.

In [102]:
df['PAIS_NAC_PADRE'].unique() # No informados como 9

array([1, 2, 3, 9])

In [103]:
traduce('PAIS_NAC_PADRE')

                               aux_traduce
                                    unique
PAIS_NAC_PADRE                            
1                                 [España]
2                     [Otro país de la UE]
3               [Otro país fuera de la UE]
9                                  [NS/NC]


In [104]:
# Se imputa None a NS/NC:
no_respuesta('PAIS_NAC_PADRE',tipo='int')

[ 1.  2.  3. nan]


In [105]:
redefine('PAIS_NAC_PADRE','origen_pa')

## 'ESTUDIOS_PADRE'  (redefinida como 'estudios_pa')
**Nivel de formación máximo alcanzado por el padre**.

In [106]:
df['ESTUDIOS_PADRE'].unique() # No informados como 9

array([8, 7, 2, 4, 9, 6, 3, 5, 1])

In [107]:
traduce('ESTUDIOS_PADRE')

                                                                                                 aux_traduce
                                                                                                      unique
ESTUDIOS_PADRE                                                                                              
1                                                                                  [No sabe leer o escribir]
2                                 [Educación primaria incompleta (ha asistido menos de 5 años a la escuela)]
3                                                                              [Educación primaria completa]
4               [Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental)]
5                                            [Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)]
6                                                   [Enseñanzas profesionales de grado medio o equivalentes]
7                  

In [108]:
# Se imputa None a NS/NC (9):
no_respuesta('ESTUDIOS_PADRE',tipo='int')

[ 1.  2.  3.  4.  5.  6.  7.  8. nan]


In [109]:
redefine('ESTUDIOS_PADRE','estudios_pa')

## Nueva variable: 'grado_med_pa'
**Padre** con o sin **estudios de enseñanzas medias o superiores**, incluidos los universitarios: 1=Con estudios de enseñanzas medias o superiores, 0=Sin estudios de enseñanzas medias o superiores.  

In [110]:
df['grad_med_pa']=0
df['grad_med_pa'].mask(df['ESTUDIOS_PADRE']>=5,1,inplace=True)
df[['ESTUDIOS_PADRE','grad_med_pa']].groupby('grad_med_pa').agg(['unique']) # Comprobación: 

,ESTUDIOS_PADRE
,unique
grad_med_pa,
0,"[2.0, 4.0, nan, 3.0, 1.0]"
1,"[8.0, 7.0, 6.0, 5.0]"


In [111]:
traduce('grad_med_pa')

                                                  aux_traduce
                                                       unique
grad_med_pa                                                  
0            [Sin estudios de enseñanzas medias o superiores]
1            [Con estudios de enseñanzas medias o superiores]


## Nueva variable: 'univ_pa'
**Padre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [112]:
df['univ_pa']=0
df['univ_pa'].mask(df['ESTUDIOS_PADRE']==8,1,inplace=True)
df[['ESTUDIOS_PADRE','univ_pa']].groupby('univ_pa').agg(['unique']) # Comprobación: 

,ESTUDIOS_PADRE
,unique
univ_pa,
0,"[7.0, 2.0, 4.0, nan, 6.0, 3.0, 5.0, 1.0]"
1,[8.0]


In [113]:
traduce('univ_pa')

                           aux_traduce
                                unique
univ_pa                               
0        [Sin estudios universitarios]
1        [Con estudios universitarios]


## 'PAIS_NAC_MADRE'
**País de nacimiento de la madre**.

In [114]:
df['PAIS_NAC_MADRE'].unique() # No informados como 9

array([1, 9, 3, 2])

In [115]:
traduce('PAIS_NAC_MADRE')

                               aux_traduce
                                    unique
PAIS_NAC_MADRE                            
1                                 [España]
2                     [Otro país de la UE]
3               [Otro país fuera de la UE]
9                                  [NS/NC]


In [116]:
# Se imputa None a NS/NC (9):
no_respuesta('PAIS_NAC_MADRE',tipo='int')

[ 1.  2.  3. nan]


In [117]:
redefine('PAIS_NAC_MADRE','origen_ma')

## 'ESTUDIOS_MADRE' (redefinida como 'estudios_ma')
**Nivel de formación máximo alcanzado por la madre**.

In [118]:
df['ESTUDIOS_MADRE'].unique() # Missing values as 9

array([8, 7, 6, 4, 5, 2, 3, 9, 1])

In [119]:
traduce('ESTUDIOS_MADRE')

                                                                                                 aux_traduce
                                                                                                      unique
ESTUDIOS_MADRE                                                                                              
1                                                                                  [No sabe leer o escribir]
2                                 [Educación primaria incompleta (ha asistido menos de 5 años a la escuela)]
3                                                                              [Educación primaria completa]
4               [Primera etapa de Enseñanza Secundaria, con o sin título (ESO, EGB, Bachillerato Elemental)]
5                                            [Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)]
6                                                   [Enseñanzas profesionales de grado medio o equivalentes]
7                  

In [120]:
# Se imputa None a NS/NC (9):
no_respuesta('ESTUDIOS_MADRE',tipo='int')

[ 1.  2.  3.  4.  5.  6.  7.  8. nan]


In [121]:
redefine('ESTUDIOS_MADRE','estudios_ma')

## Nueva variable: 'grado_med_ma'
**Madre** con o sin **estudios de enseñanzas medias o superiores**, incluidos los universitarios: 1=Con estudios de enseñanzas medias o superiores, 0=Sin estudios de enseñanzas medias o superiores.  

In [122]:
df['grad_med_ma']=0
df['grad_med_ma'].mask(df['ESTUDIOS_MADRE']>=5,1,inplace=True)
df[['ESTUDIOS_MADRE','grad_med_ma']].groupby('grad_med_ma').agg(['unique']) # Comprobación: 

,ESTUDIOS_MADRE
,unique
grad_med_ma,
0,"[4.0, 2.0, 3.0, nan, 1.0]"
1,"[8.0, 7.0, 6.0, 5.0]"


In [123]:
traduce('grad_med_ma')

                                                  aux_traduce
                                                       unique
grad_med_ma                                                  
0            [Sin estudios de enseñanzas medias o superiores]
1            [Con estudios de enseñanzas medias o superiores]


## Nueva variable: 'univ_ma'
**Madre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [124]:
df['univ_ma']=0
df['univ_ma'].mask(df['ESTUDIOS_MADRE']==8,1,inplace=True)
df[['ESTUDIOS_MADRE','univ_ma']].groupby('univ_ma').agg(['unique']) # Comprobación: 

,ESTUDIOS_MADRE
,unique
univ_ma,
0,"[7.0, 6.0, 4.0, 5.0, 2.0, 3.0, nan, 1.0]"
1,[8.0]


In [125]:
traduce('univ_ma')

                           aux_traduce
                                unique
univ_ma                               
0        [Sin estudios universitarios]
1        [Con estudios universitarios]


# BLOQUE B. EDUCACIÓN Y APRENDIZAJE DEL TITULADO DE MÁSTER

## 'EST_B1' (redefinida como 'beca')
¿Disfrutó de **alguna beca** durante sus estudios?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca' el valor asociado a No: 0=No

In [126]:
df['EST_B1'].unique() # No informados como 9

array([2, 1, 9])

In [127]:
traduce('EST_B1')

       aux_traduce
            unique
EST_B1            
1             [Sí]
2             [No]
9          [NS/NC]


In [128]:
# Se imputa None a NS/NC:
no_respuesta('EST_B1',tipo='int')

[ 1.  2. nan]


In [129]:
redefine('EST_B1','beca',si_no=True)

     aux_traduce
          unique
beca            
0.0         [No]
1.0         [Sí]


## 'EST_B2_1' (redefinida como 'beca_general')
(Solo para respuesta Sí (1) en variable 'EST_B1' (beca))  

¿Disfrutó de una **beca general para el estudio**?: 1=Sí, 2=No, 9=NS/NC.

Nota: se modifica en 'beca_general' el valor asociado a No: 0=No

In [130]:
df['EST_B2_1'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [131]:
traduce('EST_B2_1')

             aux_traduce
                  unique
EST_B2_1                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [132]:
def re_beca(col):
    """Inspeccion e imputación de valores de variables condicionadas por respuesta a 'EST_B1' cuando pueden 
    deducirse de los itinerarios de la encuesta.
    """
    print (df[['EST_B1',col]].groupby('EST_B1').agg(['unique']))
    # Se imputa respuesta No (2) a quienes manifestaron no haber disfrutado de beca (2 en 'EST_B1')
    df[col].mask(df['EST_B1']==2,2,inplace=True)
    # Se imputa None al resto de respuestas en blanco y a NS/NC:
    try: df[col].replace({' ':None,'9':None,},inplace=True)
    except:
        df[col]=df[col].astype('O')
        df[col].replace({' ':None,'9':None,},inplace=True) # Resuelve conflictos cuando se aplica más de una vez
    try: df[col]=df[col].astype('int')
    except: df[col]=df[col].astype('float')
    traduce(col,check=False)
    print (df[['EST_B1',col]].groupby('EST_B1').agg(['unique']))
    return

In [133]:
re_beca('EST_B2_1')

         EST_B2_1
           unique
EST_B1           
1.0     [1, 2, 9]
2.0           [ ]
               EST_B2_1
                 unique
EST_B1                 
1.0     [1.0, 2.0, nan]
2.0               [2.0]


In [134]:
redefine('EST_B2_1','beca_general',si_no=True)

             aux_traduce
                  unique
beca_general            
0.0                 [No]
1.0                 [Sí]


## 'EST_B2_2' (redefinida como 'beca_excel')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de algún **premio o beca de excelencia**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_excel' el valor asociado a No: 0=No.

In [135]:
df['EST_B2_2'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [136]:
traduce('EST_B2_2')

             aux_traduce
                  unique
EST_B2_2                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [137]:
re_beca('EST_B2_2')

         EST_B2_2
           unique
EST_B1           
1.0     [2, 1, 9]
2.0           [ ]
               EST_B2_2
                 unique
EST_B1                 
1.0     [2.0, 1.0, nan]
2.0               [2.0]


In [138]:
redefine('EST_B2_2','beca_excel',si_no=True)

           aux_traduce
                unique
beca_excel            
0.0               [No]
1.0               [Sí]


## 'EST_B2_3' (redefinida como 'beca_colab')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de colaboración** en la universidad?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_colab' el valor asociado a No: 0=No.

In [139]:
df['EST_B2_3'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [140]:
traduce('EST_B2_3')

             aux_traduce
                  unique
EST_B2_3                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [141]:
re_beca('EST_B2_3')

         EST_B2_3
           unique
EST_B1           
1.0     [2, 1, 9]
2.0           [ ]
               EST_B2_3
                 unique
EST_B1                 
1.0     [2.0, 1.0, nan]
2.0               [2.0]


In [142]:
redefine('EST_B2_3','beca_colab',si_no=True)

           aux_traduce
                unique
beca_colab            
0.0               [No]
1.0               [Sí]


## 'EST_B2_4' (redefinida como 'beca_pract')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de prácticas externas** o para prestar un servicio o desempeñar un trabajo fuera de la Universidad?: 1=Sí, 2=No, 9=NS/NC.  


Nota: se modifica en 'beca_pract' el valor asociado a No: 0=No.

In [143]:
df['EST_B2_4'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [144]:
traduce('EST_B2_4')

             aux_traduce
                  unique
EST_B2_4                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [145]:
re_beca('EST_B2_4')

         EST_B2_4
           unique
EST_B1           
1.0     [2, 1, 9]
2.0           [ ]
               EST_B2_4
                 unique
EST_B1                 
1.0     [2.0, 1.0, nan]
2.0               [2.0]


In [146]:
redefine('EST_B2_4','beca_pract',si_no=True)

           aux_traduce
                unique
beca_pract            
0.0               [No]
1.0               [Sí]


## 'EST_B2_5' (redefinida como 'beca_extr')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca para realizar estudios fuera de España** (incluidas becas complemento a programas Erasmus)?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_extr' el valor asociado a No: 0=No.

In [147]:
df['EST_B2_5'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [148]:
traduce('EST_B2_5')

             aux_traduce
                  unique
EST_B2_5                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [149]:
re_beca('EST_B2_5')

         EST_B2_5
           unique
EST_B1           
1.0     [1, 2, 9]
2.0           [ ]
               EST_B2_5
                 unique
EST_B1                 
1.0     [1.0, 2.0, nan]
2.0               [2.0]


In [150]:
redefine('EST_B2_5','beca_extr',si_no=True)

          aux_traduce
               unique
beca_extr            
0.0              [No]
1.0              [Sí]


## 'EST_M1' (redefinida como 'extr')
(Solo para respuesta distinta de Sí (1) en variable 'EST_B2_5')  

¿Realizó parte de sus **estudios en el extranjero**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No.

In [151]:
df['EST_M1'].unique() # No informados como '9'

array([1, 2, 9])

In [152]:
traduce('EST_M1')

       aux_traduce
            unique
EST_M1            
1             [Sí]
2             [No]
9          [NS/NC]


In [153]:
df[['EST_B2_5_','EST_M1_']].groupby('EST_B2_5_').agg(['unique']) # Inspección
# Imputada por el encuestador respuesta Si (1) a quienes manifestaron haber disfrutado de beca para ...
# ... estudiar fuera de España ('EST_B2_5')

,EST_M1_
,unique
EST_B2_5_,
No,"[Sí, No, NS/NC]"
Sí,[Sí]


In [154]:
# Se imputa None a NS/NC: 
no_respuesta('EST_M1',tipo='int')

[ 1.  2. nan]


In [155]:
df[['EST_B2_5_','EST_M1_']].groupby('EST_B2_5_').agg(['unique']) # Comprobacion: 

,EST_M1_
,unique
EST_B2_5_,
No,"[Sí, No, nan]"
Sí,[Sí]


In [156]:
redefine('EST_M1','extr',si_no=True)

     aux_traduce
          unique
extr            
0.0         [No]
1.0         [Sí]


## Nueva variable: 'extr_sin_beca'
¿Realizó parte de sus **estudios en el extranjero sin beca** específica?: 1=Si, 0=No.  

Variable proxi de renta familiar alta.  

In [157]:
df['extr_sin_beca']=0
df['extr_sin_beca'].mask((df['extr']==1)&(df['EST_B2_5']!=1),1,inplace=True)
df[['extr_sin_beca','extr','EST_B2_5']].groupby(['extr','EST_B2_5']).agg(['unique']) # Comprobación

extr_sin_beca
                     unique
extr EST_B2_5              
0.0  2.0                [0]
1.0  1.0                [0]
     2.0                [1]

In [158]:
traduce('extr_sin_beca')

              aux_traduce
                   unique
extr_sin_beca            
0                    [No]
1                    [Sí]


## 'EST_M2' (redefinida como 'tipo_beca_extr')

(Solo para respuesta Sí (1) en variable 'EST_B2_5' (o 'beca_extr'))  

**Programa o beca de movilidad**.

In [159]:
df['EST_M2'].unique() # No informados como ' ' 

array([' ', '1', '3', '2'], dtype=object)

In [160]:
traduce('EST_M2')

                                      aux_traduce
                                           unique
EST_M2                                           
                                   [No aplicable]
1                              [Programa Erasmus]
2       [Otros programas o becas dentro de la UE]
3        [Otros programas o becas fuera de la UE]


In [161]:
# Se imputa None a NS/NC (9) y a respuesta en blanco: 
no_respuesta('EST_M2')

[ 1.  2.  3. nan]


In [162]:
redefine('EST_M2','tipo_beca_extr')

In [163]:
df[['beca_extr_','tipo_beca_extr_']].groupby('beca_extr_',dropna=False).agg(['unique']) # Comprobación

,tipo_beca_extr_
,unique
beca_extr_,
No,[nan]
Sí,"[Programa Erasmus, Otros programas o becas fuera de la UE, Otros programas o becas dentro de la UE]"
NaN,[nan]


## 'EST_M3'  (redefinida como 'pais_beca_extr')  

(Solo para respuesta Sí (1) en variable 'EST_M1' (o 'extr')) 

**País de destino** cuando estudió fuera de España.

In [164]:
df['EST_M3'].sort_values().unique() # No informados como '  ' o '99'

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '11', '12',
       '21', '31', '41', '99'], dtype=object)

In [165]:
# Se imputa None a respuestas en blanco y NS/NC: 
no_respuesta('EST_M3',blank='  ',nsnc='99')

[ 1.  2.  3.  4.  5.  6.  7.  8. 11. 12. 21. 31. 41. nan]


In [166]:
df['EST_M3']=df['EST_M3'].astype('float')
traduce('EST_M3')

                                                                 aux_traduce
                                                                      unique
EST_M3                                                                      
1.0                                                            [Reino Unido]
2.0                                                               [Alemania]
3.0                                                                [Francia]
4.0                                                                 [Italia]
5.0                                                               [Portugal]
6.0     [Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia)]
7.0                                          [Otro país de la Unión Europea]
8.0                                                      [Otro país europeo]
11.0                                          [Un país de América del Norte]
12.0                          [Un país de América Central o América del Sur]

In [167]:
redefine('EST_M3','pais_beca_extr')

## 'EST_M4'
(Solo para respuesta Sí (1) en variable 'EST_M1' (o 'extr'))  

**Duración** en meses del periodo en el que estudió fuera de España.

In [168]:
df['EST_M4'].unique() # No informados como ' ' o '9'

array(['4', ' ', '1', '2', '3', '5', '9'], dtype=object)

In [169]:
traduce('EST_M4')

               aux_traduce
                    unique
EST_M4                    
            [No aplicable]
1         [De 1 a 3 meses]
2         [De 4 a 6 meses]
3         [De 7 a 9 meses]
4       [De 10 a 12 meses]
5          [Más de un año]
9                  [NS/NC]


In [170]:
df[['extr_','EST_M4_']].groupby('extr_',dropna=False).agg(['unique']) # Inspección: 

,EST_M4_
,unique
extr_,
No,[No aplicable]
Sí,"[De 10 a 12 meses, De 1 a 3 meses, De 4 a 6 meses, De 7 a 9 meses, Más de un año, NS/NC]"
NaN,[No aplicable]


In [171]:
# Se imputa None a respuestas en blanco y NS/NC
no_respuesta('EST_M4')

[ 1.  2.  3.  4.  5. nan]


## 'EST_MES' 
¿Realizó parte de sus estudios para obtener su título de máster en **otra universidad española**? 

In [172]:
df['EST_MES'].unique() # No informados como '9'

array([1, 2, 9])

In [173]:
traduce('EST_MES')

        aux_traduce
             unique
EST_MES            
1              [Sí]
2              [No]
9           [NS/NC]


In [174]:
# Se imputa None a NS/NC:
no_respuesta('EST_MES',tipo='int')

[ 1.  2. nan]


## 'EST_B8' (redefinida como 'motivacion')
**Motivo principal** por el que ha realizado sus **estudios** de máster.

In [175]:
df['EST_B8'].unique() # No informados como '9'

array([1, 2, 3, 9])

In [176]:
traduce('EST_B8')

                                              aux_traduce
                                                   unique
EST_B8                                                   
1                [Formarse pensando en su futuro laboral]
2       [Ampliar conocimientos por satisfacción personal]
3                                         [Otros motivos]
9                                                 [NS/NC]


In [177]:
# Se imputa None a NS/NC:
no_respuesta('EST_B8',tipo='int')

[ 1.  2.  3. nan]


In [178]:
redefine('EST_B8','motivacion')

## 'SAT1' (redefinida como 'sat_est_u')
¿Si tuviera que empezar de nuevo **volvería a cursar estudios universitarios**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica en 'sat_est_u' el valor asociado a la respuesta No: 0=No.

In [179]:
df['SAT1'].unique() # No informados como '9'

array([1, 2, 9])

In [180]:
traduce('SAT1')

     aux_traduce
          unique
SAT1            
1           [Sí]
2           [No]
9        [NS/NC]


In [181]:
# Se imputa None a NS/NC:
no_respuesta('SAT1',tipo='int')

[ 1.  2. nan]


In [182]:
redefine('SAT1','sat_est_u',si_no=True)

          aux_traduce
               unique
sat_est_u            
0.0              [No]
1.0              [Sí]


## 'SAT2' (redefinida como 'sat_titulo')  
(Solo para respuesta Sí (1) en 'SAT1' (o 'sat_est_u')) 

¿Si tuviera que empezar de nuevo **volvería a estudiar la misma titulación**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica en 'sat_titulo' el valor asociado a la respuesta No: 0=No.

In [183]:
df['SAT2'].unique() # No informados como ' ' o '9'

array(['1', '2', ' ', '9'], dtype=object)

In [184]:
traduce('SAT2')

         aux_traduce
              unique
SAT2                
      [No aplicable]
1               [Sí]
2               [No]
9            [NS/NC]


In [185]:
df[['SAT1_','SAT2_']].groupby('SAT1_',dropna=False).agg(['unique']) # Inspección

,SAT2_
,unique
SAT1_,
No,[No aplicable]
Sí,"[Sí, No, NS/NC]"
NaN,[No aplicable]


In [186]:
# Se imputa respuesta No (2) a quienes declararon que no volverían a cursar estudios universitarios: 
df['SAT2'].mask(df['sat_est_u']==0,2,inplace=True)
# Se imputa None a resto de respuestas en blanco y NS/NC:
no_respuesta('SAT2')

[ 1.  2. nan]


In [187]:
df[['SAT1_','SAT2_']].groupby('SAT1_',dropna=False).agg(['unique'])

,SAT2_
,unique
SAT1_,
No,[No]
Sí,"[Sí, No, nan]"
NaN,[nan]


In [188]:
redefine('SAT2','sat_titulo',si_no=True)

           aux_traduce
                unique
sat_titulo            
0.0               [No]
1.0               [Sí]


## Variables sobre otra formación realizada: variables  'EST_B11_1' a 'EST_B11_7'
Nota: Debe tenerse en cuenta que se pregunta sobre otros estudios completados, no sobre los iniciados, y que para los titulados recientes se pueden estar subestimando las titulaciones posteriores por no haber habido tiempo suficiente para terminarlas. 

## 'EST_B11_1'  (redefinida como 'grado')
¿Ha completado **estudios de grado**, diplomatura, licenciatura o equivalente?  1=Sí, 2=No.  

In [189]:
df['EST_B11_1'].unique() # Sin no informados

array([1, 2])

In [190]:
traduce('EST_B11_1')

          aux_traduce
               unique
EST_B11_1            
1                [Sí]
2                [No]


In [191]:
redefine('EST_B11_1','grado',si_no=True)

      aux_traduce
           unique
grado            
0            [No]
1            [Sí]


In [192]:
df['grado'].value_counts() # Hay titulados de máster universitario sin título de grado o equivalente

1    11402
0       81
Name: grado, dtype: int64

## 'EST_B11_2'  (redefinida como 'varios_master')   
¿Ha completado **otros estudios de máster universitario**?  1=Sí, 2=No.  

Nota: Se modifica en 'master' el valor asociado a la respuesta No: 0=No.

In [193]:
df['EST_B11_2'].unique() # Sin no informados

array([2, 1])

In [194]:
traduce('EST_B11_2')

          aux_traduce
               unique
EST_B11_2            
1                [Sí]
2                [No]


In [195]:
redefine('EST_B11_2','varios_master',si_no=True)

              aux_traduce
                   unique
varios_master            
0                    [No]
1                    [Sí]


## 'EST_B11_3'   (redefinida como 'doctor')  
¿Ha completado otros estudios de **doctorado universitario**?  1=Sí, 2=No.  

Nota: Se modifica en 'doctor' el valor asociado a la respuesta No: 0=No.

In [196]:
df['EST_B11_3'].unique() # Sin no informados

array([2, 1])

In [197]:
traduce('EST_B11_3')

          aux_traduce
               unique
EST_B11_3            
1                [Sí]
2                [No]


In [198]:
redefine('EST_B11_3','doctor',si_no=True)

       aux_traduce
            unique
doctor            
0             [No]
1             [Sí]


## Variable nueva:  'varios_titulos_u'

No informada en este dataset. Se añade por homogeneidad con la encuesta de grado de 2014.  

Se imputa valor 1 (Sí) a todos los encuestados por tener un título de máster además del de grado o equivalente

In [199]:
df['varios_titulos_u']=1
df['varios_titulos_u_']='Sí'

## 'EST_B11_4'  (redefinida como 'est_art')
¿Ha completado otros **estudios artísticos superiores** (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)?  1=Sí, 2=No.  

Nota: Se modifica en 'est_art' el valor asociado a la respuesta No: 0=No.

In [200]:
df['EST_B11_4'].unique() # Sin no informados

array([2, 1])

In [201]:
traduce('EST_B11_4')

          aux_traduce
               unique
EST_B11_4            
1                [Sí]
2                [No]


In [202]:
redefine('EST_B11_4','est_art',si_no=True)

        aux_traduce
             unique
est_art            
0              [No]
1              [Sí]


## 'EST_B11_5'   (redefinida como 'g_sup')
¿Ha completado algún **ciclo formativo de grado superior** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado superior**?  1=Sí, 2=No.  

Nota: Se modifica en 'g_sup' el valor asociado a la respuesta No: 0=No.

In [203]:
df['EST_B11_5'].unique() # Sin no informados

array([2, 1])

In [204]:
traduce('EST_B11_5')

          aux_traduce
               unique
EST_B11_5            
1                [Sí]
2                [No]


In [205]:
redefine('EST_B11_5','g_sup',si_no=True)

      aux_traduce
           unique
g_sup            
0            [No]
1            [Sí]


## 'EST_B11_6'   (redefinida como 'g_medio')
¿Ha completado algún **ciclo formativo de grado medio** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado medio**?  1=Sí, 2=No.  

Nota: Se modifica en 'g_medio' el valor asociado a la respuesta No: 0=No.

In [206]:
df['EST_B11_6'].unique() # Sin no informados

array([2, 1])

In [207]:
traduce('EST_B11_6')

          aux_traduce
               unique
EST_B11_6            
1                [Sí]
2                [No]


In [208]:
redefine('EST_B11_6','g_medio',si_no=True)

        aux_traduce
             unique
g_medio            
0              [No]
1              [Sí]


## 'EST_B12' 
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente)  

**Número de otros estudios de grado**, diplomatura, licenciatura o equivalente, incluyendo el que le dio acceso a los estudios de máster.

In [209]:
df['EST_B12'].unique() # No informados como ' '

array(['1', '2', '3', ' '], dtype=object)

In [210]:
traduce('EST_B12')

            aux_traduce
                 unique
EST_B12                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


##  Nueva variable 'n_grados'     
**Número de títulos de grado**, diplomatura, licenciatura o equivalente.

In [211]:
df['n_grados']=df['EST_B12'].replace({' ':0}).astype('int')
df[['n_grados','EST_B12']].groupby('n_grados').agg(['unique']) # Comprobación

,EST_B12
,unique
n_grados,
0,[ ]
1,[1]
2,[2]
3,[3]


In [212]:
df['n_grados'].value_counts() 

1    9037
2    2152
3     213
0      81
Name: n_grados, dtype: int64

In [213]:
a_lista('n_grados')

## 'EST_B13_AMB1' (redefinida como 'ambito_g1')
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente) 

**Ambito de estudio del primer grado**, diplomatura, licenciatura o equivalente en el que está titulado.  

Esta clasificación de ámbitos de estudios sigue el criterio definido para los títulos de grado, más fino que el aplicado a los títulos de máster. 

In [214]:
reset('EST_B13_AMB1')

In [215]:
df['EST_B13_AMB1'].sort_values().unique() # No informados como '    ' o '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '0211', '022 ', '023 ',
       '031 ', '0311', '0313', '032 ', '041 ', '0413', '042 ', '051 ',
       '052 ', '053 ', '054 ', '061 ', '071 ', '072 ', '073 ', '081 ',
       '082 ', '084 ', '091 ', '0912', '0913', '092 ', '1014', '1015',
       '109 ', '9999'], dtype=object)

In [216]:
df['EST_B13_AMB1'].nunique()

34

In [217]:
# Ajuste en formato:
df['EST_B13_AMB1']=df['EST_B13_AMB1'].str.strip()
# Ajuste en diccionario (leyenda de ámbito de estudios de grado en encuesta titulados de máster)
D_dict['EST_B13_AMB1']=diccionario('T4AMBIT')
traduce('EST_B13_AMB1')

                                                             aux_traduce
                                                                  unique
EST_B13_AMB1                                                            
                                                                      []
011                                        [Educación (Otros estudios) ]
0112                      [Formación de docentes de enseñanza infantil ]
0113                      [Formación de docentes de enseñanza primaria ]
021                                            [Artes (Otros estudios) ]
0211                  [Técnicas audiovisuales y medios de comunicación ]
022                                                       [Humanidades ]
023                                                           [Lenguas ]
031           [Ciencias sociales y del comportamiento (Otros estudios) ]
0311                                                         [Economía ]
0313                                               

In [218]:
def re_otros_estudios(col,nuevo='',n=0,blank=' ',nsnc='9',trad=True):
    """Inspección e imputación de valores de col. condicionadas por la respuesta sobre los estudios otros_estudios
    Si se especifica un valor n y un nombre 'nuevo', crea una variable 'nuevo' para la que define 
    valor 0 = No tiene n títulos de otros_estudios. Se añade la variable nueva a la lista de variables para 
    modelos descriptivos o predicctivos.
    """
    # Se imputa None a respuestas en blanco y NS/NC:
    try: df[col].replace({blank:None,nsnc:None},inplace=True)
    except: 
        df[col]=df[col].astype('O')
        df[col].replace({blank:None,nsnc:None},inplace=True) # Resuelve conflictos cuando se aplica varias veces
    try: df[col]=df[col].astype('float')
    except:pass
    if trad==True: traduce(col,check=False)
    if len(nuevo)>0 and n>0:
        df[nuevo]=df[col]
        D_dict[nuevo]=D_dict[col]
        traduce(nuevo,check=False)
        print(df[[str(nuevo)+'_',otros_estudios]].groupby(otros_estudios).agg(['unique']))
        # Se imputa valor 0 ('No tiene') a quienes tienen menos de n títulos: 
        df[nuevo].mask(df[otros_estudios]<n,0,inplace=True)
        D_dict[nuevo][0]='No tiene'
        D_dict[nuevo][str(0)]='No tiene'
        # Se imputa valor 1 ('No conocido') a las observaciones no informadas con al menos n títulos: 
        df[nuevo].mask((df[nuevo].isna()) & (df[otros_estudios]>=n),1,inplace=True)
        D_dict[nuevo][1]='No conocido'
        D_dict[nuevo][str(1)]='No conocido'
        traduce(nuevo,check=False)
        print(df[[str(nuevo)+'_',otros_estudios]].groupby(otros_estudios).agg(['unique']))
        if str(nuevo) not in v_modelos:
            v_modelos.append(nuevo)
        if str(nuevo)+'_' not in v_modelos_L:
            v_modelos_L.append(str(nuevo)+'_')
    else: print (df[col].sort_values().unique())
    return

In [219]:
# Se establece la variable otros_estudios referida a otros estudios de grado: 
otros_estudios='n_grados'

In [220]:
re_otros_estudios('EST_B13_AMB1',nuevo='ambito_g1',n=1,blank='',nsnc='9999')

                                                                                                                                                                                                                                                                                                           ambito_g1_
                                                                                                                                                                                                                                                                                                               unique
n_grados                                                                                                                                                                                                                                                                                                             
0                                                                     

In [221]:
traduce('ambito_g1')

                                                          aux_traduce
                                                               unique
ambito_g1                                                            
0.0                                                        [No tiene]
1.0                                                     [No conocido]
11.0                                    [Educación (Otros estudios) ]
21.0                                        [Artes (Otros estudios) ]
22.0                                                   [Humanidades ]
23.0                                                       [Lenguas ]
31.0       [Ciencias sociales y del comportamiento (Otros estudios) ]
32.0                                    [Periodismo y documentación ]
41.0                    [Negocios y administración (Otros estudios) ]
42.0                                                       [Derecho ]
51.0                                           [Ciencias de la vida ]
52.0                

In [222]:
df['ambito_g1_'].value_counts()

Ingeniería y profesiones afines                             1179
Dirección y administración                                   823
Ciencias de la vida                                          749
Lenguas                                                      681
Educación (Otros estudios)                                   616
Arquitectura y construcción                                  611
Ciencias químicas, físicas y geológicas                      581
Derecho                                                      552
Humanidades                                                  550
Salud (Otros estudios)                                       430
Informática                                                  420
Artes (Otros estudios)                                       356
Enfermería                                                   349
Psicología                                                   338
No conocido                                                  331
Actividades físicas y dep

# Nueva variable 'rama_g1'
**Rama de conocimiento de la primera titulación de grado** universitario.

In [223]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df['rama_g1']=df['ambito_g1'].astype('float').replace(D_ambito_a_rama)

In [224]:
df['rama_g1'].unique()

array([5., 3., 4., 2., 1., 6., 0.])

In [225]:
traduce('rama_g1')

                             aux_traduce
                                  unique
rama_g1                                 
0.0            [No tiene o no informada]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]
6.0                          [Servicios]


In [226]:
df[['EST_B13_AMB1_','rama_g1_']].groupby('EST_B13_AMB1_').agg(['unique']).head(15) # Comprobación 

,rama_g1_
,unique
EST_B13_AMB1_,
Actividades físicas y deportivas,[Servicios]
Agricultura y ganadería,[Ingeniería y arquitectura]
Arquitectura y construcción,[Ingeniería y arquitectura]
Artes (Otros estudios),[Artes y humanidades]
Ciencias de la vida,[Ciencias]
"Ciencias químicas, físicas y geológicas",[Ciencias]
Ciencias sociales y del comportamiento (Otros estudios),[Ciencias sociales y jurídicas]
Derecho,[Ciencias sociales y jurídicas]


In [227]:
# Actualización variables 'n_rama' (rama=art_hum, ciencias, cc_soc_ju, ing_arq, cc_salud)
actualiza_ramas('rama_g1')

In [228]:
df[L_ramas+['rama_g1_']].groupby(L_ramas).agg(['unique']) # Comprobación

rama_g1_
                                                                                                                       unique
n_art_hum n_ciencias n_cc_soc_ju n_ing_arq n_cc_salud n_servicios                                                            
0         0          0           0         0          1                                             [No tiene o no informada]
                                                      2                                                           [Servicios]
                                           1          1                                     [Ciencias de la salud, Servicios]
                                           2          0                                                [Ciencias de la salud]
                                 1         0          1                                [Servicios, Ingeniería y arquitectura]
                                           1          0                     [Ciencias de la salud, Ingeniería y arquitectura]
                                 2         0          0                                           [Ingeniería y arquitectura]
                     1           0         0          0                                             [No tiene o no informada]
                                                      1                            [Servicios, Ciencias sociales y jurídicas]
                                           1          0                 [Ciencias sociales y jurídicas, Ciencias de la salud]
                                 1         0          0            [Ingeniería y arquitectura, Ciencias sociales y jurídicas]
                     2           0         0          0                                       [Ciencias sociales y jurídicas]
          1          0           0         0          0                                             [No tiene o no informada]
                                                      1                                                 [Ciencias, Servicios]
                                           1          0                                      [Ciencias, Ciencias de la salud]
                                 1         0          0                                 [Ciencias, Ingeniería y arquitectura]
                     1           0         0          0                             [Ciencias, Ciencias sociales y jurídicas]
          2          0           0         0          0                                                            [Ciencias]
1         0          0           0         0          0                                             [No tiene o no informada]
                                                      1                                      [Servicios, Artes y humanidades]
                                           1          0                           [Artes y humanidades, Ciencias de la salud]
                                 1         0          0                      [Artes y humanidades, Ingeniería y arquitectura]
                     1           0         0          0                  [Artes y humanidades, Ciencias sociales y jurídicas]
          1          0           0         0          0                                       [Artes y humanidades, Ciencias]
2         0          0           0         0          0                                                 [Artes y humanidades]

## 'EST_B14_1'     
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente)   

**Momento en que terminó la primera titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [229]:
df['EST_B14_1'].sort_values().unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '4', '5', '6', '7', '9'], dtype=object)

In [230]:
# Ajuste en diccionario
D_dict['EST_B14_1']=diccionario('T1ESTMN')
traduce('EST_B14_1',save=False)

                      aux_traduce
                           unique
EST_B14_1                        
                   [No aplicable]
1            [Antes del año 2000]
2             [Entre 2000 y 2010]
3                       [En 2011]
4                       [En 2012]
5                       [En 2013]
6                       [En 2014]
7          [Después del año 2014]
9                         [NS/NC]


In [231]:
re_otros_estudios('EST_B14_1',trad=False)

[ 1.  2.  3.  4.  5.  6.  7. nan]


## 'EST_B15_1'     
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente)  

**Dónde estudió la segunda titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [232]:
df['EST_B15_1'].sort_values().unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [233]:
traduce('EST_B15_1')

                                                   aux_traduce
                                                        unique
EST_B15_1                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [234]:
re_otros_estudios('EST_B15_1')

[ 1.  2.  3. nan]


## Nueva variable: 'extr_proxy'
Variable proxy de **estudios al menos en parte en el extranjero**. 

In [235]:
df['extr_proxy']=0
df['extr_proxy'].mask((df['EST_B15_1']==2)|(df['EST_B15_1']==3),1,inplace=True)
df[['EST_B15_1_','extr_proxy']].groupby('EST_B15_1_',dropna=False).agg(['unique']) # Comprobación

,extr_proxy
,unique
EST_B15_1_,
Completamente en España,[0]
Completamente en el extranjero,[1]
Una parte en España y otra parte en el extranjero,[1]
NaN,[0]


In [236]:
traduce('extr_proxy')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]


In [237]:
def actualiza_extr_proxy(col):
    """Actualiza la variable 'extr_proxy' añadiendo los estudios en el extranjero declarados en la variable col. 
    """
    df['extr_proxy'].mask((df[col]==2)|(df[col]==3),1,inplace=True)
    traduce('extr_proxy')
    print (df[[str(col),'extr_proxy_']].groupby(str(col),dropna=False).agg(['unique'])) # Comprobación
    return

## 'EST_B13_AMB2' (redefinida como 'ambito_g2')     
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente) 

**Ambito de estudio del segundo grado**, diplomatura, licenciatura o equivalente en el que está titulado.

In [238]:
df['EST_B13_AMB2'].sort_values().unique() # No informados como '    ' o '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '0211', '022 ', '023 ',
       '031 ', '0311', '0313', '032 ', '041 ', '0413', '042 ', '051 ',
       '052 ', '053 ', '054 ', '061 ', '071 ', '072 ', '073 ', '081 ',
       '082 ', '084 ', '091 ', '0912', '0913', '092 ', '1014', '1015',
       '109 ', '9999'], dtype=object)

In [239]:
# Ajuste en formato:
df['EST_B13_AMB2']=df['EST_B13_AMB2'].str.strip()
# Ajuste en diccionario
D_dict['EST_B13_AMB2']=diccionario('T4AMBIT')
traduce('EST_B13_AMB2')

                                                             aux_traduce
                                                                  unique
EST_B13_AMB2                                                            
                                                                      []
011                                        [Educación (Otros estudios) ]
0112                      [Formación de docentes de enseñanza infantil ]
0113                      [Formación de docentes de enseñanza primaria ]
021                                            [Artes (Otros estudios) ]
0211                  [Técnicas audiovisuales y medios de comunicación ]
022                                                       [Humanidades ]
023                                                           [Lenguas ]
031           [Ciencias sociales y del comportamiento (Otros estudios) ]
0311                                                         [Economía ]
0313                                               

In [240]:
re_otros_estudios('EST_B13_AMB2','ambito_g2',2,blank='',nsnc='9999')

                                                                                                                                                                                                                                                                                                           ambito_g2_
                                                                                                                                                                                                                                                                                                               unique
n_grados                                                                                                                                                                                                                                                                                                             
0                                                                     

In [241]:
traduce('ambito_g2')

                                                          aux_traduce
                                                               unique
ambito_g2                                                            
0.0                                                        [No tiene]
1.0                                                     [No conocido]
11.0                                    [Educación (Otros estudios) ]
21.0                                        [Artes (Otros estudios) ]
22.0                                                   [Humanidades ]
23.0                                                       [Lenguas ]
31.0       [Ciencias sociales y del comportamiento (Otros estudios) ]
32.0                                    [Periodismo y documentación ]
41.0                    [Negocios y administración (Otros estudios) ]
42.0                                                       [Derecho ]
51.0                                           [Ciencias de la vida ]
52.0                

## Nueva variable 'rama_g2'
**Rama de conocimiento de la segunda titulación de grado** universitario.

In [242]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df['rama_g2']=df['ambito_g2'].replace(D_ambito_a_rama)

In [243]:
df['rama_g2'].unique()

array([0., 3., 4., 1., 2., 6., 5.])

In [244]:
traduce('rama_g2')

                             aux_traduce
                                  unique
rama_g2                                 
0.0            [No tiene o no informada]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]
6.0                          [Servicios]


In [245]:
# Actualización variables 'n_rama' (rama=art_hum, ciencias, cc_soc_ju, ing_arq, cc_salud)
actualiza_ramas('rama_g2')

In [246]:
for rama in L_ramas:
    print (rama,df[rama].unique()) # Inspección (comprobación no rigurosa)

n_art_hum [0 1 2 3]
n_ciencias [0 2 1 3]
n_cc_soc_ju [0 3 1 2]
n_ing_arq [0 2 3 1]
n_cc_salud [2 0 1 3]
n_servicios [0 1 2 3]


## 'EST_B14_2'     
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente) 

**Momento en que terminó la segunda titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [247]:
df['EST_B14_2'].sort_values().unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '4', '5', '6', '7', '9'], dtype=object)

In [248]:
# Ajuste en diccionario
D_dict['EST_B14_2']=diccionario('T1ESTMN')
traduce('EST_B14_2')

                      aux_traduce
                           unique
EST_B14_2                        
                   [No aplicable]
1            [Antes del año 2000]
2             [Entre 2000 y 2010]
3                       [En 2011]
4                       [En 2012]
5                       [En 2013]
6                       [En 2014]
7          [Después del año 2014]
9                         [NS/NC]


In [249]:
re_otros_estudios('EST_B14_2')

[ 1.  2.  3.  4.  5.  6.  7. nan]


## 'EST_B15_2'     
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente) 

**Dónde estudió la segunda titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [250]:
df['EST_B15_2'].sort_values().unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [251]:
traduce('EST_B15_2')

                                                   aux_traduce
                                                        unique
EST_B15_2                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [252]:
re_otros_estudios('EST_B15_2')

[ 1.  2.  3. nan]


In [253]:
actualiza_extr_proxy('EST_B15_2') 

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B15_2                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B13_AMB3' (redefinida como 'ambito_g3') 
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente) 

**Ambito de estudio del tercer grado**, diplomatura, licenciatura o equivalente en el que está titulado.

In [254]:
reset('EST_B13_AMB3')

In [255]:
df['EST_B13_AMB3'].sort_values().unique() # No informados como '    ' o '9999'

array(['    ', '011 ', '0112', '0113', '021 ', '0211', '022 ', '023 ',
       '031 ', '0311', '0313', '032 ', '041 ', '0413', '042 ', '051 ',
       '053 ', '054 ', '061 ', '071 ', '072 ', '073 ', '081 ', '082 ',
       '091 ', '0912', '0913', '092 ', '1014', '1015', '9999'],
      dtype=object)

In [256]:
# Ajuste en formato:
df['EST_B13_AMB3']=df['EST_B13_AMB3'].str.strip()
# Ajuste en diccionario
D_dict['EST_B13_AMB3']=diccionario('T4AMBIT')
traduce('EST_B13_AMB3')

                                                             aux_traduce
                                                                  unique
EST_B13_AMB3                                                            
                                                                      []
011                                        [Educación (Otros estudios) ]
0112                      [Formación de docentes de enseñanza infantil ]
0113                      [Formación de docentes de enseñanza primaria ]
021                                            [Artes (Otros estudios) ]
0211                  [Técnicas audiovisuales y medios de comunicación ]
022                                                       [Humanidades ]
023                                                           [Lenguas ]
031           [Ciencias sociales y del comportamiento (Otros estudios) ]
0311                                                         [Economía ]
0313                                               

In [257]:
re_otros_estudios('EST_B13_AMB3','ambito_g3',3,blank='',nsnc='9999')

                                                                                                                                                                                                                                                                                                           ambito_g3_
                                                                                                                                                                                                                                                                                                               unique
n_grados                                                                                                                                                                                                                                                                                                             
0                                                                     

In [258]:
traduce('ambito_g3')

                                                          aux_traduce
                                                               unique
ambito_g3                                                            
0.0                                                        [No tiene]
1.0                                                     [No conocido]
11.0                                    [Educación (Otros estudios) ]
21.0                                        [Artes (Otros estudios) ]
22.0                                                   [Humanidades ]
23.0                                                       [Lenguas ]
31.0       [Ciencias sociales y del comportamiento (Otros estudios) ]
32.0                                    [Periodismo y documentación ]
41.0                    [Negocios y administración (Otros estudios) ]
42.0                                                       [Derecho ]
51.0                                           [Ciencias de la vida ]
53.0                

## Nueva variable 'rama_g3'
**Rama de conocimiento de la tercera titulación de grado** universitario.

In [259]:
# Columna nueva: rama de conocimiento asociada al ámbito declarado: 
df['rama_g3']=df['ambito_g3'].replace(D_ambito_a_rama)

In [260]:
df['rama_g3'].unique()

array([0., 4., 3., 1., 2., 5., 6.])

In [261]:
traduce('rama_g3')

                             aux_traduce
                                  unique
rama_g3                                 
0.0            [No tiene o no informada]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]
6.0                          [Servicios]


In [262]:
# Actualización variables 'n_rama' (rama=art_hum, ciencias, cc_soc_ju, ing_arq, cc_salud)
actualiza_ramas('rama_g3')

In [263]:
for rama in L_ramas:
    print (rama,df[rama].unique()) # Inspección (comprobación no rigurosa)

n_art_hum [0 1 2 4 3]
n_ciencias [0 2 1 3 4]
n_cc_soc_ju [0 3 1 2 4]
n_ing_arq [0 2 3 1 4]
n_cc_salud [2 0 1 3 4]
n_servicios [0 1 2 3]


## 'EST_B14_3'     
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente)

**Momento en que terminó la tercera titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [264]:
df['EST_B14_3'].sort_values().unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '4', '5', '6', '7', '9'], dtype=object)

In [265]:
# Ajuste en diccionario
D_dict['EST_B14_3']=diccionario('T1ESTMN')
traduce('EST_B14_3')

                      aux_traduce
                           unique
EST_B14_3                        
                   [No aplicable]
1            [Antes del año 2000]
2             [Entre 2000 y 2010]
3                       [En 2011]
4                       [En 2012]
5                       [En 2013]
6                       [En 2014]
7          [Después del año 2014]
9                         [NS/NC]


In [266]:
re_otros_estudios('EST_B14_3')

[ 1.  2.  3.  4.  5.  6.  7. nan]


## 'EST_B15_3'     
(Solo para quienes declaran haber terminado estudios de grado, diplomatura, licenciatura o equivalente) 

**Dónde estudió la tercera titulación de grado**, diplomatura, licenciatura o equivalente mencionada.

In [267]:
df['EST_B15_3'].sort_values().unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [268]:
traduce('EST_B15_3')

                                                   aux_traduce
                                                        unique
EST_B15_3                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [269]:
re_otros_estudios('EST_B15_3')

[ 1.  2.  3. nan]


In [270]:
actualiza_extr_proxy('EST_B15_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B15_3                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B16'  (redefinida como 'n_master')   
(Solo para quienes declaran haber terminado estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-)  

**Número de títulos de máster** universitario que ha completado (sin incluir el que motiva esta encuesta).

In [271]:
df['EST_B16'].unique() # No informados como ' '

array([' ', '1', '2', '3'], dtype=object)

In [272]:
traduce('EST_B16')

            aux_traduce
                 unique
EST_B16                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


In [273]:
df[['EST_B16','varios_master']].groupby('varios_master',dropna=False).agg(['unique'])

,EST_B16
,unique
varios_master,
0,[ ]
1,"[1, 2, 3]"


##  Nueva variable 'n_master'     
**Número de títulos de máster** (incluyendo aquel que motiva esta encuesta).

In [274]:
df['n_master']=df['EST_B16'].replace({' ':0}).astype('int')+1
a_lista('n_master')
df[['n_master','EST_B16']].groupby('n_master').agg(['unique']) # Comprobación

,EST_B16
,unique
n_master,
1,[ ]
2,[1]
3,[2]
4,[3]


In [275]:
df[['n_master','varios_master_']].groupby('varios_master_',dropna=False).agg(['unique']) # Comprobación

,n_master
,unique
varios_master_,
No,[1]
Sí,"[2, 3, 4]"


In [276]:
# Establece el valor de 'otros_estudios' referido a estudios de máster para función re_otros_estudios
otros_estudios='n_master'

## 'EST_B17_M1'  (redefinida como 'rama_m2')   
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-) 

**Rama de conocimiento del segundo título de máster** mencionado como completado.

In [277]:
df['EST_B17_M1'].unique() # No informados como ' '

array([' ', '3', '4', '1', '5', '2'], dtype=object)

In [278]:
traduce('EST_B17_M1')

                                aux_traduce
                                     unique
EST_B17_M1                                 
                                        [ ]
1                             [No conocido]
2                                [Ciencias]
3           [Ciencias sociales y jurídicas]
4               [Ingeniería y arquitectura]
5                    [Ciencias de la salud]


In [279]:
re_otros_estudios('EST_B17_M1','rama_m2',2)

                                                                                                         rama_m2_
                                                                                                           unique
n_master                                                                                                         
1                                                                                                           [nan]
2         [Ciencias sociales y jurídicas, Ingeniería y arquitectura, No conocido, Ciencias de la salud, Ciencias]
3         [Ciencias sociales y jurídicas, Ingeniería y arquitectura, No conocido, Ciencias de la salud, Ciencias]
4         [Ciencias, Ciencias sociales y jurídicas, Ingeniería y arquitectura, Ciencias de la salud, No conocido]
                                                                                                         rama_m2_
                                                                                        

In [280]:
D_dict['rama_m2']=D_dict['rama_g2']
traduce('rama_m2')

                             aux_traduce
                                  unique
rama_m2                                 
0.0            [No tiene o no informada]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


In [281]:
proporciones('rama_m2_')

,rama_m2_
No tiene o no informada,0.684055
Ciencias sociales y jurídicas,0.118784
Ciencias de la salud,0.057215
Artes y humanidades,0.054254
Ciencias,0.044849
Ingeniería y arquitectura,0.040843


In [282]:
# Actualización variables 'n_rama' (rama=art_hum, ciencias, cc_soc_ju, ing_arq, cc_salud)
actualiza_ramas('rama_m2')

In [283]:
for rama in L_ramas:
    print (rama, df[rama].unique()) # Inspección (comprobación no rigurosa)

n_art_hum [0 1 2 3 4 5]
n_ciencias [0 2 1 3 4]
n_cc_soc_ju [0 3 1 2 4 5]
n_ing_arq [0 2 4 1 3 5]
n_cc_salud [2 0 1 3 4]
n_servicios [0 1 2 3]


## 'EST_B18_1'  
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-) 

**Momento en que terminó el segundo título de máster** mencionado como completado.

In [284]:
df['EST_B18_1'].unique() # No informados como ' ' o '9'

array([' ', '3', '1', '2', '9'], dtype=object)

In [285]:
# Ajuste en diccionario
D_dict['EST_B18_1']=diccionario('T2ESTMN')
traduce('EST_B18_1')

                                                                              aux_traduce
                                                                                   unique
EST_B18_1                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [286]:
re_otros_estudios('EST_B18_1')

[ 1.  2.  3. nan]


## 'EST_B19_1'     
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-) 

**Dónde estudió el segundo título de máster** mencionado como completado.

In [287]:
df['EST_B19_1'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [288]:
traduce('EST_B19_1')

                                                   aux_traduce
                                                        unique
EST_B19_1                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [289]:
re_otros_estudios('EST_B19_1')

[ 1.  2.  3. nan]


In [290]:
actualiza_extr_proxy('EST_B19_1')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B19_1                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B17_M2'  ( redefinida como 'rama_m3')   
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-)   

**Rama de conocimiento del tercer título de máster** mencionado como completado.  

In [291]:
df['EST_B17_M2'].unique() # No informados como ' '

array([' ', '3', '4', '2', '1', '5'], dtype=object)

In [292]:
traduce('EST_B17_M2')

                                aux_traduce
                                     unique
EST_B17_M2                                 
                                        [ ]
1                             [No conocido]
2                                [Ciencias]
3           [Ciencias sociales y jurídicas]
4               [Ingeniería y arquitectura]
5                    [Ciencias de la salud]


In [293]:
re_otros_estudios('EST_B17_M2','rama_m3',3)

                                                                                                         rama_m3_
                                                                                                           unique
n_master                                                                                                         
1                                                                                                           [nan]
2                                                                                                           [nan]
3         [Ciencias sociales y jurídicas, Ingeniería y arquitectura, No conocido, Ciencias de la salud, Ciencias]
4         [Ciencias, Ciencias sociales y jurídicas, Ingeniería y arquitectura, Ciencias de la salud, No conocido]
                                                                                                         rama_m3_
                                                                                        

In [294]:
D_dict['rama_m3']=D_dict['rama_g2']
traduce('rama_m3')

                             aux_traduce
                                  unique
rama_m3                                 
0.0            [No tiene o no informada]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


In [295]:
# Actualización variables 'n_rama' (rama=art_hum, ciencias, cc_soc_ju, ing_arq, cc_salud)
actualiza_ramas('rama_m3')

In [296]:
for rama in L_ramas:
    print (rama, df[rama].unique()) # Inspección (comprobación no rigurosa)

n_art_hum [0 1 2 3 4 5 6]
n_ciencias [0 2 1 3 4]
n_cc_soc_ju [0 3 1 2 4 5 6]
n_ing_arq [0 2 4 1 3 5]
n_cc_salud [2 0 1 3 4 5]
n_servicios [0 1 2 3]


## 'EST_B18_2'  
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-)  

**Momento en que terminó el tercer título de máster** mencionado como completado.

In [297]:
df['EST_B18_2'].unique() # No informados como ' ' o '9'

array([' ', '3', '1', '2', '9'], dtype=object)

In [298]:
# Ajuste en diccionario
D_dict['EST_B18_2']=diccionario('T2ESTMN')
traduce('EST_B18_2')

                                                                              aux_traduce
                                                                                   unique
EST_B18_2                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [299]:
re_otros_estudios('EST_B18_2')

[ 1.  2.  3. nan]


## 'EST_B19_2'     
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-) 

**Dónde estudió el tercer título de máster** mencionado como completado.

In [300]:
df['EST_B19_2'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [301]:
traduce('EST_B19_2')

                                                   aux_traduce
                                                        unique
EST_B19_2                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [302]:
re_otros_estudios('EST_B19_2')

[ 1.  2.  3. nan]


In [303]:
actualiza_extr_proxy('EST_B19_2')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B19_2                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B17_M3'  (redefinida como 'rama_m4')   
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-)  

**Rama de conocimiento del cuarto título de máster** mencionado como completado.

In [304]:
df['EST_B17_M3'].unique() # No informados como ' '

array([' ', '2', '3', '4', '5', '1'], dtype=object)

In [305]:
traduce('EST_B17_M3')

                                aux_traduce
                                     unique
EST_B17_M3                                 
                                        [ ]
1                             [No conocido]
2                                [Ciencias]
3           [Ciencias sociales y jurídicas]
4               [Ingeniería y arquitectura]
5                    [Ciencias de la salud]


In [306]:
re_otros_estudios('EST_B17_M3','rama_m4',4)

                                                                                                         rama_m4_
                                                                                                           unique
n_master                                                                                                         
1                                                                                                           [nan]
2                                                                                                           [nan]
3                                                                                                           [nan]
4         [Ciencias, Ciencias sociales y jurídicas, Ingeniería y arquitectura, Ciencias de la salud, No conocido]
                                                                                                         rama_m4_
                                                                                        

In [307]:
D_dict['rama_m4']=D_dict['rama_g2']
traduce('rama_m4')

                             aux_traduce
                                  unique
rama_m4                                 
0.0            [No tiene o no informada]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


In [308]:
# Actualización variables 'n_rama' (rama=art_hum, ciencias, cc_soc_ju, ing_arq, cc_salud)
actualiza_ramas('rama_m4')

In [309]:
for rama in L_ramas:
    print (rama, df[rama].unique()) # Inspección (comprobación no rigurosa)

n_art_hum [0 1 2 3 4 5 6]
n_ciencias [0 2 1 3 4 5]
n_cc_soc_ju [0 3 1 2 4 5 6]
n_ing_arq [0 2 4 1 3 5]
n_cc_salud [2 0 1 3 4 5]
n_servicios [0 1 2 3]


## 'EST_B18_3'  
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-)   

**Momento en que terminó el cuarto título de máster** mencionado como completado.

In [310]:
df['EST_B18_3'].unique() # No informados como ' '

array([' ', '1', '3', '2'], dtype=object)

In [311]:
# Ajuste en diccionario
D_dict['EST_B18_3']=diccionario('T2ESTMN')
traduce('EST_B18_3')

                                                                              aux_traduce
                                                                                   unique
EST_B18_3                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [312]:
re_otros_estudios('EST_B18_3')

[ 1.  2.  3. nan]


## 'EST_B19_3'     
(Solo para quienes declaran haber terminado otros estudios de máster -respuesta 1 en variable 'EST_B11_2' (o 'varios_master')-) 

**Dónde estudió el cuarto título de máster** mencionado como completado.

In [313]:
df['EST_B19_3'].unique() # No informados como ' '

array([' ', '1', '2', '3'], dtype=object)

In [314]:
traduce('EST_B19_3')

                                                   aux_traduce
                                                        unique
EST_B19_3                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]


In [315]:
re_otros_estudios('EST_B19_3')

[ 1.  2.  3. nan]


In [316]:
actualiza_extr_proxy('EST_B19_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B19_3                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


In [317]:
D_describe['rama_m4']='Rama de conocimiento del cuarto titulo de master'

## 'EST_B21'  (redefinida como 'rama_doc')   
(Solo para quienes declaran haber terminado estudios de doctorado -respuesta 1 en variable 'EST_B11_3' (o'doctor')-)  

**Rama de conocimiento de sus estudios de doctorado** (si tiene más de un título de doctor, del que considere el principal).

In [318]:
df['EST_B21'].unique() # No informados como ' '

array([' ', '5', '2', '4', '3', '1'], dtype=object)

In [319]:
traduce('EST_B21')

                             aux_traduce
                                  unique
EST_B21                                 
                                     [ ]
1                          [No conocido]
2                             [Ciencias]
3        [Ciencias sociales y jurídicas]
4            [Ingeniería y arquitectura]
5                 [Ciencias de la salud]


In [320]:
# Establece otros_estudios referido al título de doctor: 
otros_estudios='doctor'

In [321]:
re_otros_estudios('EST_B21','rama_doc',1)

                                                                                                      rama_doc_
                                                                                                         unique
doctor                                                                                                         
0                                                                                                         [nan]
1       [Ciencias de la salud, Ciencias, Ingeniería y arquitectura, Ciencias sociales y jurídicas, No conocido]
                                                                                                      rama_doc_
                                                                                                         unique
doctor                                                                                                         
0                                                                                                    [No

In [322]:
D_dict['rama_doc']=D_dict['rama_g2']
traduce('rama_doc')

                              aux_traduce
                                   unique
rama_doc                                 
0.0             [No tiene o no informada]
1.0                 [Artes y humanidades]
2.0                            [Ciencias]
3.0       [Ciencias sociales y jurídicas]
4.0           [Ingeniería y arquitectura]
5.0                [Ciencias de la salud]


In [323]:
# Actualización variables 'n_rama' (rama=art_hum, ciencias, cc_soc_ju, ing_arq, cc_salud)
actualiza_ramas('rama_doc')

In [324]:
for rama in L_ramas:
    print (df[rama].unique()) # Inspección (comprobación no rigurosa)

[0 1 2 3 4 5 6 7]
[0 2 1 3 4 5 6]
[0 3 1 2 4 5 6 7]
[0 2 4 1 3 5 6]
[2 0 1 3 4 5 6]
[0 1 2 3]


## 'EST_B22'  
(Solo para quienes declaran haber terminado estudios de doctorado -respuesta 1 en variable 'EST_B11_3' (o'doctor')-)  

**Momento en que terminó sus estudios de doctorado** (si tiene más de un título de doctor, el que considere el principal).

In [325]:
df['EST_B22'].unique() # No informados como ' ' o '9'

array([' ', '3', '1', '2', '9'], dtype=object)

In [326]:
# Ajuste en diccionario
D_dict['EST_B22']=diccionario('T2ESTMN')
traduce('EST_B22')

                                                                            aux_traduce
                                                                                 unique
EST_B22                                                                                
                                                                         [No aplicable]
1                [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2        [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                               [Después de obtener la titulación que finalizó en 2014]
9                                                                               [NS/NC]


In [327]:
re_otros_estudios('EST_B22')

[ 1.  2.  3. nan]


## 'EST_B23'  
(Solo para quienes declaran haber terminado estudios de doctorado -respuesta 1 en variable 'EST_B11_3' (o'doctor')-)   

**Donde cursó sus estudios de doctorado** (si tiene más de un título de doctor, el que considere el principal).

In [328]:
df['EST_B23'].unique() # No informados como ' '

array([' ', '2', '1', '9', '3'], dtype=object)

In [329]:
traduce('EST_B23')

                                                 aux_traduce
                                                      unique
EST_B23                                                     
                                              [No aplicable]
1                                  [Completamente en España]
2        [Una parte en España y otra parte en el extranjero]
3                           [Completamente en el extranjero]
9                                                    [NS/NC]


In [330]:
re_otros_estudios('EST_B23')

[ 1.  2.  3. nan]


In [331]:
actualiza_extr_proxy('EST_B23')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                           extr_proxy_
                                                                                unique
EST_B23                                                                               
1.0      [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                  [Al menos parte de los estudios en el extranjero]
3.0                                  [Al menos parte de los estudios en el extranjero]
NaN      [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B24'  (redefinida como 'n_gsup')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Número de ciclos formativos de grado superior o estudios artísticos superiores** que ha completado.

In [332]:
df['EST_B24'].unique() # No informados como ' '

array([' ', '1', '2', '3'], dtype=object)

In [333]:
traduce('EST_B24')

            aux_traduce
                 unique
EST_B24                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


In [334]:
# Se imputa 0 a no respuesta: 
df['EST_B24'].replace({' ':0},inplace=True)
df['EST_B24']=df['EST_B24'].astype('int')
df['EST_B24'].unique()

array([0, 1, 2, 3])

In [335]:
redefine('EST_B24','n_gsup')

In [336]:
df[['n_gsup','g_sup_','est_art_']].groupby(['g_sup_','est_art_'],dropna=False).agg(['unique']) # Inspección: 

n_gsup
                    unique
g_sup_ est_art_           
No     No              [0]
       Sí        [1, 3, 2]
Sí     No        [1, 2, 3]
       Sí        [2, 1, 3]

In [337]:
# Se establece 'otros_estudios' referido a ciclos formativos o estudios artísticos superiores: 
otros_estudios='n_gsup'

## 'EST_B25_FA1'  (redefinida como 'fam_gsup1')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Familia de estudios** del primer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [338]:
df['EST_B25_FA1'].sort_values().unique() # No informados como '  ' 

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '14', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '88', '99'], dtype=object)

In [339]:
no_respuesta('EST_B25_FA1',blank='  ',nsnc='99')
traduce('EST_B25_FA1')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 14. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 88. nan]
                                                                                                                                               aux_traduce
                                                                                                                                                    unique
EST_B25_FA1                                                                                                                                               
1.0                                                                                                                                              [Agraria]
2.0                                                                                                                                    [Marítimo-pesquera]
3.0                                                                                                                       

In [340]:
re_otros_estudios('EST_B25_FA1','fam_gsup1',1,blank='  ',nsnc='99')

                                                                                                                                                                                                                                                                                                         fam_gsup1_
                                                                                                                                                                                                                                                                                                             unique
n_gsup                                                                                                                                                                                                                                                                                                             
0                                                                           

In [341]:
traduce('fam_gsup1')

                                                                                                                                             aux_traduce
                                                                                                                                                  unique
fam_gsup1                                                                                                                                               
0.0                                                                                                                                           [No tiene]
1.0                                                                                                                                        [No conocido]
2.0                                                                                                                                  [Marítimo-pesquera]
3.0                                                                               

## 'EST_B26_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)   

**Momento en que terminó** el primer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [342]:
df['EST_B26_1'].unique() # No informados como ' ' o '9'

array([' ', '1', '3', '9', '2'], dtype=object)

In [343]:
# Ajuste en diccionario
D_dict['EST_B26_1']=diccionario('T2ESTMN')
traduce('EST_B26_1')

                                                                              aux_traduce
                                                                                   unique
EST_B26_1                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [344]:
re_otros_estudios('EST_B26_1')

[ 1.  2.  3. nan]


## 'EST_B27_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)   

**Dónde cursó** el primer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [345]:
df['EST_B27_1'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '9'], dtype=object)

In [346]:
traduce('EST_B27_1')

                                                   aux_traduce
                                                        unique
EST_B27_1                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]
9                                                      [NS/NC]


In [347]:
re_otros_estudios('EST_B27_1')

[ 1.  2.  3. nan]


In [348]:
actualiza_extr_proxy('EST_B27_1')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B27_1                                                                               
1.0        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B25_FA2'  (redefinida como 'fam_gsup2')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Familia de estudios** del segundo ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [349]:
df['EST_B25_FA2'].sort_values().unique() # No informados como '  ' 

array(['  ', '01', '03', '04', '06', '08', '09', '10', '12', '14', '18',
       '19', '20', '21', '23', '24', '25', '88'], dtype=object)

In [350]:
no_respuesta('EST_B25_FA2',blank='  ',nsnc='99')
traduce('EST_B25_FA2')

[ 1.  3.  4.  6.  8.  9. 10. 12. 14. 18. 19. 20. 21. 23. 24. 25. 88. nan]
                                                                                                                                               aux_traduce
                                                                                                                                                    unique
EST_B25_FA2                                                                                                                                               
1.0                                                                                                                                              [Agraria]
3.0                                                                                                                             [Industrias alimentarias ]
4.0                                                                                                                                              [Quími

In [351]:
re_otros_estudios('EST_B25_FA2','fam_gsup2',2,blank='  ',nsnc='99')

                                                                                                                                                                                                                                                                                                         fam_gsup2_
                                                                                                                                                                                                                                                                                                             unique
n_gsup                                                                                                                                                                                                                                                                                                             
0                                                                           

In [352]:
D_dict['fam_gsup2']=D_dict['fam_gsup1']
traduce('fam_gsup2')

                                                                                                                                             aux_traduce
                                                                                                                                                  unique
fam_gsup2                                                                                                                                               
0.0                                                                                                                                           [No tiene]
1.0                                                                                                                                        [No conocido]
3.0                                                                                                                           [Industrias alimentarias ]
4.0                                                                               

## 'EST_B26_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Momento en que terminó** el segundo ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [353]:
df['EST_B26_2'].unique() # No informados como ' ' 

array([' ', '1', '3', '2'], dtype=object)

In [354]:
# Ajuste en diccionario
D_dict['EST_B26_2']=diccionario('T2ESTMN')
traduce('EST_B26_2')

                                                                              aux_traduce
                                                                                   unique
EST_B26_2                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [355]:
re_otros_estudios('EST_B26_2')

[ 1.  2.  3. nan]


## 'EST_B27_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)  

**Dónde cursó** el segundo ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [356]:
df['EST_B27_2'].unique() # No informado como ' ' 

array([' ', '1', '2', '3'], dtype=object)

In [357]:
traduce('EST_B27_2')

                                                   aux_traduce
                                                        unique
EST_B27_2                                                     
                                                [No aplicable]
1                                    [Completamente en España]
2          [Una parte en España y otra parte en el extranjero]
3                             [Completamente en el extranjero]


In [358]:
re_otros_estudios('EST_B27_2')

[ 1.  2.  3. nan]


In [359]:
actualiza_extr_proxy('EST_B27_2')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B27_2                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
2.0                                    [Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B25_FA3'  (redefinida como 'fam_gsup3')   
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)   

**Familia de estudios** del tercer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [360]:
df['EST_B25_FA3'].unique() # No informados como '  ' o '99'

array(['  ', '88', '20', '10', '06', '18'], dtype=object)

In [361]:
no_respuesta('EST_B25_FA3',blank='  ',nsnc='99')
traduce('EST_B25_FA3')

[ 6. 10. 18. 20. 88. nan]
                                                                                                                                               aux_traduce
                                                                                                                                                    unique
EST_B25_FA3                                                                                                                                               
6.0                                                                                                                                             [Sanidad ]
10.0                                                                                                                          [Electricidad y electrónica]
18.0                                                                                                                                     [Artes gráficas ]
20.0                                        

In [362]:
re_otros_estudios('EST_B25_FA3','fam_gsup3',3,blank='  ',nsnc='99')

                                                                                                                                                                                                                                fam_gsup3_
                                                                                                                                                                                                                                    unique
n_gsup                                                                                                                                                                                                                                    
0                                                                                                                                                                                                                                    [nan]
1                                                           

In [363]:
D_dict['fam_gsup3']=D_dict['fam_gsup1']
traduce('fam_gsup3')

                                                                                                                                             aux_traduce
                                                                                                                                                  unique
fam_gsup3                                                                                                                                               
0.0                                                                                                                                           [No tiene]
6.0                                                                                                                                           [Sanidad ]
10.0                                                                                                                        [Electricidad y electrónica]
18.0                                                                              

## 'EST_B26_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)    

**Momento en que terminó** el tercer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [364]:
df['EST_B26_3'].unique() # No informados como ' ' 

array([' ', '1', '2', '3'], dtype=object)

In [365]:
# Ajuste en diccionario
D_dict['EST_B26_3']=diccionario('T2ESTMN')
traduce('EST_B26_3')

                                                                              aux_traduce
                                                                                   unique
EST_B26_3                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]


In [366]:
re_otros_estudios('EST_B26_3')

[ 1.  2.  3. nan]


## 'EST_B27_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado superior y/o estudios artísticos
superiores -respuesta 1 en variable 'EST_B11_5' (o 'g_sup') o  en 'EST_B11_4' (o 'est_art')-)    

**Dónde cursó** el tercer ciclo formativo de grado superior (formación profesional, artes plásticas y diseño) o enseñanzas deportivas de grado superior mencionado.

In [367]:
df['EST_B27_3'].unique() # No informados como ' ' 

array([' ', '1'], dtype=object)

In [368]:
traduce('EST_B27_3')

                         aux_traduce
                              unique
EST_B27_3                           
                      [No aplicable]
1          [Completamente en España]


In [369]:
re_otros_estudios('EST_B27_3')

[ 1. nan]


In [370]:
actualiza_extr_proxy('EST_B27_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B27_3                                                                               
1.0                                                         [No estudió fuera de España]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B28'  (redefinida como 'n_gmedio')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Número de ciclos formativos de grado medio** que ha completado.

In [371]:
df['EST_B28'].unique() # No informados como ' '

array([' ', '1', '2', '3'], dtype=object)

In [372]:
traduce('EST_B28')

            aux_traduce
                 unique
EST_B28                
         [No aplicable]
1                 [Uno]
2                [Dos ]
3          [Tres o más]


In [373]:
redefine('EST_B28','n_gmedio')

In [374]:
df[['n_gmedio','g_medio']].groupby('g_medio',dropna=False).agg(['unique']) # Inspección: 

,n_gmedio
,unique
g_medio,
0,[ ]
1,"[1, 2, 3]"


In [375]:
# Se imputa valor 0 a quienes declararon no haber completado ciclos formativos de grado medio: 
df['n_gmedio'].mask(df['g_medio']==0,0,inplace=True)
df['n_gmedio']=df['n_gmedio'].astype('int')
df[['n_gmedio','g_medio']].groupby('g_medio',dropna=False).agg(['unique']) # Comprobación: 

,n_gmedio
,unique
g_medio,
0,[0]
1,"[1, 2, 3]"


In [376]:
# Se establece otros_estudios referido a ciclos formativos de grado medio: 
otros_estudios='n_gmedio'

## 'EST_B29_FA1'  (redefinida como 'fam_gmedio1')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Familia de estudios** del primer ciclo formativo de grado medio mencionado.

In [377]:
df['EST_B29_FA1'].sort_values().unique() # No informados como '  ' o '99'

array(['  ', '01', '03', '04', '05', '06', '08', '09', '10', '12', '14',
       '17', '18', '19', '20', '21', '22', '23', '24', '25', '88', '99'],
      dtype=object)

In [378]:
no_respuesta('EST_B29_FA1',blank='  ',nsnc='99')
traduce('EST_B29_FA1')

[ 1.  3.  4.  5.  6.  8.  9. 10. 12. 14. 17. 18. 19. 20. 21. 22. 23. 24.
 25. 88. nan]
                                                                                     aux_traduce
                                                                                          unique
EST_B29_FA1                                                                                     
1.0                                                                                    [Agraria]
3.0                                                                   [Industrias alimentarias ]
4.0                                                                                    [Química]
5.0                                                                           [Imagen personal ]
6.0                                                                                   [Sanidad ]
8.0                                                                      [Fabricación mecánica ]
9.0                                     

In [379]:
re_otros_estudios('EST_B29_FA1','fam_gmedio1',1,blank='  ',nsnc='99')

                                                                                                                                                                                                                                                                                                         fam_gmedio1_
                                                                                                                                                                                                                                                                                                               unique
n_gmedio                                                                                                                                                                                                                                                                                                             
0                                                                     

In [380]:
#D_dict['fam_gmedio1'][0]='No tiene'
traduce('fam_gmedio1')

                                                                                     aux_traduce
                                                                                          unique
fam_gmedio1                                                                                     
0.0                                                                                   [No tiene]
1.0                                                                                [No conocido]
3.0                                                                   [Industrias alimentarias ]
4.0                                                                                    [Química]
5.0                                                                           [Imagen personal ]
6.0                                                                                   [Sanidad ]
8.0                                                                      [Fabricación mecánica ]
9.0                           

## 'EST_B30_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Momento en que terminó** el primer ciclo formativo de grado medio mencionado.

In [381]:
df['EST_B30_1'].unique() # No informados como ' ' o '9'

array([' ', '1', '3', '9', '2'], dtype=object)

In [382]:
# Ajuste en diccionario
D_dict['EST_B30_1']=diccionario('T2ESTMN')
traduce('EST_B30_1')

                                                                              aux_traduce
                                                                                   unique
EST_B30_1                                                                                
                                                                           [No aplicable]
1                  [Antes de comenzar los estudios de la titulación que finalizó en 2014]
2          [Durante la realización de los estudios de la titulación que finalizó en 2014]
3                                 [Después de obtener la titulación que finalizó en 2014]
9                                                                                 [NS/NC]


In [383]:
re_otros_estudios('EST_B30_1')

[ 1.  2.  3. nan]


## 'EST_B31_1'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Dónde cursó** el primer ciclo formativo de grado medio mencionado.

In [384]:
df['EST_B31_1'].unique() # No informados como ' ' o '9'

array([' ', '1', '3'], dtype=object)

In [385]:
traduce('EST_B31_1')

                                aux_traduce
                                     unique
EST_B31_1                                  
                             [No aplicable]
1                 [Completamente en España]
3          [Completamente en el extranjero]


In [386]:
re_otros_estudios('EST_B31_1')

[ 1.  3. nan]


In [387]:
actualiza_extr_proxy('EST_B31_1')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B31_1                                                                               
1.0        [No estudió fuera de España, Al menos parte de los estudios en el extranjero]
3.0                                    [Al menos parte de los estudios en el extranjero]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B29_FA2'  (redesignada como 'fam_gmedio2')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Familia de estudios** del segundo ciclo formativo de grado medio mencionado.

In [388]:
df['EST_B29_FA2'].sort_values().unique() # No informados como '  '

array(['  ', '06', '10', '18', '19', '25', '88'], dtype=object)

In [389]:
no_respuesta('EST_B29_FA2',blank='  ',nsnc='99')
traduce('EST_B29_FA2')

[ 6. 10. 18. 19. 25. 88. nan]
                                                                                     aux_traduce
                                                                                          unique
EST_B29_FA2                                                                                     
6.0                                                                                   [Sanidad ]
10.0                                                                [Electricidad y electrónica]
18.0                                                                           [Artes gráficas ]
19.0                                                                           [Imagen y sonido]
25.0         [Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)]
88.0                                                       [Enseñanzas artísticas profesionales]


In [390]:
re_otros_estudios('EST_B29_FA2','fam_gmedio2',2,blank='  ')

                                                                                                                                                                                              fam_gmedio2_
                                                                                                                                                                                                    unique
n_gmedio                                                                                                                                                                                                  
0                                                                                                                                                                                                    [nan]
1                                                                                                                                                                                           

In [391]:
D_dict['fam_gmedio2']=D_dict['fam_gmedio1']
traduce('fam_gmedio2')

                                                                                     aux_traduce
                                                                                          unique
fam_gmedio2                                                                                     
0.0                                                                                   [No tiene]
6.0                                                                                   [Sanidad ]
10.0                                                                [Electricidad y electrónica]
18.0                                                                           [Artes gráficas ]
19.0                                                                           [Imagen y sonido]
25.0         [Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)]
88.0                                                       [Enseñanzas artísticas profesionales]


## 'EST_B30_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Momento en que terminó** el segundo ciclo formativo de grado medio mencionado.

In [392]:
df['EST_B30_2'].unique() # No informados como ' ' 

array([' ', '1', '3'], dtype=object)

In [393]:
# Ajuste en diccionario
D_dict['EST_B30_2']=diccionario('T2ESTMN')
traduce('EST_B30_2')

                                                                      aux_traduce
                                                                           unique
EST_B30_2                                                                        
                                                                   [No aplicable]
1          [Antes de comenzar los estudios de la titulación que finalizó en 2014]
3                         [Después de obtener la titulación que finalizó en 2014]


In [394]:
re_otros_estudios('EST_B30_2')

[ 1.  3. nan]


## 'EST_31_2'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Dónde cursó** el segundo ciclo formativo de grado medio mencionado.

In [395]:
df['EST_B31_2'].unique() # No informados como ' ' 

array([' ', '1'], dtype=object)

In [396]:
traduce('EST_B31_2')

                         aux_traduce
                              unique
EST_B31_2                           
                      [No aplicable]
1          [Completamente en España]


In [397]:
re_otros_estudios('EST_B31_2')

[ 1. nan]


In [398]:
actualiza_extr_proxy('EST_B31_2')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B31_2                                                                               
1.0        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST_B29_FA3'  (redefinida como 'fam_gmedio3')   
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Familia de estudios** del tercer ciclo formativo de grado medio mencionado.

In [399]:
df['EST_B29_FA3'].unique() # No informados como '  '

array(['  ', '23'], dtype=object)

In [400]:
no_respuesta('EST_B29_FA3',blank='  ',nsnc='99')
traduce('EST_B29_FA3')

[23. nan]
                                               aux_traduce
                                                    unique
EST_B29_FA3                                               
23.0         [Servicios socioculturales y a la comunidad ]


In [401]:
re_otros_estudios('EST_B29_FA3','fam_gmedio3',3,blank='  ')

                                           fam_gmedio3_
                                                 unique
n_gmedio                                               
0                                                 [nan]
1                                                 [nan]
2                                                 [nan]
3         [Servicios socioculturales y a la comunidad ]
                                           fam_gmedio3_
                                                 unique
n_gmedio                                               
0                                            [No tiene]
1                                            [No tiene]
2                                            [No tiene]
3         [Servicios socioculturales y a la comunidad ]


In [402]:
D_dict['fam_gmedio3']=D_dict['fam_gmedio1']
traduce('fam_gmedio3')

                                               aux_traduce
                                                    unique
fam_gmedio3                                               
0.0                                             [No tiene]
23.0         [Servicios socioculturales y a la comunidad ]


## 'EST_B30_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-) 

**Momento en que terminó** el tercer ciclo formativo de grado medio mencionado

In [403]:
df['EST_B30_3'].unique() # No informados como ' ' 

array([' ', '1'], dtype=object)

In [404]:
# Ajuste en diccionario
D_dict['EST_B30_3']=diccionario('T2ESTMN')
traduce('EST_B30_3')

                                                                      aux_traduce
                                                                           unique
EST_B30_3                                                                        
                                                                   [No aplicable]
1          [Antes de comenzar los estudios de la titulación que finalizó en 2014]


In [405]:
re_otros_estudios('EST_B30_3')

[ 1. nan]


## 'EST_B31_3'  
(Solo para quienes declaran haber terminado ciclos formativos de grado medio -respuesta 1 en variable 'EST_B11_6' (o 'g_medio')-)  

**Dónde cursó** el tercer ciclo formativo de grado medio mencionado.

In [406]:
df['EST_B31_3'].unique() # No informados como ' ' 

array([' ', '1'], dtype=object)

In [407]:
traduce('EST_B31_3')

                         aux_traduce
                              unique
EST_B31_3                           
                      [No aplicable]
1          [Completamente en España]


In [408]:
re_otros_estudios('EST_B31_3')

[ 1. nan]


In [409]:
actualiza_extr_proxy('EST_B31_3')

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]
                                                                             extr_proxy_
                                                                                  unique
EST_B31_3                                                                               
1.0                                                         [No estudió fuera de España]
NaN        [Al menos parte de los estudios en el extranjero, No estudió fuera de España]


## 'EST2_NC'   

**Estudios en curso**.

In [410]:
df['EST2_NC'].unique()

array([ 9,  5,  2,  4,  1,  7,  6, 99,  3,  8])

In [411]:
traduce('EST2_NC',check=False)

In [412]:
leyenda('EST2_NC')

1 	 Grado universitario de 180 créditos (3 años)
2 	 Grado universitario de 240 créditos (4 años)
3 	 Grado universitario de más de 240 créditos (5 años o más)
9 	 No estoy cursando ninguno de los estudios anteriores
4 	 Máster universitario
5 	 Doctorado universitario
6 	 Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)
7 	 Ciclos Formativos de grado superior (formación profesional, artes plásticas y diseño) o Enseñanzas Deportivas de Grado Superior
8 	 Ciclos Formativos de grado medio (formación profesional, artes plásticas y diseño) o Enseñanzas Deportivas de Grado Medio
99 	 NS/NC


In [413]:
no_respuesta('EST2_NC',nsnc=99, tipo='int')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. nan]


## 'IDI_MT1'   (redefinida como 'idioma_m1')

**Primer idioma materno**.

In [414]:
df['IDI_MT1'].unique() # Sin no informados

array([ 1, 14,  2,  3,  8,  7,  4,  5, 13,  6, 15, 10, 12,  9, 11])

In [415]:
traduce('IDI_MT1')

             aux_traduce
                  unique
IDI_MT1                 
1              [Español]
2              [Catalán]
3              [Euskera]
4              [Gallego]
5           [Valenciano]
6               [Inglés]
7              [Francés]
8               [Alemán]
9             [Italiano]
10           [Portugués]
11              [Rumano]
12               [Árabe]
13               [Chino]
14                [Ruso]
15       [Otros idiomas]


In [416]:
redefine('IDI_MT1','idioma_m1')

In [417]:
proporciones('IDI_MT1_')

,IDI_MT1_
Español,0.833841
Catalán,0.082034
Gallego,0.025516
Euskera,0.024645
Valenciano,0.020726
Otros idiomas,0.003919
Inglés,0.002264
Francés,0.001655
Portugués,0.001132
Italiano,0.000958


## 'IDI_MT2'   (redefinida como 'idioma_m2')

**Segundo idioma materno**.

In [418]:
df['IDI_MT2'].unique() # No informados como '  '

array(['  ', '09', '03', '01', '06', '12', '02', '15', '04', '05', '07',
       '08', '14', '10', '11', '13'], dtype=object)

In [419]:
no_respuesta('IDI_MT2',blank='  ')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]


In [420]:
traduce('IDI_MT2')

             aux_traduce
                  unique
IDI_MT2                 
1.0            [Español]
2.0            [Catalán]
3.0            [Euskera]
4.0            [Gallego]
5.0         [Valenciano]
6.0             [Inglés]
7.0            [Francés]
8.0             [Alemán]
9.0           [Italiano]
10.0         [Portugués]
11.0            [Rumano]
12.0             [Árabe]
13.0             [Chino]
14.0              [Ruso]
15.0     [Otros idiomas]


In [421]:
redefine('IDI_MT2','idioma_m2')

## 'IDIOMAS'   (redefinida como 'n_idiomas')

**Número de idiomas** que habla sin contar los maternos.

Nota: En 'n_idiomas' se computan también los maternos declarados y se prescinde de la leyenda.

In [422]:
df['IDIOMAS'].unique() # Sin no informados

array([2, 1, 3, 0, 4, 5, 9])

In [423]:
traduce('IDIOMAS')

        aux_traduce
             unique
IDIOMAS            
0         [Ninguno]
1               [1]
2               [2]
3               [3]
4               [4]
5         [5 o más]
9           [NS/NC]


In [424]:
no_respuesta('IDIOMAS',tipo='int')

[ 0.  1.  2.  3.  4.  5. nan]


In [425]:
D_describe['n_idiomas']='Número de idiomas que habla incluidos los maternos'
# Se computan los idiomas maternos (al menos uno para todos los encuestados) y los no maternos declarados: 
df['aux']=1 # Sin valores no informados en 'idioma_m1'
df['aux'].mask(df['idioma_m2'].notna(),2,inplace=True) # Segundo idioma materno cuando se declara alguno
df['n_idiomas']=df['aux']+df['IDIOMAS'].where(df['IDIOMAS'].notna(),0) # Idiomas no maternos si hay dato
a_lista('n_idiomas')
del df['aux']
df[['IDIOMAS_','n_idiomas']].groupby('IDIOMAS_',dropna=False).agg(['unique']) # Comprobación

,n_idiomas
,unique
IDIOMAS_,
1,"[2.0, 3.0]"
2,"[3.0, 4.0]"
3,"[5.0, 4.0]"
4,"[5.0, 6.0]"
5 o más,"[6.0, 7.0]"
Ninguno,"[1.0, 2.0]"
NaN,"[1.0, 2.0]"


In [426]:
proporciones('n_idiomas').sort_values(by='n_idiomas')

,n_idiomas
7.0,0.002874
6.0,0.011234
1.0,0.032570
5.0,0.043281
4.0,0.171384
2.0,0.347122
3.0,0.391535


## 'IDIOMA1'  (redefinida como 'idioma1')

(Solo para quienes declaran conocer algún idioma distinto de los maternos -respuesta distinta de 0 en 'IDIOMAS')

**Primer idioma distinto de su lengua materna** que mejor conoce.

In [427]:
df['IDIOMA1'].sort_values().unique() # No informados como '  ' o '99'

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '12', '13', '14', '15', '99'], dtype=object)

In [428]:
def re_idiomas(col,nuevo=None,si_no=False,redef=True):
    """Procesa las variables referidas a los idiomas no maternos y el nivel de conocimiento de estos. 
    """
    reset(col)
    if '  ' in df[col].unique() or '99' in df[col].unique():
        no_respuesta(col,blank='  ',nsnc='99')
    else: 
        no_respuesta(col)
    traduce(col)
    if redef==True:
        if nuevo==None:
            nuevo=col.lower()
        redefine(col,nuevo,si_no=si_no)
    if str(col)[:-1]=='IDIOMA':
        print(df[[col,'IDIOMAS']].groupby('IDIOMAS').agg(['unique'])) # Inspección de exclusiones
    return

In [429]:
re_idiomas('IDIOMA1')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 12. 13. 14. 15. nan]
             aux_traduce
                  unique
IDIOMA1                 
1.0            [Español]
2.0            [Catalán]
3.0            [Euskera]
4.0            [Gallego]
5.0         [Valenciano]
6.0             [Inglés]
7.0            [Francés]
8.0             [Alemán]
9.0           [Italiano]
10.0         [Portugués]
12.0             [Árabe]
13.0             [Chino]
14.0              [Ruso]
15.0     [Otros idiomas]
                                                                            IDIOMA1
                                                                             unique
IDIOMAS                                                                            
0.0                                                                           [nan]
1.0            [6.0, nan, 7.0, 3.0, 10.0, 2.0, 1.0, 9.0, 15.0, 4.0, 5.0, 8.0, 13.0]
2.0      [6.0, 9.0, 7.0, 1.0, nan, 5.0, 3.0, 2.0, 10.0, 4.0, 8.0, 15.0, 14.0, 13.0]
3.0      

## 'NIV_ID1'  (redefinida como 'idioma1_nivel')

(Solo para quienes declaran conocer algún idioma distinto de los maternos -respuesta distinta de 0 en 'IDIOMAS')

**Nivel de conocimiento del primer idioma no materno** que mejor conoce.

In [430]:
df['NIV_ID1'].unique() # No informados como ' ' o '9'

array(['1', '9', '2', ' ', '3'], dtype=object)

In [431]:
re_idiomas('NIV_ID1','idioma1_nivel')

[ 1.  2.  3. nan]
                                                                                              aux_traduce
                                                                                                   unique
NIV_ID1                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


In [432]:
proporciones('idioma1_nivel_')

,idioma1_nivel_
Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad),0.529043
Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos),0.350605
NaN,0.063050
Nivel bajo (Entiende y usa las expresiones más comunes ),0.057302


## 'ACREIDI1'  

(Solo para quienes declaran conocer algún idioma distinto de los maternos -respuesta distinta de 0 en 'IDIOMAS')

¿Tiene **acreditación oficial** del primer idioma no materno?

In [433]:
df['ACREIDI1'].unique() # No informados como ' ' o '9'

array(['1', '2', ' ', '9'], dtype=object)

In [434]:
re_idiomas('ACREIDI1',redef=False)

[ 1.  2. nan]
         aux_traduce
              unique
ACREIDI1            
1.0             [Sí]
2.0             [No]


## 'TIPOACREIDI1'  (redefinida como 'idioma1_acred')

(Solo para quienes declaran conocer algún idioma distintos de los maternos -respuesta distinta de 0 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del primer idioma no materno.

In [435]:
df['TIPOACREIDI1'].sort_values().unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '4', '5', '6', '9'], dtype=object)

In [436]:
re_idiomas('TIPOACREIDI1','idioma1_acred')

[ 1.  2.  3.  4.  5.  6. nan]
                                       aux_traduce
                                            unique
TIPOACREIDI1                                      
1.0           [A1 (DELF A1, Fit in Deutsch 1, ….)]
2.0            [A2 (DELF A2, Fit in Deutsch 2, …)]
3.0                     [B1 (PET, DELF B1, ZD, …)]
4.0                   [B2 (FCE, DELF B2, ZDfB, …)]
5.0                    [C1 (CAE, DALF C1, PWD, …)]
6.0                    [C2 (CPE, DALF C2, ZOP, …)]


In [437]:
proporciones('idioma1_acred_',dropna=True)

,idioma1_acred_
"B2 (FCE, DELF B2, ZDfB, …)",0.368351
"C1 (CAE, DALF C1, PWD, …)",0.295542
"B1 (PET, DELF B1, ZD, …)",0.224814
"C2 (CPE, DALF C2, ZOP, …)",0.080089
"A2 (DELF A2, Fit in Deutsch 2, …)",0.021991
"A1 (DELF A1, Fit in Deutsch 1, ….)",0.009212


## 'IDIOMA2'  (redefinida como 'idioma2')

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

**Segundo idioma distinto de su lengua materna** que mejor conoce.

In [438]:
df['IDIOMA2'].sort_values().unique() # No informados como '  ' o '99'

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '99'], dtype=object)

In [439]:
re_idiomas('IDIOMA2')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]
             aux_traduce
                  unique
IDIOMA2                 
1.0            [Español]
2.0            [Catalán]
3.0            [Euskera]
4.0            [Gallego]
5.0         [Valenciano]
6.0             [Inglés]
7.0            [Francés]
8.0             [Alemán]
9.0           [Italiano]
10.0         [Portugués]
11.0            [Rumano]
12.0             [Árabe]
13.0             [Chino]
14.0              [Ruso]
15.0     [Otros idiomas]
                                                                                        IDIOMA2
                                                                                         unique
IDIOMAS                                                                                        
0.0                                                                                       [nan]
1.0                                                                                       [nan]
2.0 

## 'NIV_ID2'  (redefinida como 'idioma2_nivel')

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

**Nivel de conocimiento del segundo idioma no materno** que mejor conoce.

In [440]:
df['NIV_ID2'].unique() # No informados como ' ' o '9'

array(['2', ' ', '3', '9', '1'], dtype=object)

In [441]:
re_idiomas('NIV_ID2','idioma2_nivel')

[ 1.  2.  3. nan]
                                                                                              aux_traduce
                                                                                                   unique
NIV_ID2                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'ACREIDI2'  

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

¿Tiene **acreditación oficial** del segundo idioma no materno?

In [442]:
df['ACREIDI2'].unique() # No informados como ' ' o '9'

array(['2', ' ', '1', '9'], dtype=object)

In [443]:
re_idiomas('ACREIDI2',redef=False)

[ 1.  2. nan]
         aux_traduce
              unique
ACREIDI2            
1.0             [Sí]
2.0             [No]


## 'TIPOACREIDI2'  (redefinida como 'idioma2_acred')

(Solo para quienes declaran conocer dos idiomas distintos de los maternos -respuesta mayor que 1 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del segundo idioma no materno.

In [444]:
df['TIPOACREIDI2'].unique() # No informados como ' ' o '9'

array([' ', '4', '3', '2', '5', '6', '9', '1'], dtype=object)

In [445]:
re_idiomas('TIPOACREIDI2','idioma2_acred')

[ 1.  2.  3.  4.  5.  6. nan]
                                       aux_traduce
                                            unique
TIPOACREIDI2                                      
1.0           [A1 (DELF A1, Fit in Deutsch 1, ….)]
2.0            [A2 (DELF A2, Fit in Deutsch 2, …)]
3.0                     [B1 (PET, DELF B1, ZD, …)]
4.0                   [B2 (FCE, DELF B2, ZDfB, …)]
5.0                    [C1 (CAE, DALF C1, PWD, …)]
6.0                    [C2 (CPE, DALF C2, ZOP, …)]


### 'IDIOMA3'  (redefinida como 'idioma3')

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

**Tercer idioma distinto de su lengua materna** que mejor conoce.

In [446]:
df['IDIOMA3'].sort_values().unique() # No informados como '  ' o '99'

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '99'], dtype=object)

In [447]:
re_idiomas('IDIOMA3')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]
             aux_traduce
                  unique
IDIOMA3                 
1.0            [Español]
2.0            [Catalán]
3.0            [Euskera]
4.0            [Gallego]
5.0         [Valenciano]
6.0             [Inglés]
7.0            [Francés]
8.0             [Alemán]
9.0           [Italiano]
10.0         [Portugués]
11.0            [Rumano]
12.0             [Árabe]
13.0             [Chino]
14.0              [Ruso]
15.0     [Otros idiomas]
                                                                                        IDIOMA3
                                                                                         unique
IDIOMAS                                                                                        
0.0                                                                                       [nan]
1.0                                                                                       [nan]
2.0 

## 'NIV_ID3'  (redefinida como 'idioma3_nivel')

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

**Nivel de conocimiento del tercer idioma no materno** que mejor conoce.

In [448]:
df['NIV_ID3'].unique() # No informados como ' ' o '9'

array([' ', '3', '2', '1', '9'], dtype=object)

In [449]:
re_idiomas('NIV_ID3','idioma3_nivel')

[ 1.  2.  3. nan]
                                                                                              aux_traduce
                                                                                                   unique
NIV_ID3                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'ACREIDI3'  

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

¿Tiene **acreditación oficial** del tercer idioma no materno?

In [450]:
df['ACREIDI3'].unique() # No informados como ' ' o '9'

array([' ', '2', '9', '1'], dtype=object)

In [451]:
re_idiomas('ACREIDI3',redef=False)

[ 1.  2. nan]
         aux_traduce
              unique
ACREIDI3            
1.0             [Sí]
2.0             [No]


## 'TIPOACREIDI3'  (redefinida como 'idioma3_acred')

(Solo para quienes declaran conocer tres idiomas distintos de los maternos -respuesta mayor que 2 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del tercer idioma no materno.

In [452]:
df['TIPOACREIDI3'].unique() # No informados como ' ' o '9'

array([' ', '4', '6', '3', '9', '2', '1', '5'], dtype=object)

In [453]:
re_idiomas('TIPOACREIDI3','idioma3_acred')

[ 1.  2.  3.  4.  5.  6. nan]
                                       aux_traduce
                                            unique
TIPOACREIDI3                                      
1.0           [A1 (DELF A1, Fit in Deutsch 1, ….)]
2.0            [A2 (DELF A2, Fit in Deutsch 2, …)]
3.0                     [B1 (PET, DELF B1, ZD, …)]
4.0                   [B2 (FCE, DELF B2, ZDfB, …)]
5.0                    [C1 (CAE, DALF C1, PWD, …)]
6.0                    [C2 (CPE, DALF C2, ZOP, …)]


## 'IDIOMA4'  (redefinida como 'idioma4')

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

**Cuarto idioma distinto de su lengua materna** que mejor conoce.

In [454]:
df['IDIOMA4'].sort_values().unique() # No informados como '  ' o '99'

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '99'], dtype=object)

In [455]:
re_idiomas('IDIOMA4')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. nan]
             aux_traduce
                  unique
IDIOMA4                 
1.0            [Español]
2.0            [Catalán]
3.0            [Euskera]
4.0            [Gallego]
5.0         [Valenciano]
6.0             [Inglés]
7.0            [Francés]
8.0             [Alemán]
9.0           [Italiano]
10.0         [Portugués]
11.0            [Rumano]
12.0             [Árabe]
13.0             [Chino]
14.0              [Ruso]
15.0     [Otros idiomas]
                                                                              IDIOMA4
                                                                               unique
IDIOMAS                                                                              
0.0                                                                             [nan]
1.0                                                                             [nan]
2.0                                                   

## 'NIV_ID4'  (redefinida como 'idioma4_nivel')

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

**Nivel de conocimiento del cuarto idioma no materno** que mejor conoce.

In [456]:
df['NIV_ID4'].unique() # No informados como ' ' o '9'

array([' ', '3', '2', '1'], dtype=object)

In [457]:
re_idiomas('NIV_ID4','idioma4_nivel')

[ 1.  2.  3. nan]
                                                                                              aux_traduce
                                                                                                   unique
NIV_ID4                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'ACREIDI4'  

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

¿Tiene **acreditación oficial** del cuarto idioma no materno?

In [458]:
df['ACREIDI4'].unique() # No informados como ' ' o '9'

array([' ', '9', '2', '1'], dtype=object)

In [459]:
re_idiomas('ACREIDI4',redef=False)

[ 1.  2. nan]
         aux_traduce
              unique
ACREIDI4            
1.0             [Sí]
2.0             [No]


## 'TIPOACREIDI4'  (redefinida como 'idioma4_acred')

(Solo para quienes declaran conocer cuatro idiomas distintos de los maternos -respuesta mayor que 3 en 'IDIOMAS')

Máximo **nivel de acreditación oficial** del cuarto idioma no materno.

In [460]:
df['TIPOACREIDI4'].unique() # No informados como ' ' o '9'

array([' ', '4', '1', '3', '2', '6', '9', '5'], dtype=object)

In [461]:
re_idiomas('TIPOACREIDI4','idioma4_acred')

[ 1.  2.  3.  4.  5.  6. nan]
                                       aux_traduce
                                            unique
TIPOACREIDI4                                      
1.0           [A1 (DELF A1, Fit in Deutsch 1, ….)]
2.0            [A2 (DELF A2, Fit in Deutsch 2, …)]
3.0                     [B1 (PET, DELF B1, ZD, …)]
4.0                   [B2 (FCE, DELF B2, ZDfB, …)]
5.0                    [C1 (CAE, DALF C1, PWD, …)]
6.0                    [C2 (CPE, DALF C2, ZOP, …)]


## 'TITIDI'  

(Solo para quienes declaran tener acreditación oficial de algún idioma distinto de los maternos -respuesta Sí para alguna variable 'ACREIDIi' (i=1,2,...,4)-)

Tiene título de la **Escuela Oficial de Idiomas**. 

In [462]:
df['TITIDI'].unique() # No informados como ' ' o '9'

array(['2', '1', ' ', '9'], dtype=object)

In [463]:
traduce('TITIDI')

           aux_traduce
                unique
TITIDI                
        [No aplicable]
1                 [Sí]
2                 [No]
9              [NS/NC]


## 'TIC' (redefinida como 'ti')

Capacidad para usar el ordenador u otros **dispositivos informáticos**.

In [464]:
df['TIC'].unique() # No informados como '9'

array([2, 3, 1, 9])

In [465]:
traduce('TIC')

                        aux_traduce
                             unique
TIC                                
1        [Usuario de nivel básico ]
2    [Usuario de nivel intermedio ]
3      [Usuario de nivel avanzado ]
9                           [NS/NC]


In [466]:
no_respuesta('TIC',tipo='int')

[ 1.  2.  3. nan]


In [467]:
redefine('TIC','ti')

In [468]:
proporciones('ti_')

,ti_
Usuario de nivel intermedio,0.674301
Usuario de nivel avanzado,0.234172
Usuario de nivel básico,0.090220
NaN,0.001306


## 'EST_B38_1'  

In [469]:
D_describe['EST_B38_1']

'Ha realizado cursos conducentes a Certificado de Profesionalidad Nivel 1'

In [470]:
df['EST_B38_1'].unique() # Sin no informados

array([2, 1])

In [471]:
traduce('EST_B38_1')

          aux_traduce
               unique
EST_B38_1            
1                [Sí]
2                [No]


In [472]:
proporciones('EST_B38_1_')

,EST_B38_1_
No,0.977532
Sí,0.022468


In [473]:
reset('EST_B38_1_')

## 'EST_B39_1'  

In [474]:
D_describe['EST_B39_1']

'Le ha sido útil para el trabajo los cursos conducentes a Certificado de Profesionalidad Nivel 1'

In [475]:
df['EST_B39_1'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [476]:
traduce('EST_B39_1')

              aux_traduce
                   unique
EST_B39_1                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [477]:
reset('EST_B39_1_')

## 'EST_B38_2'  

In [478]:
D_describe['EST_B38_2']

'Ha realizado cursos conducentes a Certificado de Profesionalidad Nivel 2'

In [479]:
df['EST_B38_2'].unique() # Sin no informados

array([2, 1])

In [480]:
traduce('EST_B38_2')

          aux_traduce
               unique
EST_B38_2            
1                [Sí]
2                [No]


In [481]:
proporciones('EST_B38_2_')

,EST_B38_2_
No,0.983018
Sí,0.016982


In [482]:
reset('EST_B38_2_')

## 'EST_B39_2'  

In [483]:
D_describe['EST_B39_2']

'Le ha sido útil para el trabajo los cursos conducentes a Certificado de Profesionalidad Nivel 2'

In [484]:
df['EST_B39_2'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [485]:
traduce('EST_B39_2')

              aux_traduce
                   unique
EST_B39_2                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [486]:
reset('EST_B39_2_')

### 'EST_B38_3'  

In [487]:
D_describe['EST_B38_3']

'Ha realizado cursos conducentes a Certificado de Profesionalidad Nivel 3'

In [488]:
df['EST_B38_3'].unique() # Sin no informados

array([2, 1])

In [489]:
traduce('EST_B38_3')

          aux_traduce
               unique
EST_B38_3            
1                [Sí]
2                [No]


In [490]:
proporciones('EST_B38_3_')

,EST_B38_3_
No,0.967866
Sí,0.032134


In [491]:
reset('EST_B38_3_')

## 'EST_B39_3'  

In [492]:
D_describe['EST_B39_3']

'Le ha sido útil para el trabajo los cursos conducentes a Certificado de Profesionalidad Nivel 3'

In [493]:
df['EST_B39_3'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [494]:
traduce('EST_B39_3')

              aux_traduce
                   unique
EST_B39_3                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [495]:
proporciones('EST_B39_3_')

,EST_B39_3_
No aplicable,0.967866
Sí,0.019159
No,0.012453
NS/NC,0.000523


In [496]:
reset('EST_B39_3_')

## 'EST_B38_4'  

In [497]:
D_describe['EST_B38_4']

'Ha realizado otros cursos de Formación Ocupacional de los Servicios Públicos de Empleo'

In [498]:
df['EST_B38_4'].unique() # Sin no informados

array([2, 1])

In [499]:
traduce('EST_B38_4')

          aux_traduce
               unique
EST_B38_4            
1                [Sí]
2                [No]


In [500]:
proporciones('EST_B38_4_')

,EST_B38_4_
No,0.883741
Sí,0.116259


In [501]:
reset('EST_B38_4_')

## 'EST_B39_4'  

In [502]:
D_describe['EST_B39_4']

'Le ha sido útil para el trabajo los otros cursos de Formación Ocupacional de los Servicios Públicos de Empleo'

In [503]:
df['EST_B39_4'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [504]:
traduce('EST_B39_4')

              aux_traduce
                   unique
EST_B39_4                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [505]:
proporciones('EST_B39_4_')

,EST_B39_4_
No aplicable,0.883741
No,0.061395
Sí,0.054167
NS/NC,0.000697


In [506]:
reset('EST_B39_4_')

## 'EST_B38_5'  

In [507]:
D_describe['EST_B38_5']

'Ha realizado cursos de idiomas'

In [508]:
df['EST_B38_5'].unique() # Sin no informados

array([2, 1])

In [509]:
traduce('EST_B38_5')

          aux_traduce
               unique
EST_B38_5            
1                [Sí]
2                [No]


In [510]:
proporciones('EST_B38_5_')

,EST_B38_5_
Sí,0.510842
No,0.489158


In [511]:
reset('EST_B38_5_')

## 'EST_B39_5'  

In [512]:
D_describe['EST_B39_5']

'Le ha sido útil para el trabajo los cursos de idiomas'

In [513]:
df['EST_B39_5'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [514]:
traduce('EST_B39_5')

              aux_traduce
                   unique
EST_B39_5                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [515]:
proporciones('EST_B39_5_')

,EST_B39_5_
No aplicable,0.489158
Sí,0.309066
No,0.200732
NS/NC,0.001045


In [516]:
reset('EST_B39_5_')

## 'EST_B38_6'  

In [517]:
D_describe['EST_B38_6']

'Ha realizado cursos de informática'

In [518]:
df['EST_B38_6'].unique() # Sin no informados

array([2, 1])

In [519]:
traduce('EST_B38_6')

          aux_traduce
               unique
EST_B38_6            
1                [Sí]
2                [No]


In [520]:
proporciones('EST_B38_6_')

,EST_B38_6_
No,0.751546
Sí,0.248454


In [521]:
reset('EST_B38_6_')

## 'EST_B39_6'  

In [522]:
D_describe['EST_B39_6']

'Le ha sido útil para el trabajo los cursos de informática'

In [523]:
df['EST_B39_6'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [524]:
traduce('EST_B39_6')

              aux_traduce
                   unique
EST_B39_6                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [525]:
proporciones('EST_B39_6_')

,EST_B39_6_
No aplicable,0.751546
Sí,0.151615
No,0.095881
NS/NC,0.000958


In [526]:
reset('EST_B39_6_')

## 'EST_B38_7'  

In [527]:
D_describe['EST_B38_7']

'Ha realizado otros estudios no reglados'

In [528]:
df['EST_B38_7'].unique() # Sin no informados

array([2, 1])

In [529]:
traduce('EST_B38_7')

          aux_traduce
               unique
EST_B38_7            
1                [Sí]
2                [No]


In [530]:
proporciones('EST_B38_7_')

,EST_B38_7_
No,0.548202
Sí,0.451798


In [531]:
reset('EST_B38_7_')

## 'EST_B39_7'  

In [532]:
D_describe['EST_B39_7']

'Le ha sido útil para el trabajo los otros estudios no reglados'

In [533]:
df['EST_B39_7'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [534]:
traduce('EST_B39_7')

              aux_traduce
                   unique
EST_B39_7                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [535]:
proporciones('EST_B39_7_')

,EST_B39_7_
No aplicable,0.548202
Sí,0.275799
No,0.163546
NS/NC,0.012453


In [536]:
reset('EST_B39_7_')

# BLOQUE C. MOVILIDAD

Se omite. 

# BLOQUE D. SITUACIÓN ACTUAL DEL TITULADO DE MÁSTER

## 'TRBPRN1'  (redefinida como 'sit_lab')

**Situación laboral** actual.

In [537]:
df['TRBPRN1'].unique() # Sin valores no informados

array([1, 2, 3])

In [538]:
traduce('TRBPRN1')

            aux_traduce
                 unique
TRBPRN1                
1          [Trabajando]
2        [En desempleo]
3            [Inactivo]


In [539]:
no_respuesta('TRBPRN1')

[1. 2. 3.]


In [540]:
redefine('TRBPRN1','sit_lab')

In [541]:
proporciones('sit_lab_')

,sit_lab_
Trabajando,0.878168
En desempleo,0.068188
Inactivo,0.053645


## 'TRBPRN2'  (redefinida como 'sit_lab_detalle')

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

Tipo de **situación de inactividad**.  

Nota: 'sit_lab_detalle se define para todas las situaciones laborales. Se añadirá el detalle disponible para ocupados y para desempleados.

In [542]:
df['TRBPRN2'].unique() # No informados como ' ' o '9'

array([' ', '1', '5', '9', '4', '2', '3'], dtype=object)

In [543]:
df['TRBPRN2'].value_counts()

     10867
1      403
5       80
4       69
2       39
3       16
9        9
Name: TRBPRN2, dtype: int64

In [544]:
no_respuesta('TRBPRN2')

[ 1.  2.  3.  4.  5. nan]


In [545]:
traduce('TRBPRN2')

                                           aux_traduce
                                                unique
TRBPRN2                                               
1.0                            [(Inactivo) Estudiante]
2.0                              [(Inactivo) Jubilado]
3.0            [(Inactivo) Incapacitado para trabajar]
4.0      [(Inactivo) Dedicado a las labores del hogar]
5.0                        [(Inactivo) Otra situación]


In [546]:
df['sit_lab_detalle_']=df['TRBPRN2_']

In [547]:
df[['sit_lab_','sit_lab_detalle_']].groupby('sit_lab_').agg(['unique']) # Comprobación: 

,sit_lab_detalle_
,unique
sit_lab_,
En desempleo,[nan]
Inactivo,"[(Inactivo) Estudiante, (Inactivo) Otra situación, nan, (Inactivo) Dedicado a las labores del hogar, (Inactivo) Jubilado, (Inactivo) Incapacitado para trabajar]"
Trabajando,[nan]


In [548]:
df['sit_lab_detalle_'][df['sit_lab']==3].value_counts(dropna=False)
# Algunos inactivos no informados en 'sit_lab_detalle_'. Se les imputa '(Inactivo) Otra situación'

(Inactivo) Estudiante                          403
(Inactivo) Otra situación                       80
(Inactivo) Dedicado a las labores del hogar     69
(Inactivo) Jubilado                             39
(Inactivo) Incapacitado para trabajar           16
NaN                                              9
Name: sit_lab_detalle_, dtype: int64

In [549]:
# Se imputa '(Inactivo) Otra situación' a NS/NC:
df['sit_lab_detalle_'].mask((df['TRBPRN2'].isna()) & (df['sit_lab']==3),'(Inactivo) Otra situación',inplace=True)

In [550]:
df['sit_lab_detalle_'][df['sit_lab']==3].value_counts(dropna=False) # Comprobación

(Inactivo) Estudiante                          403
(Inactivo) Otra situación                       89
(Inactivo) Dedicado a las labores del hogar     69
(Inactivo) Jubilado                             39
(Inactivo) Incapacitado para trabajar           16
Name: sit_lab_detalle_, dtype: int64

## 'TRINAC_D4'  (redefinida como 'busqueda_tr')

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

¿Ha **buscado trabajo** desde que se tituló?  

Nota: 'busqueda_tr' se redefine incluyendo esta información para todas las situaciones laborales cuando esté disponible. 

In [551]:
df['TRINAC_D4'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [552]:
traduce('TRINAC_D4')

              aux_traduce
                   unique
TRINAC_D4                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [553]:
df[['sit_lab_','TRINAC_D4']].groupby('sit_lab_',dropna=False).agg(['unique']) # Inspección

,TRINAC_D4
,unique
sit_lab_,
En desempleo,[ ]
Inactivo,"[1, 2, 9]"
Trabajando,[ ]


In [554]:
no_respuesta('TRINAC_D4')

[ 1.  2. nan]


In [555]:
redefine('TRINAC_D4','busqueda_tr',si_no=True)

            aux_traduce
                 unique
busqueda_tr            
0.0                [No]
1.0                [Sí]


In [556]:
proporciones('busqueda_tr_',dropna=True)

,busqueda_tr_
Sí,0.825733
No,0.174267


## 'TRINAC_D5'  

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

**Momento en el que empezó a buscar trabajo**.

In [557]:
df['TRINAC_D5'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [558]:
traduce('TRINAC_D5')

                                                                      aux_traduce
                                                                           unique
TRINAC_D5                                                                        
                                                                   [No aplicable]
1            [Antes de acabar los estudios de la titulación que finalizó en 2014]
2          [Después de acabar los estudios de la titulación que finalizó en 2014]
9                                                                         [NS/NC]


In [559]:
no_respuesta('TRINAC_D5')

[ 1.  2. nan]


## 'TRINAC_D6'  

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

**Tiempo que ha estado buscando trabajo**.

In [560]:
df['TRINAC_D6'].unique() # No informados como ' ' o '9'

array([' ', '6', '4', '1', '5', '2', '3', '9'], dtype=object)

In [561]:
traduce('TRINAC_D6')

                             aux_traduce
                                  unique
TRINAC_D6                               
                          [No aplicable]
1                  [Menos de tres meses]
2                       [De 3 a 6 meses]
3                   [De 6 meses a 1 año]
4             [De 1 año a año y medio  ]
5          [De 1 año y medio a  2 años ]
6                         [2 ó más años]
9                                [NS/NC]


In [562]:
no_respuesta('TRINAC_D6')

[ 1.  2.  3.  4.  5.  6. nan]


## 'TRPARA_D5'  (complementa la variable  'busqueda_tr)

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

**Momento en el que empezó a buscar trabajo**.

Nota: Se usa esta información para completar la recogida en la variable 'busqueda_tr' (definida inicialmente para graduados en situación de inactividad). 

In [563]:
df['TRPARA_D5'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [564]:
traduce('TRPARA_D5')

                                                                      aux_traduce
                                                                           unique
TRPARA_D5                                                                        
                                                                   [No aplicable]
1            [Antes de acabar los estudios de la titulación que finalizó en 2014]
2          [Después de acabar los estudios de la titulación que finalizó en 2014]
9                                                                         [NS/NC]


In [565]:
df[['sit_lab_','TRPARA_D5_']].groupby('sit_lab_',dropna=False).agg(['unique']) # Inspección

,TRPARA_D5_
,unique
sit_lab_,
En desempleo,"[Después de acabar los estudios de la titulación que finalizó en 2014, Antes de acabar los estudios de la titulación que finalizó en 2014, NS/NC]"
Inactivo,[No aplicable]
Trabajando,[No aplicable]


In [566]:
no_respuesta('TRPARA_D5')

[ 1.  2. nan]


In [567]:
# Se añaden a 'busqueda_tr' los desempleados que declaran expresamente haber buscado trabajo: 
# (Se procede así en contradicción con la definición formal de desempleado, que requiere búsqueda de empleo).
df['busqueda_tr'].mask((df['TRPARA_D5']==1)|(df['TRPARA_D5']==2),1,inplace=True)
traduce('busqueda_tr',check=False)
df[['busqueda_tr_','TRPARA_D5_']].groupby('TRPARA_D5_',dropna=False).agg(['unique']) # Comprobación

,busqueda_tr_
,unique
TRPARA_D5_,
Antes de acabar los estudios de la titulación que finalizó en 2014,[Sí]
Después de acabar los estudios de la titulación que finalizó en 2014,[Sí]
NaN,"[nan, Sí, No]"


## 'TRPARA_D6'  

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

**Tiempo que ha estado buscando trabajo**.

In [568]:
df['TRPARA_D6'].unique() # No informados como ' ' o '9'

array([' ', '2', '3', '4', '1', '6', '5', '9'], dtype=object)

In [569]:
traduce('TRPARA_D6')

                             aux_traduce
                                  unique
TRPARA_D6                               
                          [No aplicable]
1                  [Menos de tres meses]
2                       [De 3 a 6 meses]
3                   [De 6 meses a 1 año]
4             [De 1 año a año y medio  ]
5          [De 1 año y medio a  2 años ]
6                         [2 ó más años]
9                                [NS/NC]


In [570]:
no_respuesta('TRPARA_D6')

[ 1.  2.  3.  4.  5.  6. nan]


## 'TRPARA_D7_1'  (redefinida como 'btr_internet')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **ofertas de trabajo en el periódico, internet**, etc.?

In [571]:
df['TRPARA_D7_1'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [572]:
def btr(col,nuevo):
    """Procesa columnas referidas a métodos de búsqueda de empleo utlizados por los desempleados.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo,si_no=True)
    D_describe[nuevo]=D_describe[col].split(':',1)[1].strip()
    return

In [573]:
btr('TRPARA_D7_1','btr_internet')

                aux_traduce
                     unique
TRPARA_D7_1                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]


## 'TRPARA_D7_2'  (redefinida como 'btr_spe')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **servicios públicos de empleo**?

In [574]:
df['TRPARA_D7_2'].unique() # Missing values as ' ' or '9'

array([' ', '2', '1', '9'], dtype=object)

In [575]:
btr('TRPARA_D7_2','btr_spe')

                aux_traduce
                     unique
TRPARA_D7_2                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
btr_spe            
0.0            [No]
1.0            [Sí]


## 'TRPARA_D7_3'  (redefinida como 'btr_univ')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de los **servicios de empleo de la universidad** (bolsa de trabajo, asociaciones de exalumnos, etc.)?

In [576]:
df['TRPARA_D7_3'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [577]:
btr('TRPARA_D7_3','btr_univ')

                aux_traduce
                     unique
TRPARA_D7_3                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
btr_univ            
0.0             [No]
1.0             [Sí]


## 'TRPARA_D7_4'  (redefinida como 'btr_ett')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **empresas de trabajo temporal**?

In [578]:
df['TRPARA_D7_4'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [579]:
btr('TRPARA_D7_4','btr_ett')

                aux_traduce
                     unique
TRPARA_D7_4                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]


## 'TRPARA_D7_5'  (redefinida como 'btr_bolsa')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de una **bolsa de empleo**?

In [580]:
df['TRPARA_D7_5'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [581]:
btr('TRPARA_D7_5','btr_bolsa')

                aux_traduce
                     unique
TRPARA_D7_5                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
          aux_traduce
               unique
btr_bolsa            
0.0              [No]
1.0              [Sí]


## 'TRPARA_D7_6'  (redefinida como 'btr_contactos')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo buscando ayuda entre profesionales conocidos del sector o **contactos personales** (familia, amigos)?

In [582]:
df['TRPARA_D7_6'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [583]:
btr('TRPARA_D7_6','btr_contactos')

                aux_traduce
                     unique
TRPARA_D7_6                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]


## 'TRPARA_D7_7'  (redefinida como 'btr_empresas')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo **contactando directamente con las empresas**?

In [584]:
df['TRPARA_D7_7'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [585]:
btr('TRPARA_D7_7','btr_empresas')

                aux_traduce
                     unique
TRPARA_D7_7                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
             aux_traduce
                  unique
btr_empresas            
0.0                 [No]
1.0                 [Sí]


## 'TRPARA_D7_8'  (redefinida como 'btr_opo')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Está preparando o preparó unas **oposiciones**?

In [586]:
df['TRPARA_D7_8'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [587]:
btr('TRPARA_D7_8','btr_opo')

                aux_traduce
                     unique
TRPARA_D7_8                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]


## 'TRPARA_D7_9'  (redefinida como 'btr_auto')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado equipamiento para establecer un **negocio propio**?

In [588]:
df['TRPARA_D7_9'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [589]:
btr('TRPARA_D7_9','btr_auto')

                aux_traduce
                     unique
TRPARA_D7_9                
             [No aplicable]
1                      [Sí]
2                      [No]
9                   [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]


## 'TRPARA_D7_10'  (redefinida como 'btr_otro')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo de **otro modo**?

In [590]:
df['TRPARA_D7_10'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [591]:
btr('TRPARA_D7_10','btr_otro')

                 aux_traduce
                      unique
TRPARA_D7_10                
              [No aplicable]
1                       [Sí]
2                       [No]
9                    [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]


## 'TRPARA_D8_1'  (redefinida como 'dbtr_formacion')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **carencias en la formación universitaria recibida** en cuanto a sus dificultades para encontrar trabajo.

In [592]:
df['TRPARA_D8_1'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '3', '9', '4'], dtype=object)

In [593]:
def dbtr(col,nuevo):
    """Procesa columnas referidas a las dificultades en la búsqueda de trabajo para los desempleados.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo)
    D_describe[nuevo]=D_describe[col].split(':',1)[1].strip()
    return

In [594]:
dbtr('TRPARA_D8_1','dbtr_formacion')

                       aux_traduce
                            unique
TRPARA_D8_1                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_2'  (redefinida como 'dbtr_personales')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **actividades personales** que impiden trabajar (seguir estudiando, familia, otras) en cuanto a sus dificultades para encontrar trabajo.

In [595]:
df['TRPARA_D8_2'].unique() # No informados como ' ' o '9'

array([' ', '3', '1', '2', '4', '9'], dtype=object)

In [596]:
dbtr('TRPARA_D8_2','dbtr_personales')

                       aux_traduce
                            unique
TRPARA_D8_2                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_3'  (redefinida como 'dbtr_experiencia')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de experiencia profesional** en cuanto a sus dificultades para encontrar trabajo.

In [597]:
df['TRPARA_D8_3'].unique() # No informados como ' ' o '9'

array([' ', '1', '3', '2', '4', '9'], dtype=object)

In [598]:
dbtr('TRPARA_D8_3','dbtr_experiencia')

                       aux_traduce
                            unique
TRPARA_D8_3                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_4'  (redefinida como 'dbtr_expectativas')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **exigencia de tener un trabajo acorde con sus expectativas** económicas y profesionales en cuanto a sus dificultades para encontrar trabajo.

In [599]:
df['TRPARA_D8_4'].unique() # No informados como ' ' o '9'

array([' ', '4', '2', '1', '3', '9'], dtype=object)

In [600]:
dbtr('TRPARA_D8_4','dbtr_expectativas')

                       aux_traduce
                            unique
TRPARA_D8_4                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_5'  (redefinida como 'dbtr_idiomas')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos de idiomas** en cuanto a sus dificultades para encontrar trabajo.

In [601]:
df['TRPARA_D8_5'].unique() # No informados como ' ' o '9'

array([' ', '1', '3', '2', '4', '9'], dtype=object)

In [602]:
dbtr('TRPARA_D8_5','dbtr_idiomas')

                       aux_traduce
                            unique
TRPARA_D8_5                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRPARA_D8_6'  (redefinida como 'dbtr_ti')

(Solo para quienes declaran **situación de desempleo** -respuesta 2 en 'TRBPRN1' (o 'sit_lab')-)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos informáticos** en cuanto a sus dificultades para encontrar trabajo.

In [603]:
df['TRPARA_D8_6'].unique() # No informados como ' ' o '9'

array([' ', '1', '3', '2', '9', '4'], dtype=object)

In [604]:
dbtr('TRPARA_D8_6','dbtr_ti')

                       aux_traduce
                            unique
TRPARA_D8_6                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]
[ 1.  2.  3.  4. nan]


## 'TRAC_D9'  

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

Actualmente trabaja en **más de un empleo**.

In [605]:
df['TRAC_D9'].unique() # No informados como ' ' o '9'

array(['2', '1', ' ', '9'], dtype=object)

In [606]:
traduce('TRAC_D9')

            aux_traduce
                 unique
TRAC_D9                
         [No aplicable]
1                  [Sí]
2                  [No]
9               [NS/NC]


In [607]:
no_respuesta('TRAC_D9')

[ 1.  2. nan]


## 'LUG_TRAB_PAIS'  (redefinida como 'pais_tr')

(Solo para quienes declaran situación de ocupación -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**País de trabajo** actual.

In [608]:
df['LUG_TRAB_PAIS'].unique() # No informados como '  ' o '99'

array(['01', '  ', '02', '07', '06', '11', '03', '05', '08', '09', '04',
       '10'], dtype=object)

In [609]:
traduce('LUG_TRAB_PAIS')

                        aux_traduce
                             unique
LUG_TRAB_PAIS                      
                     [No aplicable]
01                             [01]
02                             [02]
03                             [03]
04                             [04]
05                             [05]
06                             [06]
07                             [07]
08                             [08]
09                             [09]
10              [Un país de África]
11             [Un país de Oceanía]


In [610]:
no_respuesta('LUG_TRAB_PAIS',blank='  ',nsnc='99')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. nan]


In [611]:
redefine('LUG_TRAB_PAIS','pais_tr')

## 'SIT_PRO' (actualiza la variable 'sit_lab_detalle')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Situación profesional** actual.

In [612]:
df['SIT_PRO'].unique() # No informados como ' ' 

array(['2', '1', '3', ' ', '5', '6', '4'], dtype=object)

In [613]:
traduce('SIT_PRO')

                                                                              aux_traduce
                                                                                   unique
SIT_PRO                                                                                  
                                                                           [No aplicable]
1               [Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario]
2        [Asalariado con trabajo permanente o contrato de trabajo de duración indefinida]
3         [Asalariado con trabajo temporal o contrato de trabajo de duración determinada]
4                                                            [Empresario con asalariados]
5                                 [Trabajador independiente o empresario sin asalariados]
6                                                [Ayuda en la empresa o negocio familiar]


In [614]:
no_respuesta('SIT_PRO')

[ 1.  2.  3.  4.  5.  6. nan]


In [615]:
# Se actualiza la variable 'sit_lab_detalle_' (solo la literal para no reescribir los códigos de la variable ...
# ... discreta)
df['sit_lab_detalle_'].mask(df['sit_lab']==1,'(Ocupado) '+df['SIT_PRO_'],inplace=True)
df['sit_lab_detalle_'].mask(df['sit_lab']==2,'Desempleado',inplace=True)
df[['sit_lab_detalle_','sit_lab_']].groupby('sit_lab_',dropna=False).agg(['unique']) # Comprobación

,sit_lab_detalle_
,unique
sit_lab_,
En desempleo,[Desempleado]
Inactivo,"[(Inactivo) Estudiante, (Inactivo) Otra situación, (Inactivo) Dedicado a las labores del hogar, (Inactivo) Jubilado, (Inactivo) Incapacitado para trabajar]"
Trabajando,"[(Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, (Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario, (Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada, (Ocupado) Trabajador independient..."


In [616]:
# Para construir la variable 'sit_lab_detalle' ('int') a partir de la literal 'sit_lab_detalle_', se emplea ...
# ... el diccionario inverso del de 'sit_lab_detalle' (este último, definido en el notebook Graduado_2019.ipynb)
leyenda('sit_lab_detalle')

1 	 (Inactivo) Estudiante
2 	 (Inactivo) Jubilado
3 	 (Inactivo) Incapacitado para trabajar
9 	 (Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada
4 	 (Inactivo) Dedicado a las labores del hogar
5 	 (Inactivo) Otra situación
6 	 Desempleado
11 	 (Ocupado) Trabajador independiente o empresario sin asalariados
7 	 (Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario
8 	 (Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida
10 	 (Ocupado) Empresario con asalariados
12 	 (Ocupado) Ayuda en la empresa o negocio familiar


In [617]:
# Diccionario inverso auxiliar para sit_lab_detalle: 
D_inv_aux=dict()
for k,v in D_dict['sit_lab_detalle'].items(): 
    if type(k)==int: 
        D_inv_aux[v]=k        

In [618]:
df['sit_lab_detalle']=df['sit_lab_detalle_'].replace(D_inv_aux)
df[['sit_lab_detalle','sit_lab_detalle_','sit_lab_']].groupby('sit_lab_detalle').agg(['unique']) # Comprobación: 

,sit_lab_detalle_,sit_lab_
,unique,unique
sit_lab_detalle,,
1,[(Inactivo) Estudiante],[Inactivo]
2,[(Inactivo) Jubilado],[Inactivo]
3,[(Inactivo) Incapacitado para trabajar],[Inactivo]
4,[(Inactivo) Dedicado a las labores del hogar],[Inactivo]
5,[(Inactivo) Otra situación],[Inactivo]
6,[Desempleado],[En desempleo]
7,"[(Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario]",[Trabajando]
8,[(Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida],[Trabajando]


In [619]:
a_lista('sit_lab_detalle')
v_modelos_L.append('sit_lab_detalle_')

## 'JORNADA'  (redefinida como 'jornada')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

Tipo de **jornada laboral** actual.

In [620]:
df['JORNADA'].unique() # No informados como ' ' 

array(['2', ' ', '1'], dtype=object)

In [621]:
traduce('JORNADA')

                 aux_traduce
                      unique
JORNADA                     
                         [ ]
1         [A tiempo parcial]
2        [A tiempo completo]


In [622]:
no_respuesta('JORNADA')

[ 1.  2. nan]


In [623]:
redefine('JORNADA')

In [624]:
proporciones('jornada_',dropna=True)

,jornada_
A tiempo completo,0.866819
A tiempo parcial,0.133181


## 'TRABOC' (redefinida como 'ocupacion')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Ocupación** que desempeña en el puesto de trabajo actual.

In [625]:
df['TRABOC'].nunique() # No informados como '  ' o '99'

63

In [626]:
no_respuesta('TRABOC',blank='  ',nsnc='99')

[ 0. 11. 12. 13. 14. 15. 21. 22. 23. 24. 25. 26. 27. 28. 29. 31. 32. 33.
 34. 35. 36. 37. 38. 41. 42. 43. 44. 45. 50. 51. 52. 53. 54. 55. 56. 57.
 58. 59. 61. 62. 64. 71. 72. 73. 74. 75. 76. 77. 78. 81. 82. 83. 84. 91.
 92. 93. 94. 95. 96. 97. 98. nan]


In [627]:
df['ocupacion']=df['TRABOC']
df['ocupacion_']=df['ocupacion'].replace(D_ocupaciones)
# df['ocupacion_'].unique() # Comprobación: 

In [628]:
a_lista('ocupacion')
v_modelos_L.append('ocupacion_')

## 'TR_CNAE' (redefinida como 'cnae')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Actividad principal de la empresa** donde trabaja actualmente.   
Nota: se recodifica la variable 'cnae' para establecer valores numéricos.

In [629]:
df['TR_CNAE'].unique() # No informados como '  ' o '99'

array(['CC', 'PP', 'N6', '  ', 'M1', 'TT', 'M2', 'N5', 'JJ', 'AA', 'GG',
       'KK', 'QQ', 'II', 'HH', 'N1', 'OO', 'RR', 'LL', 'FF', 'N4', 'UU',
       'N3', 'DD', 'N2', 'S1', 'BB', '99', 'M3', 'S2', 'S3', 'EE', 'S4'],
      dtype=object)

In [630]:
traduce('TR_CNAE',check=False)

In [631]:
no_respuesta('TR_CNAE',blank='  ',nsnc='99')

['AA' 'BB' 'CC' 'DD' 'EE' 'FF' 'GG' 'HH' 'II' 'JJ' 'KK' 'LL' 'M1' 'M2'
 'M3' 'N1' 'N2' 'N3' 'N4' 'N5' 'N6' 'OO' 'PP' 'QQ' 'RR' 'S1' 'S2' 'S3'
 'S4' 'TT' 'UU' None]


In [632]:
df['cnae']=df['TR_CNAE'].replace(D_cnae_a_num)
traduce('cnae',check=False)

In [633]:
proporciones('cnae_',dropna=True).head(10)

,cnae_
Educación,0.277628
No conocido o no aplicable,0.125229
"Actividades profesionales, científicas y técnicas",0.117913
Actividades sanitarias y de servicios sociales,0.095097
Administración Pública y defensa; Seguridad Social obligatoria,0.066011
Industria manufacturera,0.055473
Comercio al por mayor y al por menor; reparación de vehículos de motor y motocicletas,0.039885
Construcción e ingeniería civil,0.028216
"Información y comunicaciones (actividades de edición, cinematográficas, de vídeo, de sonido, programas de televisión, telecomunicaciones, programación informática y servicios de información)",0.023861
"Actividades artísticas, culturales, recreativas, deportivas y de entrenimiento",0.023252


## 'TR_TAM' (redefinida como 'tamaño_em')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Número de trabajadores** en la empresa en la que trabaja actualmente. 

In [634]:
df['TR_TAM'].unique() # No informados como ' ' o '9'

array(['5', '3', '4', '2', ' ', '1', '0', '9'], dtype=object)

In [635]:
traduce('TR_TAM')

                                                      aux_traduce
                                                           unique
TR_TAM                                                           
                                                  [No aplicable ]
0       [Trabajadores independientes o autónomos sin asalariados]
1                                             [De 1 a 9 personas]
2                                        [Entre 10 y 19 personas]
3                                        [Entre 20 y 49 personas]
4                                       [Entre 50 y 249 personas]
5                                            [250 o más personas]
9                                                         [NS/NC]


In [636]:
no_respuesta('TR_TAM')

[ 0.  1.  2.  3.  4.  5. nan]


In [637]:
redefine('TR_TAM','tamaño_em')

In [638]:
proporciones('tamaño_em_',dropna=True)

,tamaño_em_
250 o más personas,0.434765
Entre 50 y 249 personas,0.212376
Entre 20 y 49 personas,0.123360
De 1 a 9 personas,0.103935
Trabajadores independientes o autónomos sin asalariados,0.063583
Entre 10 y 19 personas,0.061981


## 'TR_ANIO' 

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Año** en el que empezó a trabajar en su empleo actual.

In [639]:
df['TR_ANIO'].sort_values().unique() # No informados como '    ' o '9999'

array(['    ', '1975', '1977', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '9999'], dtype=object)

In [640]:
no_respuesta('TR_ANIO',blank='    ',nsnc='9999')

[1975. 1977. 1979. 1980. 1981. 1982. 1983. 1984. 1985. 1986. 1987. 1988.
 1989. 1990. 1991. 1992. 1993. 1994. 1995. 1996. 1997. 1998. 1999. 2000.
 2001. 2002. 2003. 2004. 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012.
 2013. 2014. 2015. 2016. 2017. 2018. 2019.   nan]


## 'TR_SUELDO'   (redefinida como 'sueldo')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Sueldo mensual neto** actual.

In [641]:
df['TR_SUELDO'].unique() # No informados como ' ' o '9'

array(['5', '2', '4', ' ', '3', '6', '1', '7', '9'], dtype=object)

In [642]:
traduce('TR_SUELDO')

                            aux_traduce
                                 unique
TR_SUELDO                              
                         [No aplicable]
1                  [Menos de 700 euros]
2                  [De 700 a 999 euros]
3              [De 1.000 a 1.499 euros]
4              [De 1.500 a 1.999 euros]
5              [De 2.000 a 2.499 euros]
6              [De 2.500 a 2.999 euros]
7          [De 3.000 euros en adelante]
9                               [NS/NC]


In [643]:
no_respuesta('TR_SUELDO')

[ 1.  2.  3.  4.  5.  6.  7. nan]


In [644]:
redefine('TR_SUELDO','sueldo')

## Nueva variable 'sueldo_pxy'
**sueldo_pxy**: proxy de sueldo mensual neto con una división en intervalos ajustada para facilitar la correspondencia con los definidos en la encuesta de doctores. 

In [645]:
# Diccionario de correspondencias entre 'sueldo' y 'sueldo_pxy':
# Se reunen en una las categorías 2 y 3 y las categorías 4 y 5 de 'sueldo'
D_sueldo_pxy=dict()
D_sueldo_pxy[1]=1
D_sueldo_pxy[2]=2
D_sueldo_pxy[3]=2
D_sueldo_pxy[4]=3
D_sueldo_pxy[5]=3
D_sueldo_pxy[6]=4
D_sueldo_pxy[7]=5

In [646]:
df['sueldo_pxy']=df['sueldo'].replace(D_sueldo_pxy)
traduce('sueldo_pxy',check=False)
df[['sueldo_','sueldo_pxy_']].groupby('sueldo_').agg(['unique']) # Comprobación

,sueldo_pxy_
,unique
sueldo_,
De 1.000 a 1.499 euros,[De 700 a 1.499 euros]
De 1.500 a 1.999 euros,[De 1.500 a 2.499 euros]
De 2.000 a 2.499 euros,[De 1.500 a 2.499 euros]
De 2.500 a 2.999 euros,[De 2.500 a 2.999 euros]
De 3.000 euros en adelante,[De 3.000 euros en adelante]
De 700 a 999 euros,[De 700 a 1.499 euros]
Menos de 700 euros,[Menos de 700 euros]


## 'TR_D19'     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Nivel de formación más apropiado** para realizar el trabajo actual.

In [647]:
df['TR_D19'].unique() # No informados como ' ' o '9'

array(['3', '2', ' ', '1', '4', '5', '6', '7', '9'], dtype=object)

In [648]:
traduce('TR_D19')

                                                  aux_traduce
                                                       unique
TR_D19                                                       
                                               [No aplicable]
1                                                 [Doctorado]
2                                                    [Máster]
3         [Título universitario (excepto doctorado o máster)]
4                   [Formación profesional de grado superior]
5       [Formación profesional de grado medio/ bachillerato ]
6                              [Formación profesional básica]
7                                            [ESO, EGB, etc.]
9                                                     [NS/NC]


In [649]:
no_respuesta('TR_D19')

[ 1.  2.  3.  4.  5.  6.  7. nan]


## 'TR_D20'       

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Área de estudio maś apropiada** para el trabajo actual.

In [650]:
df['TR_D20'].unique() # No informados como ' ' o '9'

array(['2', '1', '3', ' ', '4', '9'], dtype=object)

In [651]:
traduce('TR_D20')

                                        aux_traduce
                                             unique
TR_D20                                             
                                    [No aplicable ]
1       [Exclusivamente su propia área de estudios]
2             [Su propia área o alguna relacionada]
3                    [Un área totalmente diferente]
4                       [Ningún área en particular]
9                                           [NS/NC]


In [652]:
no_respuesta('TR_D20')

[ 1.  2.  3.  4. nan]


In [653]:
proporciones('TR_D20_',dropna=True)

,TR_D20_
Su propia área o alguna relacionada,0.483448
Un área totalmente diferente,0.265027
Ningún área en particular,0.131013
Exclusivamente su propia área de estudios,0.120512


## 'TR_D21'       

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

¿Se hace **uso** en el trabajo actual de los **conocimientos y habilidades** adquiridos en estos estudios?

In [654]:
df['TR_D21'].unique() # No informados como ' ' o '9'

array(['2', '1', ' ', '9'], dtype=object)

In [655]:
traduce('TR_D21')

           aux_traduce
                unique
TR_D21                
        [No aplicable]
1                 [Sí]
2                 [No]
9              [NS/NC]


In [656]:
no_respuesta('TR_D21')

[ 1.  2. nan]


In [657]:
proporciones('TR_D21')

,TR_D21
1.0,0.563267
2.0,0.311243
NaN,0.125490


# 'TR_D22_1'  (redefinida como 'vbtr_teoricos')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de los **conocimientos teóricos** para conseguir su trabajo actual. 

In [658]:
df['TR_D22_1'].unique() # No informados como ' ' o '9'

array(['3', '5', '4', ' ', '2', '1', '9'], dtype=object)

In [659]:
def vbtr(col,nuevo):
    """Procesa columnas referidas a influencia de distintos factores en el éxito en la búsqueda de empleo.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo)
    return

In [660]:
vbtr('TR_D22_1','vbtr_teoricos')

             aux_traduce
                  unique
TR_D22_1                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_2'  (redefinida como 'vbtr_practicos')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de las **habilidades prácticas** para conseguir su trabajo actual. 

In [661]:
df['TR_D22_2'].unique() # No informados como ' ' o '9'

array(['5', '4', ' ', '2', '3', '1', '9'], dtype=object)

In [662]:
vbtr('TR_D22_2','vbtr_practicos')

             aux_traduce
                  unique
TR_D22_2                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_3'  (redefinida como 'vbtr_idiomas')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de **conocer idiomas** para conseguir su trabajo actual. 

In [663]:
df['TR_D22_3'].unique() # No informados como ' ' o '9'

array(['4', '5', '2', ' ', '3', '1', '9'], dtype=object)

In [664]:
vbtr('TR_D22_3','vbtr_idiomas')

             aux_traduce
                  unique
TR_D22_3                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_4'  (redefinida como 'vbtr_ti')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de la formación o el dominio de la **informática** y de las **tecnologías de la información y la comunicación** para conseguir su trabajo actual. 

In [665]:
df['TR_D22_4'].unique() # No informados como ' ' o '9'

array(['3', '5', '2', ' ', '4', '1', '9'], dtype=object)

In [666]:
vbtr('TR_D22_4','vbtr_ti')

             aux_traduce
                  unique
TR_D22_4                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_4'  (redefinida como 'vbtr_personales')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de las **competencias personales y sociales** -personalidad, habilidades sociales, comunicación, capacidad de trabajar en grupo- para conseguir su trabajo actual. 

In [667]:
df['TR_D22_4'].unique() # No informados como ' ' o '9'

array([ 3.,  5.,  2., nan,  4.,  1.])

In [668]:
vbtr('TR_D22_4','vbtr_personales')

         aux_traduce
              unique
TR_D22_4            
1.0           [Nada]
2.0           [Poco]
3.0           [Algo]
4.0       [Bastante]
5.0          [Mucho]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D22_5'  (redefinida como 'vbtr_gestion')     

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

Influencia de la **capacidad de gestión, planificación y emprendimiento** para conseguir su trabajo actual. 

In [669]:
df['TR_D22_5'].unique() # No informados como ' ' o '9'

array(['4', '5', '3', ' ', '2', '1', '9'], dtype=object)

In [670]:
vbtr('TR_D22_5','vbtr_gestion')

             aux_traduce
                  unique
TR_D22_5                
          [No aplicable]
1                 [Nada]
2                 [Poco]
3                 [Algo]
4             [Bastante]
5                [Mucho]
9                [NS/NC]
[ 1.  2.  3.  4.  5. nan]


# 'TR_D23'  (completa la variable 'busqueda_tr')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

¿Ha **intentado conseguir otro trabajo remunerado** en las últimas 4 semanas?

Nota: Se usa esta información para actualizar la variable busqueda_tr'.

In [671]:
df['TR_D23'].unique() # No informados como ' ' o '9'

array(['3', '1', ' ', '2', '9'], dtype=object)

In [672]:
traduce('TR_D23')

                                                                                                     aux_traduce
                                                                                                          unique
TR_D23                                                                                                          
                                                                                                  [No aplicable]
1                                                                                                           [Sí]
2       [No, pero está esperando respuesta a solicitudes anteriores de empleo o los resultados de una oposición]
3                                                                                                           [No]
9                                                                                                        [NS/NC]


In [673]:
no_respuesta('TR_D23')

[ 1.  2.  3. nan]


In [674]:
# Se actualiza la variable 'busqueda_tr':
df['busqueda_tr'].mask((df['TR_D23']==1)|(df['TR_D23']==2),1,inplace=True)
traduce('busqueda_tr')
df['busqueda_tr_'][(df['TR_D23']==1)|(df['TR_D23']==2)].unique() # Comprobación

            aux_traduce
                 unique
busqueda_tr            
0.0                [No]
1.0                [Sí]


array(['Sí'], dtype=object)

# BLOQUE E. HISTORIAL LABORAL DEL GRADUADO UNIVERSITARIO

# 'HL_E0'  

¿Ha trabajado antes de comenzar el máster?

In [675]:
df['HL_E0'].unique() # Sin no informados

array([2, 1, 3])

In [676]:
D_dict['HL_E0']=diccionario('TRABMAS')
traduce('HL_E0')

                                                            aux_traduce
                                                                 unique
HL_E0                                                                  
1              [Sí, y continuó trabajando mientras realizaba el máster]
2      [Sí, aunque antes de comenzar el máster ya no estaba trabajando]
3                                                                  [No]


# 'HL_E00'  
(Solo para quienes declaran haber trabajado antes de comenzar el máster y haber dejado de trabajar antes de empezarlo -respuesta 2 en variable 'HL_E0-)

Momento en que dejó de trabajar antes de comenzar el máster. 

In [677]:
df['HL_E00'].unique() # No informados como ' ' o '9'

array(['3', ' ', '2', '1', '9'], dtype=object)

In [678]:
D_dict['HL_E00']=diccionario('TMODEJ')
traduce('HL_E00')

                                                     aux_traduce
                                                          unique
HL_E00                                                          
                                                  [No aplicable]
1           [Tres o más años antes de comenzar el máster (2010)]
2       [Uno o dos años antes de comenzar el máster (2011-2012)]
3             [Durante el año de comenzar el máster (2013-2014)]
9                                                        [NS/NC]


In [679]:
df[['HL_E0','HL_E00']].groupby('HL_E0').agg(['unique']) # Inspección

,HL_E00
,unique
HL_E0,
1,[ ]
2,"[3, 2, 1, 9]"
3,[ ]


# 'HL_E1'  

¿Realizó **prácticas en empresas**, instituciones o similares?

In [680]:
df['HL_E1'].unique() # Sin no informados

array([2, 4, 1, 3])

In [681]:
traduce('HL_E1')

                                 aux_traduce
                                      unique
HL_E1                                       
1      [Sí, como parte del plan de estudios]
2           [Sí, fuera del plan de estudios]
3             [Sí, ambos tipos de prácticas]
4                                       [No]


# 'HL_E2'  

(Solo para respuestas 1, 2 o 3 en variable 'HL_E1')

Duración en meses de las **prácticas en empresas**, instituciones o similares.

In [682]:
df['HL_E2'].unique() # No informados como ' ' o '9'

array(['3', ' ', '1', '2', '4', '5', '9'], dtype=object)

In [683]:
traduce('HL_E2')

              aux_traduce
                   unique
HL_E2                    
           [No aplicable]
1        [De 1 a 3 meses]
2        [De 4 a 6 meses]
3        [De 7 a 9 meses]
4      [De 10 a 12 meses]
5         [Más de un año]
9                 [NS/NC]


In [684]:
no_respuesta('HL_E2')

[ 1.  2.  3.  4.  5. nan]


# 'HL_E3'  

¿Ha tenido algún **trabajo remunerado mientras realizaba los estudios** (incluido el trabajo por cuenta propia, trabajos de formación, contrato en prácticas, puestos de becario, etc.)?

In [685]:
df['HL_E3'].unique() # Sin no informados

array([2, 1])

In [686]:
traduce('HL_E3')

      aux_traduce
           unique
HL_E3            
1            [Sí]
2            [No]


# 'HL_E5'  

¿**Continuó en ese trabajo** durante al menos 6 meses después de acabar estos estudios?

In [687]:
df['HL_E5'].unique() # No informados como ' '

array([' ', '2', '1'], dtype=object)

In [688]:
traduce('HL_E5')

          aux_traduce
               unique
HL_E5                
       [No aplicable]
1                [Sí]
2                [No]


In [689]:
no_respuesta('HL_E5')

[ 1.  2. nan]


# 'HL_JOR'  

**Tipo de jornada** en ese trabajo.

In [690]:
df['HL_JOR'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [691]:
traduce('HL_JOR')

                aux_traduce
                     unique
HL_JOR                     
                        [ ]
1        [A tiempo parcial]
2       [A tiempo completo]
9                   [NS/NC]


In [692]:
no_respuesta('HL_JOR')

[ 1.  2. nan]


# 'HL_E7'  

Una vez completados estos estudios, ¿**mejoraron las condiciones del trabajo** en el que continuó después de acabarlos?

In [693]:
df['HL_E7'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [694]:
traduce('HL_E7')

          aux_traduce
               unique
HL_E7                
       [No aplicable]
1                [Sí]
2                [No]
9             [NS/NC]


In [695]:
no_respuesta('HL_E7')

[ 1.  2. nan]


# 'HL_E8'  

Mientras tenía este empleo, ¿**buscó un trabajo mejor**?

In [696]:
df['HL_E8'].unique() # No informados como ' ' o '9'

array([' ', '1', '2', '9'], dtype=object)

In [697]:
traduce('HL_E8')

          aux_traduce
               unique
HL_E8                
       [No aplicable]
1                [Sí]
2                [No]
9             [NS/NC]


In [698]:
no_respuesta('HL_E8')

[ 1.  2. nan]


# 'HA_TRAB'  (redefinida como 'ha_trab')

Ha tenido **algún trabajo remunerado desde que terminó sus estudios** (distinto del que haya realizado mientras estudiaba), incluyendo trabajo por cuenta propia, trabajos de formación, en prácticas, puestos de becario, etc.

Nota: La variable 'HA_TRAB' se ha empleado como filtro para algunas de las variables definidas a continuación, formuladas solo a quienes responden Sí o a quienes responden No a esta pregunta. 

In [699]:
df['HA_TRAB'].unique() # Sin no informados

array([1, 2])

In [700]:
traduce('HA_TRAB')

        aux_traduce
             unique
HA_TRAB            
1              [Sí]
2              [No]


In [701]:
df[['HA_TRAB_','sit_lab_']].groupby(['sit_lab_']).agg(['unique']) # Inspección

,HA_TRAB_
,unique
sit_lab_,
En desempleo,"[Sí, No]"
Inactivo,"[Sí, No]"
Trabajando,[Sí]


In [702]:
redefine('HA_TRAB',si_no=True)

        aux_traduce
             unique
ha_trab            
0              [No]
1              [Sí]


# 'HATR_E11'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)

**Tiempo** transcurrido desde que finalizó estos estudios **hasta que empezó a trabajar**.

In [703]:
df['HATR_E11'].unique() # No informados como ' ' o '9'

array(['1', '9', '0', '3', '6', '5', '4', '2', ' '], dtype=object)

In [704]:
traduce('HATR_E11')

                                                                         aux_traduce
                                                                              unique
HATR_E11                                                                            
                                                                      [No aplicable]
0         [Continuó al menos 6 meses más en el trabajo que tenía mientras estudiaba]
1                                                              [Menos de tres meses]
2                                                                   [De 3 a 6 meses]
3                                                               [De 6 meses a 1 año]
4                                                        [De 1 año a  año y medio  ]
5                                                      [De 1 año y medio a  2 años ]
6                                                                     [2 ó más años]
9                                                                

In [705]:
df[['HATR_E11','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_E11
,unique
ha_trab_,
No,[ ]
Sí,"[1, 9, 0, 3, 6, 5, 4, 2]"


In [706]:
no_respuesta('HATR_E11')

[ 0.  1.  2.  3.  4.  5.  6. nan]


## 'HATR_E12'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)

Momento en que empezó a buscar trabajo.

In [707]:
df['HATR_E12'].unique() # No informados como ' ' o '9'

array(['1', '9', '0', '2', ' '], dtype=object)

In [708]:
traduce('HATR_E12')

                                                                         aux_traduce
                                                                              unique
HATR_E12                                                                            
                                                                      [No aplicable]
0         [Continuó al menos 6 meses más en el trabajo que tenía mientras estudiaba]
1             [Antes de terminar los estudios de la titulación que finalizó en 2014]
2           [Después de terminar los estudios de la titulación que finalizó en 2014]
9                                                                            [NS/NC]


In [709]:
df[['HA_TRAB_','HATR_E12_','sit_lab_']].groupby('HA_TRAB_').agg(['unique']) # Inspección
# Solo se ha preguntado a quien ha trabajado después de terminar sus estudios, con independencia de su ...
# ... situación laboral actual.

,HATR_E12_,sit_lab_
,unique,unique
HA_TRAB_,,
No,[No aplicable],"[Inactivo, En desempleo]"
Sí,"[Antes de terminar los estudios de la titulación que finalizó en 2014, NS/NC, Continuó al menos 6 meses más en el trabajo que tenía mientras estudiaba, Después de terminar los estudios de la titulación que finalizó en 2014]","[Trabajando, En desempleo, Inactivo]"


In [710]:
no_respuesta('HATR_E12')

[ 0.  1.  2. nan]


## 'HATR_BSQ1'  (redefinida como 'etr_internet', actualiza 'btr_internet')

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)

¿**Encontró trabajo** a través de ofertas de trabajo en el **periódico, internet**, etc.?  

Nota: Esta es la primera de una serie de preguntas sobre estrategias de búsqueda de empleo entre quienes han tenido alguno desde que terminaron sus estudios. La variable 'HATR_BSQ1' es distinta de 'TRPARA_D7_1' (o 'btr_internet'); 'btr_internet' se refiere a la búsqueda de trabajo a través de anuncios en prensa o internet y se formula solo a desempleados, mientras que 'TRPARA_D7_1' se refiere al **éxito de esa estrategia para encontrar trabajo** y se formula a todos los que han trabajado alguna vez desde que terminaron sus estudios.   
No obstante, esta y otras variables de esta serie permiten actualizar algunas de las variables con prefijo 'btr_' definidas originalmente solo para desempleados. 

In [711]:
df['HATR_BSQ1'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [712]:
traduce('HATR_BSQ1')

              aux_traduce
                   unique
HATR_BSQ1                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]


In [713]:
df[['HATR_BSQ1_','btr_internet_','HA_TRAB_','sit_lab_']].groupby(['sit_lab_','HA_TRAB_']).agg(['unique'])
# Solo se ha preguntado a quien ha trabajado desde que terminó sus estudios (a diferencia de 'btr_internet', ...
# ... que se preguntó a los desempleados)

HATR_BSQ1_  btr_internet_
                                unique         unique
sit_lab_     HA_TRAB_                                
En desempleo No         [No aplicable]       [Sí, No]
             Sí        [No, Sí, NS/NC]  [No, Sí, nan]
Inactivo     No         [No aplicable]          [nan]
             Sí        [Sí, No, NS/NC]          [nan]
Trabajando   Sí        [No, Sí, NS/NC]          [nan]

In [714]:
def etr(col,nuevo):
    """Procesa las columnas referidas a las estrategias de búsqueda de empleo de quienes tuvieron alguno después
    de terminar sus estudios.
    """
    traduce(col)
    no_respuesta(col)
    redefine(col,nuevo,si_no=True)
    return

In [715]:
etr('HATR_BSQ1','etr_internet')

              aux_traduce
                   unique
HATR_BSQ1                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
             aux_traduce
                  unique
etr_internet            
0.0                 [No]
1.0                 [Sí]


In [716]:
def actualiza_btr(col,btr):
    """ Actualiza las variables con prefijo btr_ definidas para las distintas estrategias de búsqueda de empleo, 
    añadiendo las respuestas declaradas por quienes han trabajado después de terminar sus estudios: registra la 
    estrategia definida en la variable 'col' como utilizada por el encuestado cuando este declara que le sirvió 
    para encontrar empleo. No se infiere que no la haya usado cuando el encuestado no le atribuye éxito. 
    """
    df[btr].mask(df[col]==1,1,inplace=True)
    traduce(btr)
    print (df[[str(col)+'_',str(btr)+'_']][df['HA_TRAB']==1].groupby([str(col)+'_'],
                                                                        dropna=False).agg(['unique']))
    return

In [717]:
actualiza_btr('HATR_BSQ1','btr_internet')

             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]
            btr_internet_
                   unique
HATR_BSQ1_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ2'  (redefinida como 'etr_spe', actualiza 'btr_spe')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de los **servicios públicos de empleo**?

In [718]:
df['HATR_BSQ2'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [719]:
etr('HATR_BSQ2','etr_spe')

              aux_traduce
                   unique
HATR_BSQ2                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
etr_spe            
0.0            [No]
1.0            [Sí]


In [720]:
actualiza_btr('HATR_BSQ2','btr_spe')

        aux_traduce
             unique
btr_spe            
0.0            [No]
1.0            [Sí]
                 btr_spe_
                   unique
HATR_BSQ2_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ3'  (redefinida como 'etr_univ', actualiza 'btr_univ')    

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de los **servicios de empleo de la universidad** (bolsa de trabajo, asociaciones de exalumnos, etc.)?

In [721]:
df['HATR_BSQ3'].unique() # No informados como ' ' o '9'

array(['1', '2', '9', ' '], dtype=object)

In [722]:
etr('HATR_BSQ3','etr_univ')

              aux_traduce
                   unique
HATR_BSQ3                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
etr_univ            
0.0             [No]
1.0             [Sí]


In [723]:
actualiza_btr('HATR_BSQ3','btr_univ')

         aux_traduce
              unique
btr_univ            
0.0             [No]
1.0             [Sí]
                btr_univ_
                   unique
HATR_BSQ3_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ4'  (redefinida como 'etr_ett', actualiza 'btr_ett')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de **empresas de trabajo temporal**?

In [724]:
df['HATR_BSQ4'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [725]:
etr('HATR_BSQ4','etr_ett')

              aux_traduce
                   unique
HATR_BSQ4                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
etr_ett            
0.0            [No]
1.0            [Sí]


In [726]:
actualiza_btr('HATR_BSQ4','btr_ett')

        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]
                 btr_ett_
                   unique
HATR_BSQ4_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ5'  (redefinida como 'etr_bolsa', actualiza 'btr_bolsa')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** a través de una **bolsa de empleo**?

In [727]:
df['HATR_BSQ5'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [728]:
etr('HATR_BSQ5','etr_bolsa')

              aux_traduce
                   unique
HATR_BSQ5                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
          aux_traduce
               unique
etr_bolsa            
0.0              [No]
1.0              [Sí]


In [729]:
actualiza_btr('HATR_BSQ5','btr_bolsa')

          aux_traduce
               unique
btr_bolsa            
0.0              [No]
1.0              [Sí]
               btr_bolsa_
                   unique
HATR_BSQ5_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ6'  (redefinida como 'etr_contactos', actualiza 'btr_contactos')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** contactando con el empleador por iniciativa propia o utilizando **contactos personales** (familia, amigos)

In [730]:
df['HATR_BSQ6'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [731]:
etr('HATR_BSQ6','etr_contactos')

              aux_traduce
                   unique
HATR_BSQ6                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
etr_contactos            
0.0                  [No]
1.0                  [Sí]


In [732]:
actualiza_btr('HATR_BSQ6','btr_contactos')

              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]
           btr_contactos_
                   unique
HATR_BSQ6_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ7'  (redefinida como 'etr_empleador', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** gracias a que el **empleador se puso en contacto** con usted?

In [733]:
df['HATR_BSQ7'].unique()

array(['2', '1', '9', ' '], dtype=object)

In [734]:
etr('HATR_BSQ7','etr_empleador')

              aux_traduce
                   unique
HATR_BSQ7                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
etr_empleador            
0.0                  [No]
1.0                  [Sí]


In [735]:
actualiza_btr('HATR_BSQ7','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
                btr_otro_
                   unique
HATR_BSQ7_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN             [nan, No]


## 'HATR_BSQ8'  (redefinida como 'etr_practicas', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿Continuó con las **prácticas en empresas** o instituciones que realizó durante la carrera?

In [736]:
df['HATR_BSQ8'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [737]:
etr('HATR_BSQ8','etr_practicas')

              aux_traduce
                   unique
HATR_BSQ8                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
              aux_traduce
                   unique
etr_practicas            
0.0                  [No]
1.0                  [Sí]


In [738]:
actualiza_btr('HATR_BSQ8','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
                btr_otro_
                   unique
HATR_BSQ8_               
No          [nan, Sí, No]
Sí                   [Sí]
NaN             [nan, No]


## 'HATR_BSQ9'  (redefinida como 'etr_opo', actualiza 'btr_opo')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** preparando unas **oposiciones**?

In [739]:
df['HATR_BSQ9'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [740]:
etr('HATR_BSQ9','etr_opo')

              aux_traduce
                   unique
HATR_BSQ9                
           [No aplicable]
1                    [Sí]
2                    [No]
9                 [NS/NC]
[ 1.  2. nan]
        aux_traduce
             unique
etr_opo            
0.0            [No]
1.0            [Sí]


In [741]:
actualiza_btr('HATR_BSQ9','btr_opo')

        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]
                 btr_opo_
                   unique
HATR_BSQ9_               
No          [nan, No, Sí]
Sí                   [Sí]
NaN         [nan, No, Sí]


## 'HATR_BSQ10'  (redefinida como 'etr_auto', actualiza 'btr_auto)  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-)  

¿**Encontró trabajo** montando un **negocio propio**?

In [742]:
df['HATR_BSQ10'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [743]:
etr('HATR_BSQ10','etr_auto')

               aux_traduce
                    unique
HATR_BSQ10                
            [No aplicable]
1                     [Sí]
2                     [No]
9                  [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
etr_auto            
0.0             [No]
1.0             [Sí]


In [744]:
actualiza_btr('HATR_BSQ10','btr_auto')

         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]
                 btr_auto_
                    unique
HATR_BSQ10_               
No           [nan, No, Sí]
Sí                    [Sí]
NaN          [nan, No, Sí]


## 'HATR_BSQ11'  (redefinida como 'etr_otro', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿**Encontró trabajo de otra forma**?

In [745]:
df['HATR_BSQ11'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [746]:
etr('HATR_BSQ11','etr_otro')

               aux_traduce
                    unique
HATR_BSQ11                
            [No aplicable]
1                     [Sí]
2                     [No]
9                  [NS/NC]
[ 1.  2. nan]
         aux_traduce
              unique
etr_otro            
0.0             [No]
1.0             [Sí]


In [747]:
actualiza_btr('HATR_BSQ11','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
                 btr_otro_
                    unique
HATR_BSQ11_               
No           [nan, Sí, No]
Sí                    [Sí]
NaN              [nan, No]


## 'P_CONTI'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿**Continua en su primer empleo**?

In [748]:
df['P_CONTI'].unique() # No informados como ' ' 

array(['1', '2', ' '], dtype=object)

In [749]:
traduce('P_CONTI')

            aux_traduce
                 unique
P_CONTI                
         [No aplicable]
1                  [Sí]
2                  [No]


In [750]:
df[['P_CONTI','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,P_CONTI
,unique
ha_trab_,
No,[ ]
Sí,"[1, 2]"


In [751]:
no_respuesta('P_CONTI')

[ 1.  2. nan]


## 'PR_PAIS'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**País** en el que desempeñaba su **primer empleo**.

In [752]:
df['PR_PAIS'].unique() # No informados como '  ' o '99'

array(['01', '07', '02', '  ', '06', '05', '08', '09', '04', '10', '03',
       '11'], dtype=object)

In [753]:
no_respuesta('PR_PAIS',blank='  ',nsnc='99')
traduce('PR_PAIS')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. nan]
                                            aux_traduce
                                                 unique
PR_PAIS                                                
1.0                                            [España]
2.0                                       [Reino Unido]
3.0                                          [Alemania]
4.0                                           [Francia]
5.0                     [Otro país de la Unión Europea]
6.0                [Otro país europeo (fuera de la UE)]
7.0                      [Un país de América del Norte]
8.0      [Un país de América Central o América del Sur]
9.0                                   [Un país de Asia]
10.0                                [Un país de África]
11.0                               [Un país de Oceanía]


In [754]:
df[['ha_trab_','PR_PAIS']].groupby(['ha_trab_']).agg(['unique']) # Inspección

,PR_PAIS
,unique
ha_trab_,
No,[nan]
Sí,"[1.0, 7.0, 2.0, 6.0, 5.0, 8.0, 9.0, 4.0, 10.0, 3.0, 11.0]"


## 'PR_OCU'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Ocupación** que desempeñaba en su **primer empleo**.

In [755]:
df['PR_OCU'].unique() # No informados como '  ' o '99'

array(['24', '22', '23', '31', '58', '27', '25', '95', '44', '43', '99',
       '55', '41', '45', '37', '21', '57', '51', '12', '26', '38', '28',
       '59', '35', '52', '93', '36', '33', '75', '54', '77', '13', '92',
       '11', '98', '81', '73', '34', '29', '14', '91', '  ', '56', '97',
       '53', '42', '15', '82', '94', '84', '32', '78', '61', '00', '96',
       '72', '64', '71', '62', '76', '74', '83'], dtype=object)

In [756]:
df[['PR_OCU','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_OCU
,unique
ha_trab_,
No,[ ]
Sí,"[24, 22, 23, 31, 58, 27, 25, 95, 44, 43, 99, 55, 41, 45, 37, 21, 57, 51, 12, 26, 38, 28, 59, 35, 52, 93, 36, 33, 75, 54, 77, 13, 92, 11, 98, 81, 73, 34, 29, 14, 91, 56, 97, 53, 42, 15, 82, 94, 84, 32, 78, 61, 00, 96, 72, 64, 71, 62, 76, 74, 83]"


In [757]:
no_respuesta('PR_OCU',blank='  ',nsnc='99')

[ 0. 11. 12. 13. 14. 15. 21. 22. 23. 24. 25. 26. 27. 28. 29. 31. 32. 33.
 34. 35. 36. 37. 38. 41. 42. 43. 44. 45. 51. 52. 53. 54. 55. 56. 57. 58.
 59. 61. 62. 64. 71. 72. 73. 74. 75. 76. 77. 78. 81. 82. 83. 84. 91. 92.
 93. 94. 95. 96. 97. 98. nan]


## 'PR_SIT_PR'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Situación profesional** en su **primer empleo**.

In [758]:
df['PR_SIT_PR'].unique() # No informados como ' '

array(['2', '3', '1', '5', '6', '4', ' '], dtype=object)

In [759]:
traduce('PR_SIT_PR')

                                                                                aux_traduce
                                                                                     unique
PR_SIT_PR                                                                                  
                                                                             [No aplicable]
1                 [Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario]
2          [Asalariado con trabajo permanente o contrato de trabajo de duración indefinida]
3           [Asalariado con trabajo temporal o contrato de trabajo de duración determinada]
4                                                              [Empresario con asalariados]
5                                   [Trabajador independiente o empresario sin asalariados]
6                                                  [Ayuda en la empresa o negocio familiar]


In [760]:
df[['PR_SIT_PR_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_SIT_PR_
,unique
ha_trab_,
No,[No aplicable]
Sí,"[Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, Asalariado con trabajo temporal o contrato de trabajo de duración determinada, Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario, Trabajador independiente o empresario sin asalariados, Ayuda en..."


In [761]:
no_respuesta('PR_SIT_PR')

[ 1.  2.  3.  4.  5.  6. nan]


## 'PR_JOR'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

Tipo de **jornada de trabajo** en su **primer empleo**.

In [762]:
df['PR_JOR'].unique() # No informados como ' '

array(['2', '1', ' '], dtype=object)

In [763]:
traduce('PR_JOR')

                aux_traduce
                     unique
PR_JOR                     
                        [ ]
1        [A tiempo parcial]
2       [A tiempo completo]


In [764]:
df[['PR_JOR_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_JOR_
,unique
ha_trab_,
No,[ ]
Sí,"[A tiempo completo, A tiempo parcial]"


In [765]:
no_respuesta('PR_JOR')

[ 1.  2. nan]


## 'PR_NIVEL'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Nivel de formación** más apropiado para su **primer empleo**.

In [766]:
df['PR_NIVEL'].unique() # No informados como ' ' o '9'

array(['3', '2', '4', '7', '5', '1', '6', ' ', '9'], dtype=object)

In [767]:
traduce('PR_NIVEL')

                                                    aux_traduce
                                                         unique
PR_NIVEL                                                       
                                                 [No aplicable]
1                                                   [Doctorado]
2                                                      [Máster]
3           [Título universitario (excepto doctorado o máster)]
4                     [Formación profesional de grado superior]
5         [Formación profesional de grado medio/ bachillerato ]
6                                [Formación profesional básica]
7                                              [ESO, EGB, etc.]
9                                                       [NS/NC]


In [768]:
df[['PR_NIVEL','ha_trab_','sit_lab_']].groupby(['ha_trab_','sit_lab_']).agg(['unique']) # Inspección

PR_NIVEL
                                         unique
ha_trab_ sit_lab_                              
No       En desempleo                       [ ]
         Inactivo                           [ ]
Sí       En desempleo  [2, 3, 4, 7, 6, 5, 9, 1]
         Inactivo      [7, 5, 2, 3, 4, 6, 1, 9]
         Trabajando    [3, 2, 4, 7, 5, 1, 6, 9]

In [769]:
no_respuesta('PR_NIVEL')

[ 1.  2.  3.  4.  5.  6.  7. nan]


## 'PR_AREA'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Área de estudio** más apropiada para su **primer empleo**.

In [770]:
df['PR_AREA'].unique() # No informados como ' ' o '9'

array(['2', '1', '3', '4', ' ', '9'], dtype=object)

In [771]:
traduce('PR_AREA')

                                         aux_traduce
                                              unique
PR_AREA                                             
                                     [No aplicable ]
1        [Exclusivamente su propia área de estudios]
2              [Su propia área o alguna relacionada]
3                     [Un área totalmente diferente]
4                        [Ningún área en particular]
9                                            [NS/NC]


In [772]:
df[['PR_AREA_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_AREA_
,unique
ha_trab_,
No,[No aplicable ]
Sí,"[Su propia área o alguna relacionada, Exclusivamente su propia área de estudios, Un área totalmente diferente, Ningún área en particular, NS/NC]"


In [773]:
no_respuesta('PR_AREA')

[ 1.  2.  3.  4. nan]


## 'PR_CONOC'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿Se hacía uso en su **primer empleo** de los **conocimientos y habilidades** adquiridos en estos estudios?

In [774]:
df['PR_CONOC'].unique() # No informados como ' ' o '9'

array(['2', '1', '9', ' '], dtype=object)

In [775]:
traduce('PR_CONOC')

             aux_traduce
                  unique
PR_CONOC                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [776]:
df[['PR_CONOC_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_CONOC_
,unique
ha_trab_,
No,[No aplicable]
Sí,"[No, Sí, NS/NC]"


In [777]:
no_respuesta('PR_CONOC')

[ 1.  2. nan]


## 'PR_SUELDO'  (redefinida como 'pr_sueldo')

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Sueldo mensual neto del primer trabajo** o de su empleo actual en el momento en el que empezó.

In [778]:
df['PR_SUELDO'].unique() # No informados como ' ' o '9'

array(['1', '5', '2', '3', '4', '9', '7', '6', ' '], dtype=object)

In [779]:
traduce('PR_SUELDO')

                            aux_traduce
                                 unique
PR_SUELDO                              
                         [No aplicable]
1                  [Menos de 700 euros]
2                  [De 700 a 999 euros]
3              [De 1.000 a 1.499 euros]
4              [De 1.500 a 1.999 euros]
5              [De 2.000 a 2.499 euros]
6              [De 2.500 a 2.999 euros]
7          [De 3.000 euros en adelante]
9                               [NS/NC]


In [780]:
df[['PR_SUELDO','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,PR_SUELDO
,unique
ha_trab_,
No,[ ]
Sí,"[1, 5, 2, 3, 4, 9, 7, 6]"


In [781]:
no_respuesta('PR_SUELDO')

[ 1.  2.  3.  4.  5.  6.  7. nan]


In [782]:
redefine('PR_SUELDO','pr_sueldo')

## 'HATR_TIEMP'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Tiempo que ha estado trabajando** desde que terminó sus estudios. 

In [783]:
df['HATR_TIEMP'].unique() # No informados como ' ' o '9'

array(['5', '4', '1', '3', '2', ' ', '9'], dtype=object)

In [784]:
traduce('HATR_TIEMP')

                            aux_traduce
                                 unique
HATR_TIEMP                             
                         [No aplicable]
1                    [Menos de 6 meses]
2                  [De 6 meses a 1 año]
3              [De 1 año a año y medio]
4           [De 1 año y medio a 2 años]
5                        [2 o más años]
9                               [NS/NC]


In [785]:
df[['HATR_TIEMP','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_TIEMP
,unique
ha_trab_,
No,[ ]
Sí,"[5, 4, 1, 3, 2, 9]"


In [786]:
no_respuesta('HATR_TIEMP')

[ 1.  2.  3.  4.  5. nan]


## 'HATR_NUM'  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

**Número de empleadores distintos** que ha tenido.

In [787]:
df['HATR_NUM'].unique() # No informados como '  ' o '99'

array(['02', '04', '03', '01', '05', '08', '06', '09', '00', '07', '  ',
       '10', '11', '99'], dtype=object)

In [788]:
no_respuesta('HATR_NUM',blank='  ',nsnc='99')
traduce('HATR_NUM')

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. nan]
         aux_traduce
              unique
HATR_NUM            
0.0            [0.0]
1.0            [1.0]
2.0            [2.0]
3.0            [3.0]
4.0            [4.0]
5.0            [5.0]
6.0            [6.0]
7.0            [7.0]
8.0            [8.0]
9.0            [9.0]
10.0          [10.0]
11.0      [11 o más]


In [789]:
df[['HATR_NUM','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_NUM
,unique
ha_trab_,
No,[nan]
Sí,"[2.0, 4.0, 3.0, 1.0, 5.0, 8.0, 6.0, 9.0, 0.0, 7.0, 10.0, 11.0, nan]"


## 'HATR_E27'  (redefinida como 'utilidad')

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios -respuesta 1 en 'HA_TRAB' o 'ha_trab'-) 

¿Cree que el título universitario le ha servido para encontrar trabajo?

In [790]:
df['HATR_E27'].unique() # No informados como ' ' o '9'

array(['1', '2', ' ', '9'], dtype=object)

In [791]:
traduce('HATR_E27')

             aux_traduce
                  unique
HATR_E27                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [792]:
df[['HATR_E27_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,HATR_E27_
,unique
ha_trab_,
No,[No aplicable]
Sí,"[Sí, No, NS/NC]"


In [793]:
no_respuesta('HATR_E27')

[ 1.  2. nan]


In [794]:
redefine('HATR_E27','utilidad',si_no=True)

         aux_traduce
              unique
utilidad            
0.0             [No]
1.0             [Sí]


In [795]:
proporciones('utilidad_')

,utilidad_
Sí,0.540625
No,0.441000
NaN,0.018375


## 'NOTRAB_BUS'  (completa la variable 'busqueda_tr')

(Solo para quienes **declaran no haber trabajado** después de haber terminado sus estudios -respuesta 2 en 'HA_TRAB' o 0 en 'ha_trab'-) 

¿**Ha buscado empleo** alguna vez?

Nota: Se usa esta información para completar la recogida en la variable 'busqueda_tr'.

In [796]:
df['NOTRAB_BUS'].unique() # No informados como ' ' o '9'

array([' ', '2', '1', '9'], dtype=object)

In [797]:
traduce('NOTRAB_BUS')

               aux_traduce
                    unique
NOTRAB_BUS                
            [No aplicable]
1                     [Sí]
2                     [No]
9                  [NS/NC]


In [798]:
df[['NOTRAB_BUS_','ha_trab_']].groupby('ha_trab_').agg(['unique']) # Inspección

,NOTRAB_BUS_
,unique
ha_trab_,
No,"[No, Sí, NS/NC]"
Sí,[No aplicable]


In [799]:
df[['NOTRAB_BUS_','busqueda_tr_','sit_lab']][df['sit_lab']==3].groupby(['busqueda_tr_','sit_lab'],
                                                                       dropna=False).agg(['unique'])
# No contradice la información facilitada por desempleados en la variable 'TRINAC_D4' (recogida en 'busqueda_tr')

,,NOTRAB_BUS_
,,unique
busqueda_tr_,sit_lab,
No,3.0,"[No, No aplicable]"
Sí,3.0,"[No aplicable, Sí]"
NaN,3.0,[NS/NC]


In [800]:
df[['NOTRAB_BUS_','sit_lab_']].groupby('sit_lab_',dropna=False).agg(['unique'])
# Sin información para quienes han trabajado después de terminar los estudios (incluye todos los ocupados y ...
# ... los desempleados que sí han trabajado).

,NOTRAB_BUS_
,unique
sit_lab_,
En desempleo,"[No aplicable, Sí]"
Inactivo,"[No aplicable, No, Sí, NS/NC]"
Trabajando,[No aplicable]


In [801]:
no_respuesta('NOTRAB_BUS')

[ 1.  2. nan]


In [802]:
# Se usa esta información para actualizar la variable 'busqueda_tr':
df['busqueda_tr'].mask(df['NOTRAB_BUS']==1,1,inplace=True)
df['busqueda_tr'].mask(df['NOTRAB_BUS']==2,0,inplace=True)
traduce('busqueda_tr')
df['busqueda_tr'][df['NOTRAB_BUS']==1].unique() # Comprobación

            aux_traduce
                 unique
busqueda_tr            
0.0                [No]
1.0                [Sí]


array([1.])

In [803]:
df['busqueda_tr'][df['NOTRAB_BUS']==2].unique() # Comprobación 

array([0.])

## 'NOTRAB_BUSM'  

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de haber terminado los estudios) 

Momento en que empezó a buscar trabajo. 

In [804]:
df['NOTRAB_BUSM'].unique()

array([' ', '1', '2', '9'], dtype=object)

In [805]:
traduce('NOTRAB_BUSM')

                                                                        aux_traduce
                                                                             unique
NOTRAB_BUSM                                                                        
                                                                     [No aplicable]
1              [Antes de acabar los estudios de la titulación que finalizó en 2014]
2            [Después de acabar los estudios de la titulación que finalizó en 2014]
9                                                                           [NS/NC]


In [806]:
df[['NOTRAB_BUSM_','NOTRAB_BUS_','sit_lab_']].groupby(['NOTRAB_BUS_','sit_lab_'],
                                                      dropna=False).agg(['unique']) # Inspección
# Sin información para quienes declaran haber trabajado después de terminar sus estudios.

NOTRAB_BUSM_
                                                                                                                                                                     unique
NOTRAB_BUS_ sit_lab_                                                                                                                                                       
No          Inactivo                                                                                                                                         [No aplicable]
Sí          En desempleo         [Después de acabar los estudios de la titulación que finalizó en 2014, Antes de acabar los estudios de la titulación que finalizó en 2014]
            Inactivo      [Antes de acabar los estudios de la titulación que finalizó en 2014, Después de acabar los estudios de la titulación que finalizó en 2014, NS/NC]
NaN         En desempleo                                                                                                                                     [No aplicable]
            Inactivo                                                                                                                                         [No aplicable]
            Trabajando                                                                                                                                       [No aplicable]

In [807]:
no_respuesta('NOTRAB_BUSM')

[ 1.  2. nan]


## 'NOTRAB_BUST'  

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

**Tiempo que lleva buscando trabajo**. 

In [808]:
df['NOTRAB_BUST'].unique() # No informados como ' ' o '9'

array([' ', '3', '6', '1', '2', '5', '4', '9'], dtype=object)

In [809]:
traduce('NOTRAB_BUST')

                               aux_traduce
                                    unique
NOTRAB_BUST                               
                            [No aplicable]
1                    [Menos de tres meses]
2                         [De 3 a 6 meses]
3                     [De 6 meses a 1 año]
4               [De 1 año a año y medio  ]
5            [De 1 año y medio a  2 años ]
6                           [2 ó más años]
9                                  [NS/NC]


In [810]:
df[['NOTRAB_BUST_','NOTRAB_BUS_','sit_lab_']].groupby(['NOTRAB_BUS_','sit_lab_'],
                                                      dropna=False).agg(['unique']) # Inspección

NOTRAB_BUST_
                                                                                                                                                         unique
NOTRAB_BUS_ sit_lab_                                                                                                                                           
No          Inactivo                                                                                                                             [No aplicable]
Sí          En desempleo  [2 ó más años, De 3 a 6 meses, De 1 año y medio a  2 años , Menos de tres meses, De 1 año a año y medio  , NS/NC, De 6 meses a 1 año]
            Inactivo      [De 6 meses a 1 año, 2 ó más años, Menos de tres meses, De 3 a 6 meses, De 1 año y medio a  2 años , De 1 año a año y medio  , NS/NC]
NaN         En desempleo                                                                                                                         [No aplicable]
            Inactivo                                                                                                                             [No aplicable]
            Trabajando                                                                                                                           [No aplicable]

In [811]:
no_respuesta('NOTRAB_BUST')

[ 1.  2.  3.  4.  5.  6. nan]


## 'NOTRAB_BSQ1'  (completa la información de la variable 'btr_internet')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de anuncios en periódicos, **internet**, etc.?

Nota: 'NOTRAB_BSQ1' recoge la misma información que la variable 'TRPARA_D7_1' definida para desempleados (redefinida como 'btr_internet'). 'NOTRAB_BSQ1' está definida para quienes no han trabajado y sí han buscado empleo. Esto incluye algunos desempleados. Se ha comprobado que la información de estas dos variables no es contradictoria. 

In [812]:
df['NOTRAB_BSQ1'].unique() # No informados como ' '

array([' ', '1', '2'], dtype=object)

In [813]:
traduce('NOTRAB_BSQ1')

                aux_traduce
                     unique
NOTRAB_BSQ1                
             [No aplicable]
1                      [Sí]
2                      [No]


In [814]:
df[['NOTRAB_BSQ1_','NOTRAB_BUS_','ha_trab_']].groupby('NOTRAB_BUS_',dropna=False).agg(['unique']) # Inspección
# Solo se ha preguntado a quienes declaran no haber trabajado después de terminar sus estudios y haber ...
# ... buscado empleo. 

,NOTRAB_BSQ1_,ha_trab_
,unique,unique
NOTRAB_BUS_,,
No,[No aplicable],[No]
Sí,"[Sí, No]",[No]
NaN,[No aplicable],"[Sí, No]"


In [815]:
df[['NOTRAB_BSQ1','btr_internet_','sit_lab_']][df['sit_lab']==2].groupby(['NOTRAB_BSQ1','sit_lab_'],
                                                                         dropna=False).agg(['unique']) 
# No contradice lo declarado por los desempleados en la variable 'TRPARA_D7_1' (redefinida como 'btr_internet')

,,btr_internet_
,,unique
NOTRAB_BSQ1,sit_lab_,
,En desempleo,"[No, Sí, nan]"
1,En desempleo,[Sí]
2,En desempleo,[No]


In [816]:
def actualiza_btr(col,btr):
    """ Actualiza las variables con prefijo btr_ definidas para las distintas estrategias de búsqueda de empleo, 
    añadiendo las respuestas declaradas por quienes no han trabajado y sí han buscado empleo.
    """
    no_respuesta(col)
    df[btr].mask(df[col]==1,1,inplace=True)
    df[btr].mask(df[col]==2,0,inplace=True)
    traduce(btr)
    print (df[[str(col)+'_',str(btr)+'_']][df['NOTRAB_BUS']==1].groupby([str(col)+'_'],
                                                                        dropna=False).agg(['unique']))
    return

In [817]:
actualiza_btr('NOTRAB_BSQ1','btr_internet')

[ 1.  2. nan]
             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]
             btr_internet_
                    unique
NOTRAB_BSQ1_              
No                    [No]
Sí                    [Sí]


## 'NOTRAB_BSQ2'  (completa la información de la variable 'btr_spe')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de los **servicios públicos de empleo**?

Nota: 'NOTRAB_BSQ2' recoge la misma información que la variable 'TRPARA_D7_2' definida para desempleados (redefinida como 'btr_sep'). 

In [818]:
df['NOTRAB_BSQ2'].unique() # No informados como ' ' 

array([' ', '2', '1'], dtype=object)

In [819]:
traduce('NOTRAB_BSQ2')

                aux_traduce
                     unique
NOTRAB_BSQ2                
             [No aplicable]
1                      [Sí]
2                      [No]


In [820]:
actualiza_btr('NOTRAB_BSQ2','btr_spe')

[ 1.  2. nan]
        aux_traduce
             unique
btr_spe            
0.0            [No]
1.0            [Sí]
             btr_spe_
               unique
NOTRAB_BSQ2_         
No               [No]
Sí               [Sí]


## 'NOTRAB_BSQ3'  (completa la información de la variable 'btr_univ')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de los **servicios de empleo de la universidad** (bolsa de trabajo, asociaciones de exalumnos, etc.)?

Nota: 'NOTRAB_BSQ3' recoge la misma información que la variable 'TRPARA_D7_3' definida para desempleados (redefinida como 'btr_univ'). 

In [821]:
df['NOTRAB_BSQ3'].unique() # No informados como ' ' 

array([' ', '2', '1'], dtype=object)

In [822]:
traduce('NOTRAB_BSQ3')

                aux_traduce
                     unique
NOTRAB_BSQ3                
             [No aplicable]
1                      [Sí]
2                      [No]


In [823]:
actualiza_btr('NOTRAB_BSQ3','btr_univ')

[ 1.  2. nan]
         aux_traduce
              unique
btr_univ            
0.0             [No]
1.0             [Sí]
             btr_univ_
                unique
NOTRAB_BSQ3_          
No                [No]
Sí                [Sí]


## 'NOTRAB_BSQ4'  (completa la información de la variable 'btr_ett')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de **empresas de trabajo temporal**?

Nota: 'NOTRAB_BSQ4' recoge la misma información que la variable 'TRPARA_D7_4' definida para desempleados (redefinida como 'btr_ett'). 

In [824]:
df['NOTRAB_BSQ4'].unique() # No informados como ' ' o '9'

array([' ', '2', '1'], dtype=object)

In [825]:
traduce('NOTRAB_BSQ4')

                aux_traduce
                     unique
NOTRAB_BSQ4                
             [No aplicable]
1                      [Sí]
2                      [No]


In [826]:
actualiza_btr('NOTRAB_BSQ4','btr_ett')

[ 1.  2. nan]
        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]
             btr_ett_
               unique
NOTRAB_BSQ4_         
No               [No]
Sí               [Sí]


## 'NOTRAB_BSQ5'  (completa la información de la variable 'btr_bolsa')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo a través de una **bolsa de empleo**?

Nota: 'NOTRAB_BSQ5' recoge la misma información que la variable 'TRPARA_D7_5' definida para desempleados (redefinida como 'btr_bolsa'). 

In [827]:
df['NOTRAB_BSQ5'].unique() # No informados como ' ' o '9'

array([' ', '1', '2'], dtype=object)

In [828]:
traduce('NOTRAB_BSQ5')

                aux_traduce
                     unique
NOTRAB_BSQ5                
             [No aplicable]
1                      [Sí]
2                      [No]


In [829]:
actualiza_btr('NOTRAB_BSQ5','btr_bolsa')

[ 1.  2. nan]
          aux_traduce
               unique
btr_bolsa            
0.0              [No]
1.0              [Sí]
             btr_bolsa_
                 unique
NOTRAB_BSQ5_           
No                 [No]
Sí                 [Sí]


## 'NOTRAB_BSQ6'  (completa la información de la variable 'btr_contactos')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo buscando ayuda entre profesionales conocidos del sector o **contactos personales** (familia, amigos)?

Nota: 'NOTRAB_BSQ6' recoge la misma información que la variable 'TRPARA_D7_6' definida para desempleados (redefinida como 'btr_contactos'). 

In [830]:
df['NOTRAB_BSQ6'].unique() # No informados como ' ' o '9'

array([' ', '2', '1'], dtype=object)

In [831]:
traduce('NOTRAB_BSQ6')

                aux_traduce
                     unique
NOTRAB_BSQ6                
             [No aplicable]
1                      [Sí]
2                      [No]


In [832]:
actualiza_btr('NOTRAB_BSQ6','btr_contactos')

[ 1.  2. nan]
              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]
             btr_contactos_
                     unique
NOTRAB_BSQ6_               
No                     [No]
Sí                     [Sí]


## 'NOTRAB_BSQ7'  (completa la información de la variable 'btr_empresas')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo **contactando directamente con las empresas**?  

Nota: 'NOTRAB_BSQ7' recoge la misma información que la variable 'TRPARA_D7_7' definida para desempleados (redefinida como 'btr_empresas'). 

In [833]:
df['NOTRAB_BSQ7'].unique() # No informados como ' ' o '9'

array([' ', '2', '1'], dtype=object)

In [834]:
traduce('NOTRAB_BSQ7')

                aux_traduce
                     unique
NOTRAB_BSQ7                
             [No aplicable]
1                      [Sí]
2                      [No]


In [835]:
actualiza_btr('NOTRAB_BSQ7','btr_empresas')

[ 1.  2. nan]
             aux_traduce
                  unique
btr_empresas            
0.0                 [No]
1.0                 [Sí]
             btr_empresas_
                    unique
NOTRAB_BSQ7_              
No                    [No]
Sí                    [Sí]


## 'NOTRAB_BSQ8'  (completa la información de la variable 'btr_opo')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo **preparando unas oposiciones**?  

Nota: 'NOTRAB_BSQ8' recoge la misma información que la variable 'TRPARA_D7_8' definida para desempleados (redefinida como 'btr_opo'). 

In [836]:
df['NOTRAB_BSQ8'].unique() # No informados como ' ' o '9'

array([' ', '2', '1'], dtype=object)

In [837]:
traduce('NOTRAB_BSQ8')

                aux_traduce
                     unique
NOTRAB_BSQ8                
             [No aplicable]
1                      [Sí]
2                      [No]


In [838]:
actualiza_btr('NOTRAB_BSQ8','btr_opo')

[ 1.  2. nan]
        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]
             btr_opo_
               unique
NOTRAB_BSQ8_         
No               [No]
Sí               [Sí]


## 'NOTRAB_BSQ9'  (completa la información de la variable 'btr_auto')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo buscando equipamiento para establecer un **negocio propio**?  

Nota: 'NOTRAB_BSQ9' recoge la misma información que la variable 'TRPARA_D7_9' definida para desempleados (redefinida como 'btr_auto'). 

In [839]:
df['NOTRAB_BSQ9'].unique() # No informados como ' ' o '9'

array([' ', '2', '1'], dtype=object)

In [840]:
traduce('NOTRAB_BSQ9')

                aux_traduce
                     unique
NOTRAB_BSQ9                
             [No aplicable]
1                      [Sí]
2                      [No]


In [841]:
actualiza_btr('NOTRAB_BSQ9','btr_auto')

[ 1.  2. nan]
         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]
             btr_auto_
                unique
NOTRAB_BSQ9_          
No                [No]
Sí                [Sí]


## 'NOTRAB_BSQ10'  (completa la información de la variable 'btr_otro')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

¿Ha buscado trabajo de **otro modo**?  

Nota: 'NOTRAB_BSQ10' recoge la misma información que la variable 'TRPARA_D7_10' definida para desempleados (redefinida como 'btr_otro'). 

In [842]:
df['NOTRAB_BSQ10'].unique() # No informados como ' ' o '9'

array([' ', '2', '1'], dtype=object)

In [843]:
traduce('NOTRAB_BSQ10')

                 aux_traduce
                      unique
NOTRAB_BSQ10                
              [No aplicable]
1                       [Sí]
2                       [No]


In [844]:
actualiza_btr('NOTRAB_BSQ10','btr_otro')

[ 1.  2. nan]
         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
              btr_otro_
                 unique
NOTRAB_BSQ10_          
No                 [No]
Sí                 [Sí]


## 'NOTRAB_DIF1'  (completa la información de la variable 'dbtr_formacion')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **carencias en la formación universitaria recibida** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF1' recoge la misma información que la variable 'TRPARA_D8_1' definida para desempleados (redefinida como 'dbtr_formacion'). 

In [845]:
df['NOTRAB_DIF1'].unique() # No informados como ' ' o '9'

array([' ', '4', '1', '2', '3', '9'], dtype=object)

In [846]:
traduce('NOTRAB_DIF1')

                       aux_traduce
                            unique
NOTRAB_DIF1                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [847]:
df[['NOTRAB_DIF1_','NOTRAB_BUS_']].groupby('NOTRAB_BUS_',dropna=False).agg(['unique']) # Inspección

,NOTRAB_DIF1_
,unique
NOTRAB_BUS_,
No,[No aplicable]
Sí,"[Muy importante, Nada importante, Poco importante, Bastante importante, NS/NC]"
NaN,[No aplicable]


In [848]:
df[['NOTRAB_DIF1_','dbtr_formacion_','sit_lab_']][df['sit_lab']==2].groupby('NOTRAB_DIF1_',
                                                                            dropna=False).agg(['unique']) 
# No hay contradicción con lo declarado en 'TRPARA_D8_1' (o 'dbtr_formacion') por los desempleados.

,dbtr_formacion_,sit_lab_
,unique,unique
NOTRAB_DIF1_,,
Bastante importante,[Bastante importante],[En desempleo]
Muy importante,[Muy importante],[En desempleo]
Nada importante,[Nada importante],[En desempleo]
No aplicable,"[Nada importante, Poco importante, Bastante importante, nan, Muy importante]",[En desempleo]
Poco importante,[Poco importante],[En desempleo]


In [849]:
def actualiza_dbtr(col,dbtr):
    """Actualiza las variables con prefijo 'dbtr_' (definidas originalmente para desempleados), añadiendo la 
    información que facilitan quienes declaran no haber trabajado desde que terminaron sus estudios y haber 
    buscado trabajo. 
    """
    no_respuesta(col)
    cond=(df[col]==1)|(df[col]==2)|(df[col]==3)|(df[col]==4)
    df[dbtr].mask(cond,df[col],inplace=True)
    traduce(dbtr)
    print (df[[str(col)+'_',str(dbtr)+'_']][df['NOTRAB_BUS']==1].groupby(str(col)+'_',
                                                                         dropna=False).agg(['unique']))
    return

In [850]:
actualiza_dbtr('NOTRAB_DIF1','dbtr_formacion')

[ 1.  2.  3.  4. nan]
                          aux_traduce
                               unique
dbtr_formacion                       
1.0                 [Nada importante]
2.0                 [Poco importante]
3.0             [Bastante importante]
4.0                  [Muy importante]
                           dbtr_formacion_
                                    unique
NOTRAB_DIF1_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF2'  (completa la información de la variable 'dbtr_personales')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **actividades personales** que impiden trabajar (seguir estudiando, familia, otras) en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF2' recoge la misma información que la variable 'TRPARA_D8_2' definida para desempleados (redefinida como 'dbtr_personales'). 

In [851]:
df['NOTRAB_DIF2'].unique() # No informados como ' ' o '9'

array([' ', '4', '1', '3', '2', '9'], dtype=object)

In [852]:
traduce('NOTRAB_DIF2')

                       aux_traduce
                            unique
NOTRAB_DIF2                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [853]:
actualiza_dbtr('NOTRAB_DIF2','dbtr_personales')

[ 1.  2.  3.  4. nan]
                           aux_traduce
                                unique
dbtr_personales                       
1.0                  [Nada importante]
2.0                  [Poco importante]
3.0              [Bastante importante]
4.0                   [Muy importante]
                          dbtr_personales_
                                    unique
NOTRAB_DIF2_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF3'  (completa la información de la variable 'dbtr_experiencia')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de experiencia profesional** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF3' recoge la misma información que la variable 'TRPARA_D8_3' definida para desempleados (redefinida como 'dbtr_experiencia'). 

In [854]:
df['NOTRAB_DIF3'].unique() # No informados como ' ' o '9'

array([' ', '4', '3', '1', '2', '9'], dtype=object)

In [855]:
traduce('NOTRAB_DIF3')

                       aux_traduce
                            unique
NOTRAB_DIF3                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [856]:
actualiza_dbtr('NOTRAB_DIF3','dbtr_experiencia')

[ 1.  2.  3.  4. nan]
                            aux_traduce
                                 unique
dbtr_experiencia                       
1.0                   [Nada importante]
2.0                   [Poco importante]
3.0               [Bastante importante]
4.0                    [Muy importante]
                         dbtr_experiencia_
                                    unique
NOTRAB_DIF3_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF4'  (completa la información de la variable 'dbtr_expectativas')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **exigencia de tener un trabajo acorde con sus expectativas** económicas y profesionales en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF4' recoge la misma información que la variable 'TRPARA_D8_4' definida para desempleados (redefinida como 'dbtr_formación'). 

In [857]:
df['NOTRAB_DIF4'].unique() # No informados como ' ' o '9'

array([' ', '9', '3', '2', '4', '1'], dtype=object)

In [858]:
traduce('NOTRAB_DIF4')

                       aux_traduce
                            unique
NOTRAB_DIF4                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [859]:
actualiza_dbtr('NOTRAB_DIF4','dbtr_expectativas')

[ 1.  2.  3.  4. nan]
                             aux_traduce
                                  unique
dbtr_expectativas                       
1.0                    [Nada importante]
2.0                    [Poco importante]
3.0                [Bastante importante]
4.0                     [Muy importante]
                        dbtr_expectativas_
                                    unique
NOTRAB_DIF4_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF5'  (completa la información de la variable 'dbtr_idiomas')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos de idiomas** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF5' recoge la misma información que la variable 'TRPARA_D8_5' definida para desempleados (redefinida como 'dbtr_idiomas'). 

In [860]:
df['NOTRAB_DIF5'].unique() # No informados como ' ' o '9'

array([' ', '4', '1', '3', '2', '9'], dtype=object)

In [861]:
traduce('NOTRAB_DIF5')

                       aux_traduce
                            unique
NOTRAB_DIF5                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [862]:
actualiza_dbtr('NOTRAB_DIF5','dbtr_idiomas')

[ 1.  2.  3.  4. nan]
                        aux_traduce
                             unique
dbtr_idiomas                       
1.0               [Nada importante]
2.0               [Poco importante]
3.0           [Bastante importante]
4.0                [Muy importante]
                             dbtr_idiomas_
                                    unique
NOTRAB_DIF5_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'NOTRAB_DIF6'  (completa la información de la variable 'dbtr_ti')

(Solo para quienes **declaran haber buscado empleo** -respuesta 1 en 'NOTRAB_BUS'-. Implica que no han trabajado después de terminar los estudios) 

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos informáticos** en cuanto a sus dificultades para encontrar trabajo.

Nota: 'NOTRAB_DIF6' recoge la misma información que la variable 'TRPARA_D8_6' definida para desempleados (redefinida como 'dbtr_ti'). 

In [863]:
df['NOTRAB_DIF6'].unique() # No informados como ' ' o '9'

array([' ', '9', '1', '3', '2', '4'], dtype=object)

In [864]:
traduce('NOTRAB_DIF6')

                       aux_traduce
                            unique
NOTRAB_DIF6                       
                    [No aplicable]
1                [Nada importante]
2                [Poco importante]
3            [Bastante importante]
4                 [Muy importante]
9                          [NS/NC]


In [865]:
actualiza_dbtr('NOTRAB_DIF6','dbtr_ti')

[ 1.  2.  3.  4. nan]
                   aux_traduce
                        unique
dbtr_ti                       
1.0          [Nada importante]
2.0          [Poco importante]
3.0      [Bastante importante]
4.0           [Muy importante]
                                  dbtr_ti_
                                    unique
NOTRAB_DIF6_                              
Bastante importante  [Bastante importante]
Muy importante            [Muy importante]
Nada importante          [Nada importante]
Poco importante          [Poco importante]
NaN                                  [nan]


## 'RECHA_TR' (redefinida como 'rechazo')

¿Ha **rechazado algún trabajo** por considerarlo poco adecuado desde que se ha titulado?

In [866]:
df['RECHA_TR'].unique() # No aparece la respuesta no aplicable: se pregunta a todos los encuestados. 
# No informados como '9'

array([1, 2, 9])

In [867]:
traduce('RECHA_TR')

         aux_traduce
              unique
RECHA_TR            
1               [Sí]
2               [No]
9            [NS/NC]


In [868]:
no_respuesta('RECHA_TR',tipo='int')
redefine('RECHA_TR','rechazo',si_no=True)

[ 1.  2. nan]
        aux_traduce
             unique
rechazo            
0.0            [No]
1.0            [Sí]


## 'RECHA_M1'  (redefinida como 'rech_estudios')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque no le permitía **seguir estudiando**?

In [869]:
df['RECHA_M1'].unique() # No informados como ' ' o '9'

array(['1', ' ', '2', '9'], dtype=object)

In [870]:
traduce('RECHA_M1')

             aux_traduce
                  unique
RECHA_M1                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [871]:
df[['rechazo_','RECHA_M1']].groupby('rechazo_',dropna=True).agg(['unique']) # Inspección

,RECHA_M1
,unique
rechazo_,
No,[ ]
Sí,"[1, 2, 9]"


In [872]:
def re_rechazo(col,nuevo):
    """ Procesa las columnas referidas a motivos de rechazo de una oferta de trabajo.
    """
    no_respuesta(col)
    redefine(col,nuevo,si_no=True)
    D_describe[nuevo]='Motivo para rechazar oferta de empleo:'+D_describe[col].split(':')[1]
    print(df[[str(col)+'_',str(nuevo)+'_']].groupby(str(col)+'_',dropna=True).agg(['unique']))
    return

In [873]:
re_rechazo('RECHA_M1','rech_estudios')

[ 1.  2. nan]
              aux_traduce
                   unique
rech_estudios            
0.0                  [No]
1.0                  [Sí]
          rech_estudios_
                  unique
RECHA_M1_               
No                  [No]
Sí                  [Sí]


## 'RECHA_M2'  (redefinida como 'rech_personales')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque el horario de trabajo era incompatible con **obligaciones personales o familiares**?

In [874]:
df['RECHA_M2'].unique() # No informados como ' ' o '9'

array(['2', ' ', '1', '9'], dtype=object)

In [875]:
traduce('RECHA_M2')

             aux_traduce
                  unique
RECHA_M2                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [876]:
re_rechazo('RECHA_M2','rech_personales')

[ 1.  2. nan]
                aux_traduce
                     unique
rech_personales            
0.0                    [No]
1.0                    [Sí]
          rech_personales_
                    unique
RECHA_M2_                 
No                    [No]
Sí                    [Sí]


## 'RECHA_M3'  (redefinida como 'rech_residencia')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque aceptarlo implicaba **cambiar de lugar de residencia**?

In [877]:
df['RECHA_M3'].unique() # No informados como ' ' o '9'

array(['2', ' ', '1', '9'], dtype=object)

In [878]:
traduce('RECHA_M3')

             aux_traduce
                  unique
RECHA_M3                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [879]:
re_rechazo('RECHA_M3','rech_residencia')

[ 1.  2. nan]
                aux_traduce
                     unique
rech_residencia            
0.0                    [No]
1.0                    [Sí]
          rech_residencia_
                    unique
RECHA_M3_                 
No                    [No]
Sí                    [Sí]


## 'RECHA_M4'  (redefinida como 'rech_distancia')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque el trayecto entre el domicilio y el puesto de trabajo no estaba a una **distancia** razonable?

In [880]:
df['RECHA_M4'].unique() # No informados como ' ' o '9'

array(['2', ' ', '1', '9'], dtype=object)

In [881]:
traduce('RECHA_M4')

             aux_traduce
                  unique
RECHA_M4                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [882]:
re_rechazo('RECHA_M4','rech_distancia')

[ 1.  2. nan]
               aux_traduce
                    unique
rech_distancia            
0.0                   [No]
1.0                   [Sí]
          rech_distancia_
                   unique
RECHA_M4_                
No                   [No]
Sí                   [Sí]


## 'RECHA_M5'  (redefinida como 'rech_retribucion')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque consideraba que **no estaba adecuadamente remunerado**?

In [883]:
df['RECHA_M5'].unique() # No informados como ' ' o '9'

array(['1', ' ', '2', '9'], dtype=object)

In [884]:
traduce('RECHA_M5')

             aux_traduce
                  unique
RECHA_M5                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [885]:
re_rechazo('RECHA_M5','rech_retribucion')

[ 1.  2. nan]
                 aux_traduce
                      unique
rech_retribucion            
0.0                     [No]
1.0                     [Sí]
          rech_retribucion_
                     unique
RECHA_M5_                  
No                     [No]
Sí                     [Sí]


## 'RECHA_M6'  (redefinida como 'rech_formacion')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque el nivel del puesto de trabajo **no estaba acorde con sus habilidades y conocimientos**?

In [886]:
df['RECHA_M6'].unique() # No informados como ' ' o '9'

array(['2', ' ', '1', '9'], dtype=object)

In [887]:
traduce('RECHA_M6')

             aux_traduce
                  unique
RECHA_M6                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [888]:
re_rechazo('RECHA_M6','rech_formacion')

[ 1.  2. nan]
               aux_traduce
                    unique
rech_formacion            
0.0                   [No]
1.0                   [Sí]
          rech_formacion_
                   unique
RECHA_M6_                
No                   [No]
Sí                   [Sí]


## 'RECHA_M7'  (redefinida como 'rech_alternativa')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque tenía **otra oferta de trabajo o ya estaba trabajando**?

In [889]:
df['RECHA_M7'].unique() # No informados como ' ' o '9'

array(['1', ' ', '2', '9'], dtype=object)

In [890]:
traduce('RECHA_M7')

             aux_traduce
                  unique
RECHA_M7                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [891]:
re_rechazo('RECHA_M7','rech_alternativa')

[ 1.  2. nan]
                 aux_traduce
                      unique
rech_alternativa            
0.0                     [No]
1.0                     [Sí]
          rech_alternativa_
                     unique
RECHA_M7_                  
No                     [No]
Sí                     [Sí]


## 'RECHA_M8'  (redefinida como 'rech_opo')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo porque **estaba preparando oposiciones**?

In [892]:
df['RECHA_M8'].unique() # No informados como ' ' o '9'

array(['2', ' ', '1', '9'], dtype=object)

In [893]:
traduce('RECHA_M8')

             aux_traduce
                  unique
RECHA_M8                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [894]:
re_rechazo('RECHA_M8','rech_opo')

[ 1.  2. nan]
         aux_traduce
              unique
rech_opo            
0.0             [No]
1.0             [Sí]
          rech_opo_
             unique
RECHA_M8_          
No             [No]
Sí             [Sí]


## 'RECHA_M9'  (redefinida como 'rech_otro')

(Solo para quienes **declaran haber rechazado una oferta de trabajo** -respuesta 1 en 'RECHA_TR' o 'rechazo'-)  

(**Motivo del rechazo**) ¿Ha rechazado una oferta de trabajo por **otros motivos**?

In [895]:
df['RECHA_M9'].unique() # No informados como ' ' o '9'

array(['2', ' ', '1', '9'], dtype=object)

In [896]:
traduce('RECHA_M9')

             aux_traduce
                  unique
RECHA_M9                
          [No aplicable]
1                   [Sí]
2                   [No]
9                [NS/NC]


In [897]:
re_rechazo('RECHA_M9','rech_otro')

[ 1.  2. nan]
          aux_traduce
               unique
rech_otro            
0.0              [No]
1.0              [Sí]
          rech_otro_
              unique
RECHA_M9_           
No              [No]
Sí              [Sí]


# BLOQUE F. INFORMACIÓN DEL HOGAR

## 'HO_F1'  (redefinida como 'e_civil')

**Estado civil**.

In [898]:
df['HO_F1'].unique() # No informados como '9'

array([1, 2, 4, 5, 9, 3])

In [899]:
traduce('HO_F1')

        aux_traduce
             unique
HO_F1              
1         [Soltero]
2          [Casado]
3           [Viudo]
4        [Separado]
5      [Divorciado]
9           [NS/NC]


In [900]:
no_respuesta('HO_F1',tipo='int')
redefine('HO_F1','e_civil')

[ 1.  2.  3.  4.  5. nan]


In [901]:
df['e_civil_'].unique()

array(['Soltero', 'Casado', 'Separado', 'Divorciado', nan, 'Viudo'],
      dtype=object)

## 'HO_PAREJ'  (redefinida como 'pareja')

¿Convive en pareja?

In [902]:
df['HO_PAREJ'].unique() # No informados como '9'

array([2, 1, 9])

In [903]:
traduce('HO_PAREJ')

         aux_traduce
              unique
HO_PAREJ            
1               [Sí]
2               [No]
9            [NS/NC]


In [904]:
no_respuesta('HO_PAREJ',tipo='int')
redefine('HO_PAREJ','pareja',si_no=True)

[ 1.  2. nan]
       aux_traduce
            unique
pareja            
0.0           [No]
1.0           [Sí]


## 'HO_NPER'  (redefinida como 'n_hogar')

**Número de personas que viven en el hogar**, incluyendo al entrevistado. 

In [905]:
df['HO_NPER'].unique() # No informados como '9'

array([3, 2, 1, 4, 5, 6, 9])

In [906]:
traduce('HO_NPER')

        aux_traduce
             unique
HO_NPER            
1               [1]
2               [2]
3               [3]
4               [4]
5               [5]
6         [6 o más]
9           [NS/NC]


In [907]:
no_respuesta('HO_NPER',tipo='int')
redefine('HO_NPER','n_hogar')

[ 1.  2.  3.  4.  5.  6. nan]


## 'HO_NPER_TR'  (redefinida como 'n_hogar_tr')

**Número de personas en el hogar que trabajan** actualmente. 

In [908]:
df['HO_NPER_TR'].unique() # No informados como '9'

array([3, 1, 2, 5, 0, 4, 9, 6])

In [909]:
traduce('HO_NPER_TR')

           aux_traduce
                unique
HO_NPER_TR            
0                  [0]
1                  [1]
2                  [2]
3                  [3]
4                  [4]
5                  [5]
6            [6 o más]
9              [NS/NC]


In [910]:
no_respuesta('HO_NPER_TR',tipo='int')
redefine('HO_NPER_TR','n_hogar_tr')

[ 0.  1.  2.  3.  4.  5.  6. nan]


## 'HO_TIPO'  

**Tipo de hogar**.

In [911]:
df['HO_TIPO'].unique() # No informados como '9'

array([8, 4, 1, 6, 5, 3, 2, 7, 9])

In [912]:
traduce('HO_TIPO')

                                                                                     aux_traduce
                                                                                          unique
HO_TIPO                                                                                         
1                                                                            [Hogar unipersonal]
2                             [Padre o madre solo/a que convive con algún hijo menor de 25 años]
3                        [Padre o madre solo/a que convive con todos sus hijos de 25 años o más]
4                                                    [Pareja sin hijos que convivan en el hogar]
5                            [Pareja con algún hijo menor de 25 años y que conviven en el hogar]
6           [Pareja con algún hijo menor pero todos de 25 años ó más y que conviven en el hogar]
7        [Pareja, o un padre/madre, que conviven con hijos menores de 25 años, y otras personas]
8                             

In [913]:
no_respuesta('HO_TIPO')

[1. 2. 3. 4. 5. 6. 7. 8. 9.]


# BLOQUE VARIABLES ADICIONALES

Se omite.

## Organización de archivos csv

In [914]:
df[v_originales+v_originales_L].to_csv('data/Master_19_procesado.csv')

In [915]:
df[v_modelos+v_modelos_L].to_csv('data/Master_19_modelos.csv')

In [916]:
len(v_originales),len(v_originales_L),len(v_modelos),len(v_modelos_L)

(291, 224, 143, 132)

In [917]:
df_dict = pd.DataFrame.from_dict(D_dict, orient='columns')
df_dict.to_csv('data/D_dict.csv')

In [918]:
df_describe = pd.DataFrame.from_dict(D_describe, orient='index',columns=['valor'])
df_describe.to_csv('data/D_describe.csv')